In [4]:
from pathlib import Path

# ⬇️ Change this if your CSVs are in another folder
DATA_DIR = Path(".")            # e.g., Path("/path/to/your/folder")

PATHS_CSV   = DATA_DIR / "eon_paths_timeseries.csv"
LINKS_CSV   = DATA_DIR / "eon_links_timeseries.csv"
FEWSHOT_CSV = DATA_DIR / "eon_target_fewshot.csv"

OUT_DIR = Path("./outputs")
(OUT_DIR / "figs").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "metrics").mkdir(parents=True, exist_ok=True)

print("Looking for files:")
for p in [PATHS_CSV, LINKS_CSV, FEWSHOT_CSV]:
    print(f"  {p}  ->  {'OK' if p.exists() else 'NOT FOUND'}")

# Hard stop if any file is missing
assert PATHS_CSV.exists() and LINKS_CSV.exists() and FEWSHOT_CSV.exists(), "Place all 3 CSVs in DATA_DIR."


Looking for files:
  eon_paths_timeseries.csv  ->  OK
  eon_links_timeseries.csv  ->  OK
  eon_target_fewshot.csv  ->  OK


In [5]:
import pandas as pd

paths = pd.read_csv(PATHS_CSV)
links = pd.read_csv(LINKS_CSV)
few   = pd.read_csv(FEWSHOT_CSV)

print("Loaded shapes:")
print("  paths :", paths.shape)
print("  links :", links.shape)
print("  fewshot:", few.shape)

display(paths.head(3))
display(links.head(3))
display(few.head(3))


Loaded shapes:
  paths : (10800, 24)
  links : (5310, 16)
  fewshot: (600, 24)


,topology,day,src,dst,path,hops,distance_km,latency_ms,avg_utilization,min_osnr_db,...,symbol_rate_gbaud,bitrate_gbps,req_osnr_db,osnr_margin_db,est_ber,qot_ok,failure_present,failure_type,fail_link,split
0,NSFNET,0,1,13,1->8->10->11->12->13,5,1403.889557,7.019448,0.787454,22.537527,...,64.0,256.0,19.8,2.737527,0.003510,1,0,none,NaN,train_source
1,NSFNET,0,14,1,14->11->10->8->1,4,1032.394401,5.161972,0.795083,22.537527,...,32.0,64.0,12.5,10.037527,0.000012,1,0,none,NaN,train_source
2,NSFNET,0,5,8,5->7->8,2,811.524544,4.057623,0.636687,23.277962,...,40.0,80.0,12.5,10.777962,0.000007,1,0,none,NaN,train_source


,topology,day,u,v,edge_id,length_km,osnr_db,snr_db,signal_dbm,noise_dbm,center_freq_offset_ghz,filter_bw_scale,bandwidth_utilization,latency_ms,failure_active,failure_type
0,NSFNET,0,1,2,1-2,474.507123,23.855619,20.355619,-1.05308,-24.908699,0.0,1.0,0.787887,2.372536,0,none
1,NSFNET,1,1,2,1-2,474.507123,23.923705,20.423705,-1.05308,-24.976784,0.0,1.0,0.920905,2.372536,0,none
2,NSFNET,2,1,2,1-2,474.507123,24.722090,21.222090,-1.05308,-25.775170,0.0,1.0,0.782731,2.372536,0,none


,topology,day,src,dst,path,hops,distance_km,latency_ms,avg_utilization,min_osnr_db,...,symbol_rate_gbaud,bitrate_gbps,req_osnr_db,osnr_margin_db,est_ber,qot_ok,failure_present,failure_type,fail_link,split
0,GEANT2,62,7,13,7->19->14->13,3,776.881395,3.884407,0.768215,21.194588,...,32.0,192.0,25.0,-3.805412,0.009921,0,0,none,NaN,test_target
1,GEANT2,6,14,13,14->13,1,104.679224,0.523396,0.676073,26.576763,...,40.0,240.0,25.8,0.776763,0.007348,0,0,none,NaN,test_target
2,GEANT2,63,23,22,23->19->14->5->4->22,5,1153.974412,5.769872,0.728540,21.304553,...,32.0,192.0,25.0,-3.695447,0.009914,0,0,none,NaN,test_target


In [3]:
pip install pandas


   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.0 MB 375.2 kB/s eta 0:00:28
   ---- ----------------------------------- 1.3/11.0 MB 604.1 kB/s eta 0:00:17
   ----- ---------------------------------- 1.6/11.0 MB 689.4 kB/s eta 0:00:14
   -------- ------------------------------- 2.4/11.0 MB 983.5 kB/s eta 0:00:09
   ---------- ----------------------------- 2.9/11.0 MB 1.1 MB/s eta 0:00:08
   -------------- ------------------------- 3.9

In [5]:
def split_counts(df):
    return df['split'].value_counts().to_frame('rows')

def class_balance(df):
    return {
        'qot_ok': df['qot_ok'].value_counts().to_dict(),
        'failure_present': df['failure_present'].value_counts().to_dict(),
        'failure_type': df['failure_type'].value_counts().to_dict(),
    }

print("Split counts (paths):")
display(split_counts(paths))

print("Class balance (paths):")
display(class_balance(paths))


Split counts (paths):


,rows
split,
test_target,5400
train_source,4560
val_source,840


Class balance (paths):


{'qot_ok': {1: 8323, 0: 2477},
 'failure_present': {0: 9577, 1: 1223},
 'failure_type': {'none': 9577, 'shift': 661, 'tighten': 562}}

In [ ]:
%pip uninstall -y sklearn
%pip install -U scikit-learn

# verify
import sklearn, sys
print("sklearn version:", sklearn.__version__)
print("python exe:", sys.executable)


Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.7 MB 1.4 MB/s eta 0:00:06
   ---- ----------------------------------- 1.0/8.7 MB 1.5 MB/s eta 0:00:06
   ------- -------------------------------- 1.6/8.7 MB 1.7 MB/s eta 0:00:05
   -------- ------------------------------- 1.8/8.7 MB 1.8 MB/s eta 0:00:04
   ---------- ----------------------------- 2.4/8.7 MB 1.9 MB/s eta 0:00:04
   -------------- ------------------------- 3.1/8.7 MB 2.1 MB/s eta 0:00:03
   ------------------ --------------------- 3.9/8.7 MB 2.3 MB/s eta 0:00:03
   --------------------- ------------------ 4.7/8.7 MB 2.5 MB/s eta 0:00:02
   ------------------------- -------------- 5.5/8.7 MB 2.6 MB/s eta 0:00:02
   --------------------------- ------------ 6.0/8.7 MB 2.7 MB/s eta 0:00:01
   ------------------------------

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# --- Split helper
def get_splits(df):
    train = df[df['split']=="train_source"].copy()
    val   = df[df['split']=="val_source"].copy()
    test  = df[df['split']=="test_target"].copy()
    return train, val, test

# --- Feature maker (numerics + modulation code)
NUM_COLS = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps','req_osnr_db','osnr_margin_db'
]
CAT_COLS = ['modulation']

def make_xy(df):
    X = df[NUM_COLS + CAT_COLS].copy()
    X['modulation'] = X['modulation'].astype('category').cat.codes
    y = df['qot_ok'].astype(int).values
    return X, y

# --- Train/eval
train, val, test = get_splits(paths)
Xtr, ytr = make_xy(train)
Xva, yva = make_xy(val)
Xte, yte = make_xy(test)

scaler = StandardScaler()
Xtr = scaler.fit_transform(Xtr)
Xva = scaler.transform(Xva)
Xte = scaler.transform(Xte)

clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                    alpha=1e-4, learning_rate_init=1e-3,
                    max_iter=200, random_state=42)
clf.fit(Xtr, ytr)

yte_pred = clf.predict(Xte)
yte_proba = clf.predict_proba(Xte)[:,1]

print("QoT baseline (NSFNET→GEANT2 zero-shot):")
print("  Accuracy:", accuracy_score(yte, yte_pred))
print("  F1 score:", f1_score(yte, yte_pred, average="macro"))
print("  AUC:", roc_auc_score(yte, yte_proba))

QoT baseline (NSFNET→GEANT2 zero-shot):
  Accuracy: 0.9792592592592593
  F1 score: 0.972386542384192
  AUC: 0.99828865139197


In [12]:
# NSFNET→GEANT2 zero-shot failure_present (0/1) with safer preprocessing

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

# --- helper: splits from `paths`
def get_splits(df):
    train = df[df['split']=="train_source"].copy()
    val   = df[df['split']=="val_source"].copy()
    test  = df[df['split']=="test_target"].copy()
    return train, val, test

# --- features
NUM_COLS = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps','req_osnr_db','osnr_margin_db'
]
CAT_COLS = ['modulation']

# --- split
train, val, test = get_splits(paths)

Xtr, ytr = train[NUM_COLS + CAT_COLS], train['failure_present'].astype(int).values
Xva, yva = val[NUM_COLS + CAT_COLS],   val['failure_present'].astype(int).values
Xte, yte = test[NUM_COLS + CAT_COLS],  test['failure_present'].astype(int).values

# --- preprocessors (no leakage; OHE is robust to unseen categories)
pre = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), NUM_COLS),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), CAT_COLS),
    ],
    remainder="drop"
)

# --- model (early stopping uses an internal split on the training data)
clf = MLPClassifier(
    hidden_layer_sizes=(256, 256),
    activation="relu",
    alpha=1e-4,
    learning_rate_init=1e-3,
    max_iter=400,
    random_state=42,
    early_stopping=True,
    n_iter_no_change=20,
    validation_fraction=0.15
)

pipe = Pipeline(steps=[("pre", pre), ("clf", clf)])

# (optional) handle class imbalance on train via sample weights
wtr = compute_sample_weight(class_weight="balanced", y=ytr)

# --- fit on source (NSFNET)
pipe.fit(Xtr, ytr, clf__sample_weight=wtr)

# --- evaluate on source-val and target-test (GEANT2)
def eval_and_print(name, X, y):
    y_pred = pipe.predict(X)
    print(f"{name}: "
          f"acc={accuracy_score(y, y_pred):.4f}, "
          f"macro-F1={f1_score(y, y_pred, average='macro'):.4f}, "
          f"bal-acc={balanced_accuracy_score(y, y_pred):.4f}")

print("Failure Detection (NSFNET→GEANT2 zero-shot):")
eval_and_print("val_source", Xva, yva)
eval_and_print("test_target (GEANT2)", Xte, yte)


Failure Detection (NSFNET→GEANT2 zero-shot):
val_source: acc=1.0000, macro-F1=1.0000, bal-acc=1.0000
test_target (GEANT2): acc=0.9954, macro-F1=0.9907, bal-acc=0.9915


C:\Users\devon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [14]:
# Sanity check: why detection is ~perfect, and how it changes if we drop fingerprint features.

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import numpy as np
import pandas as pd

# --- splits
def get_splits(df):
    tr = df[df['split']=="train_source"].copy()
    va = df[df['split']=="val_source"].copy()
    te = df[df['split']=="test_target"].copy()
    return tr, va, te

train, val, test = get_splits(paths)

# ----- helpers
NUM_COLS_ALL = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps','req_osnr_db','osnr_margin_db'
]
CAT_COLS = ['modulation']

def make_xy(df, drop_fingerprints=False):
    cols = NUM_COLS_ALL.copy()
    if drop_fingerprints:
        # remove direct failure fingerprints
        cols.remove('max_center_offset_ghz')
        cols.remove('min_filter_bw_scale')
    X = df[cols + CAT_COLS].copy()
    X['modulation'] = X['modulation'].astype('category').cat.codes
    y = df['failure_present'].astype(int).values
    return X, y

def train_eval(drop_fingerprints=False):
    Xtr, ytr = make_xy(train, drop_fingerprints)
    Xva, yva = make_xy(val, drop_fingerprints)
    Xte, yte = make_xy(test, drop_fingerprints)
    scaler = StandardScaler()
    Xtr = scaler.fit_transform(Xtr)
    Xte = scaler.transform(Xte)
    clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                        alpha=1e-4, learning_rate_init=1e-3,
                        max_iter=200, random_state=42)
    clf.fit(Xtr, ytr)
    ypred = clf.predict(Xte)
    acc = accuracy_score(yte, ypred)
    f1  = f1_score(yte, ypred, average='macro')
    return acc, f1, yte, ypred

print("Class balance on GEANT2 test (failure_present):")
print(test['failure_present'].value_counts().to_dict(), "\n")

# (A) Model WITH fingerprints (what you ran)
accA, f1A, y_true, y_predA = train_eval(drop_fingerprints=False)
print("A) With fingerprints  -> Acc=%.4f  Macro-F1=%.4f" % (accA, f1A))
print("Confusion matrix (A):\n", confusion_matrix(y_true, y_predA), "\n")

# (B) Simple rule baseline (offset>0 OR scale<0.999)
rule_pred = ((test['max_center_offset_ghz']>0).astype(int) | (test['min_filter_bw_scale']<0.999).astype(int)).values
accR = accuracy_score(y_true, rule_pred)
f1R  = f1_score(y_true, rule_pred, average='macro')
print("B) Heuristic rule     -> Acc=%.4f  Macro-F1=%.4f" % (accR, f1R))
print("Confusion matrix (B):\n", confusion_matrix(y_true, rule_pred), "\n")

# (C) Model WITHOUT fingerprints (harder, more realistic if telemetry is incomplete)
accC, f1C, _, y_predC = train_eval(drop_fingerprints=True)
print("C) Drop fingerprints  -> Acc=%.4f  Macro-F1=%.4f" % (accC, f1C))
print("Confusion matrix (C):\n", confusion_matrix(y_true, y_predC))


Class balance on GEANT2 test (failure_present):
{0: 4610, 1: 790} 

A) With fingerprints  -> Acc=0.9969  Macro-F1=0.9937
Confusion matrix (A):
 [[4606    4]
 [  13  777]] 

B) Heuristic rule     -> Acc=0.9807  Macro-F1=0.9634
Confusion matrix (B):
 [[4506  104]
 [   0  790]] 

C) Drop fingerprints  -> Acc=0.8385  Macro-F1=0.4880
Confusion matrix (C):
 [[4498  112]
 [ 760   30]]


C:\Users\devon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [15]:
# Predict failure_type: {'none','shift','tighten'} on GEANT2 using a model trained on NSFNET.

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_sample_weight

# --- splits
def get_splits(df):
    tr = df[df['split']=="train_source"].copy()
    va = df[df['split']=="val_source"].copy()
    te = df[df['split']=="test_target"].copy()
    return tr, va, te

train, val, test = get_splits(paths)

# --- target as categorical codes with fixed order
ORDER = ['none','shift','tighten']
cat_type = pd.api.types.CategoricalDtype(categories=ORDER, ordered=False)

# --- features
NUM_ALL = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps','req_osnr_db','osnr_margin_db'
]
CAT = ['modulation']

def make_xy(df, drop_fingerprints=False):
    cols = NUM_ALL.copy()
    if drop_fingerprints:
        cols.remove('max_center_offset_ghz')
        cols.remove('min_filter_bw_scale')
    X = df[cols + CAT].copy()
    X['modulation'] = X['modulation'].astype('category').cat.codes
    y = df['failure_type'].astype(cat_type).cat.codes.values  # 0=none,1=shift,2=tighten
    return X, y

def run(drop_fingerprints=False, seed=42):
    Xtr, ytr = make_xy(train, drop_fingerprints)
    Xva, yva = make_xy(val, drop_fingerprints)
    Xte, yte = make_xy(test, drop_fingerprints)

    scaler = StandardScaler()
    Xtr = scaler.fit_transform(Xtr); Xva = scaler.transform(Xva); Xte = scaler.transform(Xte)

    # class-balanced sample weights (important due to 'none' majority)
    sw = compute_sample_weight(class_weight='balanced', y=ytr)

    clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                        alpha=1e-4, learning_rate_init=1e-3,
                        max_iter=200, random_state=seed)
    clf.fit(Xtr, ytr, sample_weight=sw)

    ypred = clf.predict(Xte)
    macro_f1 = f1_score(yte, ypred, average='macro')
    print(("WITH" if not drop_fingerprints else "DROP") + " fingerprints ⇒ Macro-F1=%.4f" % macro_f1)
    print("Confusion matrix (rows=GT, cols=Pred) order", ORDER)
    print(confusion_matrix(yte, ypred))
    print(classification_report(yte, ypred, target_names=ORDER, digits=4))

print("Failure Type (NSFNET→GEANT2 zero-shot):")
print("\n(A) Easy mode (WITH fingerprints)")
run(drop_fingerprints=False)

print("\n(B) Hard mode (DROP fingerprints)")
run(drop_fingerprints=True)


Failure Type (NSFNET→GEANT2 zero-shot):

(A) Easy mode (WITH fingerprints)
WITH fingerprints ⇒ Macro-F1=0.9085
Confusion matrix (rows=GT, cols=Pred) order ['none', 'shift', 'tighten']
[[4605    1    4]
 [   2  379   98]
 [   9    0  302]]
              precision    recall  f1-score   support

        none     0.9976    0.9989    0.9983      4610
       shift     0.9974    0.7912    0.8824       479
     tighten     0.7475    0.9711    0.8448       311

    accuracy                         0.9789      5400
   macro avg     0.9142    0.9204    0.9085      5400
weighted avg     0.9832    0.9789    0.9791      5400


(B) Hard mode (DROP fingerprints)
DROP fingerprints ⇒ Macro-F1=0.3272
Confusion matrix (rows=GT, cols=Pred) order ['none', 'shift', 'tighten']
[[4254   54  302]
 [ 444    0   35]
 [ 279    0   32]]
              precision    recall  f1-score   support

        none     0.8547    0.9228    0.8875      4610
       shift     0.0000    0.0000    0.0000       479
     tighten     0

C:\Users\devon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [17]:
# === GEANT2 heuristic localization (compact single cell) ===
# Assumes DataFrames: paths, links

import re, numpy as np, pandas as pd

# ---- config (edit if needed) ----
OFFSET_SCALE = 20.0      # GHz scale for offset term
TIME_COL     = None      # e.g., 'time_idx' if both tables have it; else keep None
LAG          = 0         # set 1 to use telemetry at t-1 (avoids peeking)
ZSCORE       = False     # set True to z-score per (topology,day[,time])

# ---- helpers ----
def norm_eid(e):
    s = str(e).strip().replace("(","").replace(")","").replace("[","").replace("]","")
    a,b = map(int, re.findall(r"\d+", s)[:2])
    return f"{min(a,b)}-{max(a,b)}"

def path_to_eids(p):
    ns = list(map(int, re.findall(r"\d+", str(p))))
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(ns[:-1], ns[1:])]

def zscore_in_groups(df, cols, by):
    out = df.copy()
    for c in cols:
        g = out.groupby(by)[c]
        mu = g.transform("mean")
        sd = g.transform("std").replace(0, np.nan).fillna(1.0)
        out[c] = (out[c] - mu) / sd
    return out

# ---- prep links ----
lk = links.copy()
if "edge_id" not in lk.columns:
    assert {"u","v"}.issubset(lk.columns), "links must have edge_id or (u,v)"
    lk["edge_id"] = lk.apply(lambda r: norm_eid((r["u"], r["v"])), axis=1)

if lk["bandwidth_utilization"].max(skipna=True) > 1.0:
    lk["bandwidth_utilization"] = lk["bandwidth_utilization"].astype(float) / 100.0

keys = ["topology","day"]
if TIME_COL and (TIME_COL in lk.columns) and (TIME_COL in paths.columns):
    if LAG: lk = lk.copy(); lk[TIME_COL] = lk[TIME_COL] + int(LAG)
    keys.append(TIME_COL)

# ---- filter GEANT2 failures ----
df = paths.loc[
    (paths["split"]=="test_target") &
    (paths["failure_present"]==1) &
    (paths["fail_link"].astype(str).str.len()>0)
].copy().reset_index(drop=True)

df["gt_eid"] = df["fail_link"].map(norm_eid)
df["eids"]   = df["path"].map(path_to_eids)
df["row_id"] = df.index

# ---- explode edges & join telemetry ----
cand = (df[["row_id"]+keys+["eids"]]
        .explode("eids").rename(columns={"eids":"edge_id"}))
cand = cand.merge(
    lk[keys+["edge_id","center_freq_offset_ghz","filter_bw_scale","bandwidth_utilization"]],
    on=keys+["edge_id"], how="left", validate="many_to_one"
)

# ---- penalty ----
cand["offset"] = cand["center_freq_offset_ghz"].astype(float)
cand["scale"]  = cand["filter_bw_scale"].astype(float).clip(lower=1e-6)
cand["util"]   = cand["bandwidth_utilization"].astype(float).clip(0,1)

if ZSCORE:
    tmp = cand.copy()
    tmp["invscale_minus1"] = 1.0/tmp["scale"] - 1.0
    tmp = zscore_in_groups(tmp, ["offset","invscale_minus1"], by=keys)
    cand["pen"] = (tmp["offset"])**2 + tmp["invscale_minus1"].clip(lower=0)*cand["util"]
else:
    cand["pen"] = (cand["offset"]/float(OFFSET_SCALE))**2 + (1.0/cand["scale"] - 1.0).clip(lower=0)*cand["util"]

# ---- pick Top-1 / Top-3 per path ----
cand["rank"]  = cand.groupby("row_id")["pen"].rank(ascending=False, method="first")
pred_top1     = cand.loc[cand["rank"]==1, ["row_id","edge_id"]].rename(columns={"edge_id":"pred"})
top3_list     = cand.loc[cand["rank"]<=3].groupby("row_id")["edge_id"].apply(list).rename("top3")

res = df[["row_id","gt_eid","eids"]].merge(pred_top1, on="row_id", how="left").merge(top3_list, on="row_id", how="left")

# exclude rows where no telemetry matched (no prediction)
res_eval = res.dropna(subset=["pred"]).copy()
misses   = len(res) - len(res_eval)

# ---- metrics ----
top1 = (res_eval["pred"]==res_eval["gt_eid"]).mean()
top3 = res_eval.apply(lambda r: r["gt_eid"] in (r["top3"] or []), axis=1).mean()
def hop_err(r):
    try: return abs(r["eids"].index(r["pred"]) - r["eids"].index(r["gt_eid"]))
    except: return np.nan
res_eval["hop_err"] = res_eval.apply(hop_err, axis=1)
mhe = float(np.nanmean(res_eval["hop_err"]))

# diagnostic: is GT the per-row max penalty? (just to verify no "fixing")
mx   = cand.groupby("row_id")["pen"].max()
gt_p = (cand.merge(res_eval[["row_id","gt_eid"]], left_on=["row_id","edge_id"], right_on=["row_id","gt_eid"], how="inner"))["pen"]
gt_frac = float(np.mean(np.isclose(gt_p.values, mx.loc[res_eval["row_id"]].values))) if len(gt_p) else float("nan")

print(f"Evaluated rows (GEANT2 failures): {len(res_eval)}  (skipped due to missing joins: {misses})")
print(f"Top-1 accuracy: {top1:.3f} | Top-3 accuracy: {top3:.3f} | Mean hop-error: {mhe:.2f}")
print(f"GT equals per-row max penalty (fraction): {gt_frac:.3f}")


Evaluated rows (GEANT2 failures): 790  (skipped due to missing joins: 0)
Top-1 accuracy: 1.000 | Top-3 accuracy: 1.000 | Mean hop-error: 0.00
GT equals per-row max penalty (fraction): 1.000


In [18]:
# 1) Ties check: kitni rows me max penalty par tie tha?
ties = cand.groupby("row_id").apply(lambda g: (g["pen"]==g["pen"].max()).sum()).gt(1).sum()
print("Rows with ties at max penalty:", int(ties))


Rows with ties at max penalty: 0


C:\Users\devon\AppData\Local\Temp\ipykernel_15148\498411556.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ties = cand.groupby("row_id").apply(lambda g: (g["pen"]==g["pen"].max()).sum()).gt(1).sum()


In [21]:
# Simple QoT-guided rerouting on GEANT2 (test_target)
# Requirements: networkx
import numpy as np, pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

try:
    import networkx as nx
except ImportError as e:
    raise ImportError("Please install networkx: pip install networkx")

# --- 1) Ensure we have a QoT model (reuse if already trained earlier)
def ensure_qot_model(paths_df):
    NUM_COLS = [
        'hops','distance_km','latency_ms',
        'avg_utilization','min_osnr_db','min_snr_db',
        'max_center_offset_ghz','min_filter_bw_scale',
        'symbol_rate_gbaud','bitrate_gbps','req_osnr_db','osnr_margin_db'
    ]
    CAT_COLS = ['modulation']

    def get_splits(df):
        tr = df[df['split']=="train_source"].copy()
        va = df[df['split']=="val_source"].copy()
        te = df[df['split']=="test_target"].copy()
        return tr, va, te

    def make_xy(df):
        X = df[NUM_COLS + CAT_COLS].copy()
        X['modulation'] = X['modulation'].astype('category').cat.codes
        y = df['qot_ok'].astype(int).values
        return X, y

    # try to reuse existing clf/scaler
    if 'clf' in globals() and 'scaler' in globals():
        return globals()['clf'], globals()['scaler'], NUM_COLS, CAT_COLS

    train, val, test = get_splits(paths_df)
    Xtr, ytr = make_xy(train)

    scaler_ = StandardScaler()
    Xtr = scaler_.fit_transform(Xtr)

    clf_ = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                         alpha=1e-4, learning_rate_init=1e-3,
                         max_iter=200, random_state=42)
    clf_.fit(Xtr, ytr)

    return clf_, scaler_, NUM_COLS, CAT_COLS

clf, scaler, NUM_COLS, CAT_COLS = ensure_qot_model(paths)

# --- 2) Utility: build per-day GEANT2 graph from links CSV
def build_graph_for_day(day):
    sub = links[(links['topology']=='GEANT2') & (links['day']==day)]
    G = nx.Graph()
    for _, r in sub.iterrows():
        u, v = int(r['u']), int(r['v'])
        G.add_edge(u, v,
                   edge_id=f"{min(u,v)}-{max(u,v)}",
                   length_km=float(r['length_km']),
                   latency_ms=float(r['latency_ms']))
    return G, sub.set_index('edge_id')

# --- 3) Aggregate features for an arbitrary candidate path (keeping tx settings same as original)
def agg_features_for_path(eids, day, tx_row, links_idx):
    rows = links_idx.loc[eids]
    # Aggregates from link telemetry (same day)
    hops = len(eids)
    distance_km = float(rows['length_km'].sum())
    latency_ms  = float(rows['latency_ms'].sum())
    avg_util    = float(rows['bandwidth_utilization'].mean())
    min_osnr    = float(rows['osnr_db'].min())
    min_snr     = float(rows['snr_db'].min())
    max_shift   = float(rows['center_freq_offset_ghz'].max())
    min_scale   = float(rows['filter_bw_scale'].min())

    # Keep original transponder settings (assumption)
    mod  = tx_row['modulation']
    srb  = tx_row['symbol_rate_gbaud']
    br   = tx_row['bitrate_gbps']
    req  = tx_row['req_osnr_db']
    margin = min_osnr - req

    # Build feature row in the model's schema
    X = pd.DataFrame([{
        'hops':hops,'distance_km':distance_km,'latency_ms':latency_ms,
        'avg_utilization':avg_util,'min_osnr_db':min_osnr,'min_snr_db':min_snr,
        'max_center_offset_ghz':max_shift,'min_filter_bw_scale':min_scale,
        'symbol_rate_gbaud':srb,'bitrate_gbps':br,'req_osnr_db':req,'osnr_margin_db':margin,
        'modulation':mod
    }])
    # encode modulation
    X['modulation'] = X['modulation'].astype('category').cat.codes
    return X

# --- 4) Reroute simulation on a sample of GEANT2 test rows
test_gea = paths[(paths['split']=='test_target') & (paths['topology']=='GEANT2')].copy()

# sample to keep it fast; you can increase n
sample = test_gea.sample(n=min(500, len(test_gea)), random_state=7).reset_index(drop=True)

K = 3  # try up to K shortest alternate paths
salvaged = 0
considered = 0
extra_dist, extra_lat = [], []

def edge_ids_from_nodes(nodes):
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(nodes[:-1], nodes[1:])]

for _, row in sample.iterrows():
    day = int(row['day'])
    G, links_idx = build_graph_for_day(day)

    # original path as nodes & eids
    nodes = [int(x) for x in row['path'].split('->')]
    orig_eids = edge_ids_from_nodes(nodes)

    # original prediction
    Xorig = pd.DataFrame([row[NUM_COLS + CAT_COLS]])
    Xorig['modulation'] = Xorig['modulation'].astype('category').cat.codes
    Xorig_scaled = scaler.transform(Xorig)
    pred_orig = int(clf.predict(Xorig_scaled)[0])

    if pred_orig == 1:
        continue  # already good; no reroute needed

    considered += 1

    # k-shortest paths by length; skip identical to original
    try:
        # precompute edge weights from link length
        for u, v, d in G.edges(data=True):
            pass
        kpaths_nodes = []
        for k, p in enumerate(nx.shortest_simple_paths(G, nodes[0], nodes[-1], weight=lambda u,v,d: d['length_km'])):
            if k >= K+1: break
            if p == nodes:
                continue
            kpaths_nodes.append(p)
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        continue

    rerouted = False
    for cand_nodes in kpaths_nodes:
        cand_eids = edge_ids_from_nodes(cand_nodes)
        # features for candidate using same TX settings as original row
        Xcand = agg_features_for_path(cand_eids, day, row, links_idx)
        Xcand_scaled = scaler.transform(Xcand[NUM_COLS + CAT_COLS].assign(modulation= Xcand['modulation']))
        pred_cand = int(clf.predict(Xcand_scaled)[0])
        if pred_cand == 1:
            # compute overheads
            orig_dist = float(links_idx.loc[orig_eids]['length_km'].sum())
            cand_dist = float(links_idx.loc[cand_eids]['length_km'].sum())
            orig_lat  = float(links_idx.loc[orig_eids]['latency_ms'].sum())
            cand_lat  = float(links_idx.loc[cand_eids]['latency_ms'].sum())
            extra_dist.append(cand_dist - orig_dist)
            extra_lat.append(cand_lat - orig_lat)
            salvaged += 1
            rerouted = True
            break

# --- 5) Summary
print("Rerouting policy: QoT-guided with up to K=%d alternate paths" % K)
print("Demands predicted-bad (considered):", considered)
print("Salvaged (predicted-good after reroute):", salvaged)
rate = salvaged / considered if considered>0 else 0.0
print("Salvage rate: %.1f%%" % (100*rate))
if extra_dist:
    print("Avg extra distance (km): %.1f" % (np.mean(extra_dist)))
    print("Avg extra latency (ms): %.3f" % (np.mean(extra_lat)))
else:
    print("No successful reroutes → no overhead stats.")


Rerouting policy: QoT-guided with up to K=3 alternate paths
Demands predicted-bad (considered): 431
Salvaged (predicted-good after reroute): 96
Salvage rate: 22.3%
Avg extra distance (km): 438.2
Avg extra latency (ms): 2.191


In [20]:
pip install networkx

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 1.7 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.0 MB 1.9 MB/s eta 0:00:01
   ------------------------------ --------- 1.6/2.0 MB 2.0 MB/s eta 0:00:01
   ------------------------------ --------- 1.6/2.0 MB 2.0 MB/s eta 0:00:01
   ------------------------------------ --- 1.8/2.0 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 1.4 MB/s  0:00:01
Note: you may need to restart the kernel to use updated packages.


In [22]:
# Few-shot QoT adaptation on GEANT2 held-out (compare zero-shot vs k-shot)
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import pandas as pd

# ---------- config ----------
k = 100          # change to 10 / 40 / 200 etc.
seed = 42

# ---------- build held-out test by removing few-shot rows ----------
key_cols = ['topology','day','src','dst','path','hops','distance_km']
few_k = few.sample(n=min(k, len(few)), random_state=seed).copy()
few_k['__key__'] = few_k[key_cols].astype(str).agg('|'.join, axis=1)

test_all = paths[paths['split']=='test_target'].copy()
test_all['__key__'] = test_all[key_cols].astype(str).agg('|'.join, axis=1)
heldout = test_all[~test_all['__key__'].isin(set(few_k['__key__']))].copy()

train = paths[paths['split']=='train_source'].copy()  # NSFNET only

# ---------- feature builders ----------
NUM_EASY = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps','req_osnr_db','osnr_margin_db'
]
NUM_HARD = [c for c in NUM_EASY if c not in ['req_osnr_db','osnr_margin_db']]
CAT = ['modulation']

def make_xy(df, hard=False):
    cols = (NUM_HARD if hard else NUM_EASY) + CAT
    X = df[cols].copy()
    X['modulation'] = X['modulation'].astype('category').cat.codes
    y = df['qot_ok'].astype(int).values
    return X, y, cols

def run_variant(hard=False, tag="HARD"):
    # train on NSFNET
    Xtr, ytr, cols = make_xy(train, hard)
    Xho, yho, _    = make_xy(heldout, hard)

    scaler = StandardScaler()
    Xtr = scaler.fit_transform(Xtr)
    Xho = scaler.transform(Xho)

    clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                        alpha=1e-4, learning_rate_init=1e-3,
                        max_iter=200, random_state=seed)
    clf.fit(Xtr, ytr)

    # zero-shot on held-out GEANT2
    y0_pred = clf.predict(Xho)
    metrics_zero = {
        "acc": accuracy_score(yho, y0_pred),
        "f1":  f1_score(yho, y0_pred, average='macro')
    }
    try:
        y0_prob = clf.predict_proba(Xho)[:,1]
        metrics_zero["auc"] = roc_auc_score(yho, y0_prob)
    except Exception:
        pass

    # few-shot fine-tune on GEANT2 labeled k rows
    Xfs, yfs, _ = make_xy(few_k, hard)
    Xfs = scaler.transform(Xfs)
    clf.partial_fit(Xfs, yfs)

    # re-test on held-out
    yA_pred = clf.predict(Xho)
    metrics_k = {
        "acc": accuracy_score(yho, yA_pred),
        "f1":  f1_score(yho, yA_pred, average='macro')
    }
    try:
        yA_prob = clf.predict_proba(Xho)[:,1]
        metrics_k["auc"] = roc_auc_score(yho, yA_prob)
    except Exception:
        pass

    print(f"\n[{tag}] QoT few-shot (k={len(few_k)}) on GEANT2 held-out")
    print("Zero-shot :", {k: round(v,4) for k,v in metrics_zero.items()})
    print("k-shot    :", {k: round(v,4) for k,v in metrics_k.items()})
    print("Δ (k - 0) :", {m: round(metrics_k[m]-metrics_zero.get(m,0), 4) for m in metrics_k})

# ---------- Run both variants ----------
run_variant(hard=False, tag="EASY (with margin & req)")
run_variant(hard=True,  tag="HARD (drop margin & req)")



[EASY (with margin & req)] QoT few-shot (k=100) on GEANT2 held-out
Zero-shot : {'acc': 0.9796, 'f1': 0.9725, 'auc': 0.9983}
k-shot    : {'acc': 0.9836, 'f1': 0.9783, 'auc': 0.9988}
Δ (k - 0) : {'acc': 0.004, 'f1': 0.0058, 'auc': 0.0005}

[HARD (drop margin & req)] QoT few-shot (k=100) on GEANT2 held-out
Zero-shot : {'acc': 0.9808, 'f1': 0.9748, 'auc': 0.9984}
k-shot    : {'acc': 0.9811, 'f1': 0.9748, 'auc': 0.998}
Δ (k - 0) : {'acc': 0.0004, 'f1': 0.0, 'auc': -0.0003}


In [23]:
# Few-shot adaptation for failure_present (0/1) on GEANT2 held-out
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

# --- config ---
k = 100        # try 10 / 40 / 100 / 200
seed = 42

# --- build held-out test by removing few-shot rows ---
key_cols = ['topology','day','src','dst','path','hops','distance_km']
few_k = few.sample(n=min(k, len(few)), random_state=seed).copy()
few_k['__key__'] = few_k[key_cols].astype(str).agg('|'.join, axis=1)

test_all = paths[paths['split']=='test_target'].copy()
test_all['__key__'] = test_all[key_cols].astype(str).agg('|'.join, axis=1)
heldout = test_all[~test_all['__key__'].isin(set(few_k['__key__']))].copy()

train = paths[paths['split']=='train_source'].copy()  # NSFNET

# --- features ---
NUM_EASY = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps','req_osnr_db','osnr_margin_db'
]
NUM_HARD = [c for c in NUM_EASY if c not in ['max_center_offset_ghz','min_filter_bw_scale']]
CAT = ['modulation']

def make_xy(df, hard=False):
    cols = (NUM_HARD if hard else NUM_EASY) + CAT
    X = df[cols].copy()
    X['modulation'] = X['modulation'].astype('category').cat.codes
    y = df['failure_present'].astype(int).values
    return X, y, cols

def run_variant(hard=False, tag="HARD"):
    # train on NSFNET
    Xtr, ytr, cols = make_xy(train, hard)
    Xho, yho, _    = make_xy(heldout, hard)

    scaler = StandardScaler()
    Xtr = scaler.fit_transform(Xtr)
    Xho = scaler.transform(Xho)

    clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                        alpha=1e-4, learning_rate_init=1e-3,
                        max_iter=200, random_state=seed)
    clf.fit(Xtr, ytr)

    # zero-shot on held-out
    y0 = clf.predict(Xho)
    m0 = {"acc": accuracy_score(yho, y0), "f1": f1_score(yho, y0, average='macro')}

    # few-shot fine-tune on GEANT2 labeled k rows
    Xfs, yfs, _ = make_xy(few_k, hard)
    Xfs = scaler.transform(Xfs)
    clf.partial_fit(Xfs, yfs)

    # re-test on held-out
    yA = clf.predict(Xho)
    mA = {"acc": accuracy_score(yho, yA), "f1": f1_score(yho, yA, average='macro')}

    print(f"\n[{tag}] Failure detection few-shot (k={len(few_k)})")
    print("Zero-shot :", {k: round(v,4) for k,v in m0.items()})
    print("k-shot    :", {k: round(v,4) for k,v in mA.items()})
    print("Δ (k - 0) :", {m: round(mA[m]-m0[m], 4) for m in mA})

# Run both variants
run_variant(hard=False, tag="EASY (with fingerprints)")
run_variant(hard=True,  tag="HARD (drop fingerprints)")



[EASY (with fingerprints)] Failure detection few-shot (k=100)
Zero-shot : {'acc': 0.9968, 'f1': 0.9933}
k-shot    : {'acc': 0.9968, 'f1': 0.9933}
Δ (k - 0) : {'acc': 0.0, 'f1': 0.0}

[HARD (drop fingerprints)] Failure detection few-shot (k=100)
Zero-shot : {'acc': 0.8447, 'f1': 0.4906}
k-shot    : {'acc': 0.7874, 'f1': 0.5146}
Δ (k - 0) : {'acc': -0.0574, 'f1': 0.024}


C:\Users\devon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [24]:
# Few-shot for failure_type: {'none','shift','tighten'} on GEANT2 (held-out)
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_sample_weight

# ---- config ----
k = 100        # try 10 / 40 / 100 / 200
seed = 42
ORDER = ['none','shift','tighten']
cat_type = pd.api.types.CategoricalDtype(categories=ORDER, ordered=False)

# ---- build held-out test by removing few-shot rows ----
key_cols = ['topology','day','src','dst','path','hops','distance_km']
few_k = few.sample(n=min(k, len(few)), random_state=seed).copy()
few_k['__key__'] = few_k[key_cols].astype(str).agg('|'.join, axis=1)

test_all = paths[paths['split']=='test_target'].copy()
test_all['__key__'] = test_all[key_cols].astype(str).agg('|'.join, axis=1)
heldout = test_all[~test_all['__key__'].isin(set(few_k['__key__']))].copy()

train = paths[paths['split']=='train_source'].copy()  # NSFNET

# ---- features ----
NUM_EASY = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps','req_osnr_db','osnr_margin_db'
]
# HARD: drop direct fingerprints (offset/scale) so typing isn't trivial
NUM_HARD = [c for c in NUM_EASY if c not in ['max_center_offset_ghz','min_filter_bw_scale']]
CAT = ['modulation']

def make_xy(df, hard=False):
    cols = (NUM_HARD if hard else NUM_EASY) + CAT
    X = df[cols].copy()
    X['modulation'] = X['modulation'].astype('category').cat.codes
    y = df['failure_type'].astype(cat_type).cat.codes.values  # 0=none, 1=shift, 2=tighten
    return X, y, cols

def run_variant(hard=False, tag="HARD"):
    # Train on NSFNET
    Xtr, ytr, cols = make_xy(train, hard)
    Xho, yho, _    = make_xy(heldout, hard)
    Xfs, yfs, _    = make_xy(few_k, hard)

    scaler = StandardScaler()
    Xtr = scaler.fit_transform(Xtr)
    Xho = scaler.transform(Xho)
    Xfs = scaler.transform(Xfs)

    # Class-imbalance handling via sample weights (train only)
    sw = compute_sample_weight(class_weight='balanced', y=ytr)

    clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                        alpha=1e-4, learning_rate_init=1e-3,
                        max_iter=200, random_state=seed)

    clf.fit(Xtr, ytr, sample_weight=sw)

    # Zero-shot on GEANT2 held-out
    y0 = clf.predict(Xho)
    m0 = f1_score(yho, y0, average='macro')

    # Few-shot fine-tune on GEANT2 labeled k rows
    # (partial_fit without weights keeps it simple & robust)
    clf.partial_fit(Xfs, yfs)

    # Re-test on held-out
    yA = clf.predict(Xho)
    mA = f1_score(yho, yA, average='macro')

    print(f"\n[{tag}] Failure Type 3-way few-shot (k={len(few_k)}) — Macro-F1")
    print("Zero-shot :", round(m0,4))
    print("k-shot    :", round(mA,4))
    print("Δ (k-0)   :", round(mA - m0, 4))

    # Optional quick breakdown (helps spot which class improved)
    print("\nConfusion matrix (rows=GT, cols=Pred) — order", ORDER)
    print(confusion_matrix(yho, yA))
    print("\nPer-class report after k-shot:")
    print(classification_report(yho, yA, target_names=ORDER, digits=4))

# Run both variants
run_variant(hard=False, tag="EASY (with fingerprints)")
run_variant(hard=True,  tag="HARD (drop fingerprints)")



[EASY (with fingerprints)] Failure Type 3-way few-shot (k=100) — Macro-F1
Zero-shot : 0.9085
k-shot    : 0.9536
Δ (k-0)   : 0.0451

Confusion matrix (rows=GT, cols=Pred) — order ['none', 'shift', 'tighten']
[[4554    2    4]
 [   1  411   39]
 [  11    0  278]]

Per-class report after k-shot:
              precision    recall  f1-score   support

        none     0.9974    0.9987    0.9980      4560
       shift     0.9952    0.9113    0.9514       451
     tighten     0.8660    0.9619    0.9115       289

    accuracy                         0.9892      5300
   macro avg     0.9529    0.9573    0.9536      5300
weighted avg     0.9900    0.9892    0.9893      5300


[HARD (drop fingerprints)] Failure Type 3-way few-shot (k=100) — Macro-F1
Zero-shot : 0.3287
k-shot    : 0.3111
Δ (k-0)   : -0.0176

Confusion matrix (rows=GT, cols=Pred) — order ['none', 'shift', 'tighten']
[[3777  126  657]
 [ 386    1   64]
 [ 239    4   46]]

Per-class report after k-shot:
              precision    r

C:\Users\devon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [25]:
# Label-free TTA-lite for QoT on GEANT2 (held-out): BN-adapt + test-time augmentations
import numpy as np, pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

# ---------- 1) Features (HARD: drop margin/req) ----------
NUM_HARD = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
CAT = ['modulation']

def make_xy_qot(df):
    X = df[NUM_HARD + CAT].copy()
    X['modulation'] = X['modulation'].astype('category').cat.codes
    y = df['qot_ok'].astype(int).values
    return X, y

# ---------- 2) Train on NSFNET; evaluate zero-shot on GEANT2 ----------
train = paths[paths['split']=='train_source'].copy()
test  = paths[paths['split']=='test_target'].copy()  # GEANT2 entire test domain

Xtr, ytr = make_xy_qot(train)
Xte, yte = make_xy_qot(test)

sc_src = StandardScaler()
Xtr_s = sc_src.fit_transform(Xtr)
Xte_s = sc_src.transform(Xte)

clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                    alpha=1e-4, learning_rate_init=1e-3,
                    max_iter=200, random_state=42)
clf.fit(Xtr_s, ytr)

# zero-shot
p0 = clf.predict_proba(Xte_s)[:,1]
y0 = (p0 >= 0.5).astype(int)
m0 = {
    "acc": accuracy_score(yte, y0),
    "f1" : f1_score(yte, y0, average='macro'),
    "auc": roc_auc_score(yte, p0)
}
print("Zero-shot (HARD features):", {k: round(v,4) for k,v in m0.items()})

# ---------- 3) TTA-lite: BN-adapt + test-time augmentations ----------
def bn_adapt_transform(X_source_scaled, X_raw_batch, sc_src, gamma=0.3):
    """
    BN-adapt style: blend source scaler stats with *current batch* stats.
    X_source_scaled is unused except for shape; we re-compute transform using blended stats.
    gamma: weight for batch stats (0=no adapt, 1=only batch).
    """
    mu_src = sc_src.mean_
    sd_src = sc_src.scale_
    mu_b   = X_raw_batch.mean(axis=0)
    sd_b   = X_raw_batch.std(axis=0, ddof=0) + 1e-6

    mu_new = (1-gamma)*mu_src + gamma*mu_b
    sd_new = (1-gamma)*sd_src + gamma*sd_b
    Xb_s   = (X_raw_batch - mu_new) / sd_new
    return Xb_s

def jitter_batch(Xb, strength=0.02, rng=None):
    """
    Small, realistic jitter on numeric columns (not categorical codes).
    We assume 'modulation' already encoded as int; do not jitter that column (last column).
    """
    if rng is None:
        rng = np.random.RandomState(7)
    Xn = Xb.copy()
    # do not jitter the last column (modulation code)
    noise = rng.normal(loc=0.0, scale=strength, size=Xn[:,:-1].shape)
    Xn[:,:-1] = Xn[:,:-1] * (1 + noise)
    return Xn

# batch params
BATCH = 256
GAMMA = 0.3      # BN blend weight
AUG_N = 5        # augmentations per batch sample
JSTRENGTH = 0.03

pA = np.zeros_like(p0)
for start in range(0, len(Xte), BATCH):
    end = min(len(Xte), start+BATCH)
    Xb_raw = Xte.iloc[start:end].values

    # BN-adapt scaling for this batch
    Xb_s = bn_adapt_transform(None, Xb_raw, sc_src, gamma=GAMMA)

    # Monte-Carlo test-time augmentations
    probs = []
    for _ in range(AUG_N):
        Xb_j = jitter_batch(Xb_s, strength=JSTRENGTH)
        probs.append(clf.predict_proba(Xb_j)[:,1])
    p_batch = np.mean(probs, axis=0)
    pA[start:end] = p_batch

yA = (pA >= 0.5).astype(int)
mA = {
    "acc": accuracy_score(yte, yA),
    "f1" : f1_score(yte, yA, average='macro'),
    "auc": roc_auc_score(yte, pA)
}
print("TTA-lite (BN-adapt + aug):", {k: round(v,4) for k,v in mA.items()})
print("Δ (TTA - zero):", {k: round(mA[k]-m0[k],4) for k in m0})



Zero-shot (HARD features): {'acc': 0.9809, 'f1': 0.9754, 'auc': 0.9984}
TTA-lite (BN-adapt + aug): {'acc': 0.9828, 'f1': 0.9775, 'auc': 0.9987}
Δ (TTA - zero): {'acc': 0.0019, 'f1': 0.0021, 'auc': 0.0003}


In [26]:
# Train a per-edge classifier on NSFNET (fail_link vs others) and evaluate on GEANT2 (zero-shot).
# Prediction per path = edge with highest failure probability.

import numpy as np, pandas as pd
from sklearn.linear_model import LogisticRegression

# --- Quick helpers ---
lk_idx = links.set_index(["topology","day","edge_id"])

def edge_ids_from_path(path_str):
    ns = [int(x) for x in path_str.split("->")]
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(ns[:-1], ns[1:])]

def build_edge_dataset(df):
    """Expand each failed path row into per-edge samples with features + label(1 if fail_link)."""
    rows = []
    for _, r in df.iterrows():
        eids = edge_ids_from_path(r["path"])
        feats = []
        for eid in eids:
            try:
                rec = lk_idx.loc[(r["topology"], r["day"], eid)]
            except KeyError:
                feats = []  # missing join; skip whole path
                break
            feats.append(dict(
                eid=eid,
                center_shift=float(rec["center_freq_offset_ghz"]),
                tighten_intensity=max(0.0, 1.0/float(rec["filter_bw_scale"]) - 1.0),
                util=float(rec["bandwidth_utilization"]),
                osnr=float(rec["osnr_db"]),
                snr=float(rec["snr_db"]),
                length=float(rec["length_km"]),
                latency=float(rec["latency_ms"])
            ))
        if not feats:
            continue
        # path-level relative features (bottleneck cues)
        min_osnr = min(f["osnr"] for f in feats)
        for idx, f in enumerate(feats):
            rows.append({
                "topology": r["topology"],
                "day": int(r["day"]),
                "path": r["path"],
                "eid": f["eid"],
                # absolute edge fingerprints
                "center_shift": f["center_shift"],
                "tighten_intensity": f["tighten_intensity"],
                "util": f["util"],
                "osnr": f["osnr"],
                "snr": f["snr"],
                "length": f["length"],
                "latency": f["latency"],
                # relative feature: how close to bottleneck
                "osnr_gap_to_min": f["osnr"] - min_osnr,
                # label
                "y": int(f["eid"] == r["fail_link"])
            })
    return pd.DataFrame(rows)

# --- Build splits for localization (only failures with valid fail_link) ---
def with_loc(df):
    return df[(df["failure_present"]==1) & (df["fail_link"].astype(str)!="")].copy()

loc_train = with_loc(paths[paths["split"]=="train_source"])
loc_test  = with_loc(paths[paths["split"]=="test_target"])

# --- Expand to per-edge datasets ---
train_edges = build_edge_dataset(loc_train)
test_edges  = build_edge_dataset(loc_test)

print("Per-edge samples:", len(train_edges), "(train) |", len(test_edges), "(test)")

# --- Train logistic regression (balanced) ---
FEATS = ["center_shift","tighten_intensity","util","osnr","snr","length","latency","osnr_gap_to_min"]
X_tr, y_tr = train_edges[FEATS].values, train_edges["y"].values

clf_edge = LogisticRegression(max_iter=2000, class_weight="balanced")
clf_edge.fit(X_tr, y_tr)

# --- Evaluate per path: choose edge with highest predicted prob ---
from collections import defaultdict

# group test edges by (topology, day, path)
groups = defaultdict(list)
for i, r in test_edges.iterrows():
    key = (r["topology"], r["day"], r["path"])
    groups[key].append(r)

top1_hits, top3_hits, hop_errors = 0, 0, []
n_paths = 0

for key, edge_list in groups.items():
    dfp = pd.DataFrame(edge_list)
    probs = clf_edge.predict_proba(dfp[FEATS].values)[:,1]
    dfp = dfp.assign(prob=probs)

    # sort by prob desc
    dfp = dfp.sort_values("prob", ascending=False).reset_index(drop=True)

    # ground-truth edge id
    # fetch GT from original loc_test row
    topo, day, path_str = key
    gt = loc_test[(loc_test["topology"]==topo) & (loc_test["day"]==day) & (loc_test["path"]==path_str)]["fail_link"].iloc[0]

    # metrics
    n_paths += 1
    pred_top1 = dfp.loc[0, "eid"]
    if pred_top1 == gt:
        top1_hits += 1
    if gt in dfp.head(3)["eid"].tolist():
        top3_hits += 1

    # hop-error
    eids = edge_ids_from_path(path_str)
    try:
        hop_errors.append(abs(eids.index(pred_top1) - eids.index(gt)))
    except ValueError:
        pass

top1 = top1_hits / max(1, n_paths)
top3 = top3_hits / max(1, n_paths)
mhe  = float(np.mean(hop_errors)) if hop_errors else float("nan")

print(f"Paths evaluated (GEANT2 failures): {n_paths}")
print(f"Graph-aware (edge LR) — Top-1: {top1:.3f} | Top-3: {top3:.3f} | Mean hop-error: {mhe:.2f}")

# (Optional) Compare to heuristic from Step-5 quickly (recompute here):
def heuristic_penalty(row):
    # same as Step-5: (offset/20)^2 + max(0, 1/scale -1) * util
    rec = lk_idx.loc[(row["topology"], row["day"], row["eid"])]
    offset = float(rec["center_freq_offset_ghz"])
    scale  = float(rec["filter_bw_scale"])
    util   = float(rec["bandwidth_utilization"])
    return (offset/20.0)**2 + max(0.0, (1.0/max(scale, 1e-6) - 1.0)) * util

top1_h, top3_h, hop_h, n2 = 0, 0, [], 0
for key, edge_list in groups.items():
    dfp = pd.DataFrame(edge_list)
    dfp["pen"] = dfp.apply(heuristic_penalty, axis=1)
    dfp = dfp.sort_values("pen", ascending=False).reset_index(drop=True)

    topo, day, path_str = key
    gt = loc_test[(loc_test["topology"]==topo) & (loc_test["day"]==day) & (loc_test["path"]==path_str)]["fail_link"].iloc[0]
    n2 += 1
    pred1 = dfp.loc[0, "eid"]
    if pred1 == gt: top1_h += 1
    if gt in dfp.head(3)["eid"].tolist(): top3_h += 1

    eids = edge_ids_from_path(path_str)
    try:
        hop_h.append(abs(eids.index(pred1) - eids.index(gt)))
    except ValueError:
        pass

print(f"Heuristic — Top-1: {top1_h/max(1,n2):.3f} | Top-3: {top3_h/max(1,n2):.3f} | Mean hop-error: {np.mean(hop_h) if hop_h else float('nan'):.2f}")


Per-edge samples: 1556 (train) | 3248 (test)
Paths evaluated (GEANT2 failures): 790
Graph-aware (edge LR) — Top-1: 0.980 | Top-3: 1.000 | Mean hop-error: 0.03
Heuristic — Top-1: 1.000 | Top-3: 1.000 | Mean hop-error: 0.00


In [27]:
# Few-shot localization: train edge-level LR on NSFNET → evaluate on GEANT2 held-out,
# then fine-tune with k few-shot GEANT2 failure paths and re-evaluate.

import numpy as np, pandas as pd
from sklearn.linear_model import LogisticRegression

# ---------- config ----------
k = 100          # try 10 / 40 / 100 / 200
seed = 42
rng = np.random.RandomState(seed)

# ---------- helpers ----------
lk_idx = links.set_index(["topology","day","edge_id"])

def edge_ids_from_path(path_str):
    ns = [int(x) for x in path_str.split("->")]
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(ns[:-1], ns[1:])]

def with_loc(df):
    return df[(df["failure_present"]==1) & (df["fail_link"].astype(str)!="")].copy()

def build_edge_dataset(df):
    """Expand failed paths into per-edge samples with features + label (1 if edge==fail_link)."""
    rows = []
    for _, r in df.iterrows():
        eids = edge_ids_from_path(r["path"])
        feats = []
        # collect per-edge telemetry
        ok = True
        for eid in eids:
            try:
                rec = lk_idx.loc[(r["topology"], r["day"], eid)]
            except KeyError:
                ok = False
                break
            feats.append(dict(
                eid=eid,
                center_shift=float(rec["center_freq_offset_ghz"]),
                tighten_intensity=max(0.0, 1.0/float(rec["filter_bw_scale"]) - 1.0),
                util=float(rec["bandwidth_utilization"]),
                osnr=float(rec["osnr_db"]),
                snr=float(rec["snr_db"]),
                length=float(rec["length_km"]),
                latency=float(rec["latency_ms"])
            ))
        if not ok or not feats:
            continue
        min_osnr = min(f["osnr"] for f in feats)  # per-path bottleneck ref
        for f in feats:
            rows.append({
                "topology": r["topology"], "day": int(r["day"]), "path": r["path"],
                "eid": f["eid"],
                "center_shift": f["center_shift"],
                "tighten_intensity": f["tighten_intensity"],
                "util": f["util"], "osnr": f["osnr"], "snr": f["snr"],
                "length": f["length"], "latency": f["latency"],
                "osnr_gap_to_min": f["osnr"] - min_osnr,
                "y": int(f["eid"] == r["fail_link"])
            })
    return pd.DataFrame(rows)

def eval_localization(edge_df, loc_df, clf, feats):
    """Per-path: pick edge with highest P(fail). Return Top-1/Top-3/MHE and counts."""
    from collections import defaultdict
    groups = defaultdict(list)
    for _, r in edge_df.iterrows():
        groups[(r["topology"], r["day"], r["path"])].append(r)
    top1=top3=0; hop_err=[]; n=0
    for key, lst in groups.items():
        dfp = pd.DataFrame(lst)
        probs = clf.predict_proba(dfp[feats].values)[:,1]
        dfp = dfp.assign(prob=probs).sort_values("prob", ascending=False).reset_index(drop=True)
        topo, day, path_str = key
        gt = loc_df[(loc_df["topology"]==topo) & (loc_df["day"]==day) & (loc_df["path"]==path_str)]["fail_link"]
        if gt.empty: 
            continue
        gt = gt.iloc[0]
        n += 1
        pred1 = dfp.loc[0,"eid"]
        if pred1 == gt: top1 += 1
        if gt in dfp.head(3)["eid"].tolist(): top3 += 1
        eids = edge_ids_from_path(path_str)
        try:
            hop_err.append(abs(eids.index(pred1) - eids.index(gt)))
        except ValueError:
            pass
    return dict(
        paths=n, top1=top1/max(1,n), top3=top3/max(1,n),
        mhe=(float(np.mean(hop_err)) if hop_err else float("nan"))
    )

FEATS = ["center_shift","tighten_intensity","util","osnr","snr","length","latency","osnr_gap_to_min"]

# ---------- splits ----------
loc_train_paths = with_loc(paths[paths["split"]=="train_source"])      # NSFNET (fail-only)
loc_test_paths  = with_loc(paths[paths["split"]=="test_target"])       # GEANT2 (all fail)

# few-shot = only failure rows from the few-shot CSV
few_fail = with_loc(few).reset_index(drop=True)
k_eff = min(k, len(few_fail))
few_k = few_fail.sample(n=k_eff, random_state=seed)

# held-out = GEANT2 failures minus few_k (by composite key)
key_cols = ["topology","day","path"]
few_keys = set(few_k[key_cols].astype(str).agg("|".join, axis=1))
loc_test_paths = loc_test_paths.copy()
loc_test_paths["__key__"] = loc_test_paths[key_cols].astype(str).agg("|".join, axis=1)
loc_heldout_paths = loc_test_paths[~loc_test_paths["__key__"].isin(few_keys)].copy()

# ---------- build per-edge datasets ----------
train_edges   = build_edge_dataset(loc_train_paths)
heldout_edges = build_edge_dataset(loc_heldout_paths)
few_edges     = build_edge_dataset(few_k)

print(f"Edge samples: train={len(train_edges)}, held-out={len(heldout_edges)}, few-shot={len(few_edges)}  (k_fail={k_eff})")

# ---------- train on NSFNET edges ----------
clf_edge = LogisticRegression(max_iter=2000, class_weight="balanced")
clf_edge.fit(train_edges[FEATS].values, train_edges["y"].values)

# zero-shot eval on GEANT2 held-out failures
m0 = eval_localization(heldout_edges, loc_heldout_paths, clf_edge, FEATS)
print("Zero-shot  :", {k: (round(v,3) if isinstance(v,float) else v) for k,v in m0.items()})

# ---------- fine-tune by re-training on (train + few-shot) ----------
if len(few_edges) > 0:
    combo_X = pd.concat([train_edges[FEATS], few_edges[FEATS]], axis=0)
    combo_y = pd.concat([train_edges["y"],   few_edges["y"]],   axis=0)
    clf_edge_ft = LogisticRegression(max_iter=2000, class_weight="balanced")
    clf_edge_ft.fit(combo_X.values, combo_y.values)
    mA = eval_localization(heldout_edges, loc_heldout_paths, clf_edge_ft, FEATS)
    print("k-shot     :", {k: (round(v,3) if isinstance(v,float) else v) for k,v in mA.items()})
    print("Δ (k-0)    :", {k: (round(mA[k]-m0[k],3) if isinstance(mA[k],float) else mA[k]) for k in m0})
else:
    print("No few-shot failure rows available → skipping fine-tune.")


Edge samples: train=1556, held-out=2825, few-shot=423  (k_fail=100)
Zero-shot  : {'paths': 690, 'top1': 0.98, 'top3': 1.0, 'mhe': 0.032}
k-shot     : {'paths': 690, 'top1': 0.994, 'top3': 1.0, 'mhe': 0.014}
Δ (k-0)    : {'paths': 690, 'top1': 0.014, 'top3': 0.0, 'mhe': -0.017}


In [27]:
# Few-shot localization: train edge-level LR on NSFNET → evaluate on GEANT2 held-out,
# then fine-tune with k few-shot GEANT2 failure paths and re-evaluate.

import numpy as np, pandas as pd
from sklearn.linear_model import LogisticRegression

# ---------- config ----------
k = 100          # try 10 / 40 / 100 / 200
seed = 42
rng = np.random.RandomState(seed)

# ---------- helpers ----------
lk_idx = links.set_index(["topology","day","edge_id"])

def edge_ids_from_path(path_str):
    ns = [int(x) for x in path_str.split("->")]
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(ns[:-1], ns[1:])]

def with_loc(df):
    return df[(df["failure_present"]==1) & (df["fail_link"].astype(str)!="")].copy()

def build_edge_dataset(df):
    """Expand failed paths into per-edge samples with features + label (1 if edge==fail_link)."""
    rows = []
    for _, r in df.iterrows():
        eids = edge_ids_from_path(r["path"])
        feats = []
        # collect per-edge telemetry
        ok = True
        for eid in eids:
            try:
                rec = lk_idx.loc[(r["topology"], r["day"], eid)]
            except KeyError:
                ok = False
                break
            feats.append(dict(
                eid=eid,
                center_shift=float(rec["center_freq_offset_ghz"]),
                tighten_intensity=max(0.0, 1.0/float(rec["filter_bw_scale"]) - 1.0),
                util=float(rec["bandwidth_utilization"]),
                osnr=float(rec["osnr_db"]),
                snr=float(rec["snr_db"]),
                length=float(rec["length_km"]),
                latency=float(rec["latency_ms"])
            ))
        if not ok or not feats:
            continue
        min_osnr = min(f["osnr"] for f in feats)  # per-path bottleneck ref
        for f in feats:
            rows.append({
                "topology": r["topology"], "day": int(r["day"]), "path": r["path"],
                "eid": f["eid"],
                "center_shift": f["center_shift"],
                "tighten_intensity": f["tighten_intensity"],
                "util": f["util"], "osnr": f["osnr"], "snr": f["snr"],
                "length": f["length"], "latency": f["latency"],
                "osnr_gap_to_min": f["osnr"] - min_osnr,
                "y": int(f["eid"] == r["fail_link"])
            })
    return pd.DataFrame(rows)

def eval_localization(edge_df, loc_df, clf, feats):
    """Per-path: pick edge with highest P(fail). Return Top-1/Top-3/MHE and counts."""
    from collections import defaultdict
    groups = defaultdict(list)
    for _, r in edge_df.iterrows():
        groups[(r["topology"], r["day"], r["path"])].append(r)
    top1=top3=0; hop_err=[]; n=0
    for key, lst in groups.items():
        dfp = pd.DataFrame(lst)
        probs = clf.predict_proba(dfp[feats].values)[:,1]
        dfp = dfp.assign(prob=probs).sort_values("prob", ascending=False).reset_index(drop=True)
        topo, day, path_str = key
        gt = loc_df[(loc_df["topology"]==topo) & (loc_df["day"]==day) & (loc_df["path"]==path_str)]["fail_link"]
        if gt.empty: 
            continue
        gt = gt.iloc[0]
        n += 1
        pred1 = dfp.loc[0,"eid"]
        if pred1 == gt: top1 += 1
        if gt in dfp.head(3)["eid"].tolist(): top3 += 1
        eids = edge_ids_from_path(path_str)
        try:
            hop_err.append(abs(eids.index(pred1) - eids.index(gt)))
        except ValueError:
            pass
    return dict(
        paths=n, top1=top1/max(1,n), top3=top3/max(1,n),
        mhe=(float(np.mean(hop_err)) if hop_err else float("nan"))
    )

FEATS = ["center_shift","tighten_intensity","util","osnr","snr","length","latency","osnr_gap_to_min"]

# ---------- splits ----------
loc_train_paths = with_loc(paths[paths["split"]=="train_source"])      # NSFNET (fail-only)
loc_test_paths  = with_loc(paths[paths["split"]=="test_target"])       # GEANT2 (all fail)

# few-shot = only failure rows from the few-shot CSV
few_fail = with_loc(few).reset_index(drop=True)
k_eff = min(k, len(few_fail))
few_k = few_fail.sample(n=k_eff, random_state=seed)

# held-out = GEANT2 failures minus few_k (by composite key)
key_cols = ["topology","day","path"]
few_keys = set(few_k[key_cols].astype(str).agg("|".join, axis=1))
loc_test_paths = loc_test_paths.copy()
loc_test_paths["__key__"] = loc_test_paths[key_cols].astype(str).agg("|".join, axis=1)
loc_heldout_paths = loc_test_paths[~loc_test_paths["__key__"].isin(few_keys)].copy()

# ---------- build per-edge datasets ----------
train_edges   = build_edge_dataset(loc_train_paths)
heldout_edges = build_edge_dataset(loc_heldout_paths)
few_edges     = build_edge_dataset(few_k)

print(f"Edge samples: train={len(train_edges)}, held-out={len(heldout_edges)}, few-shot={len(few_edges)}  (k_fail={k_eff})")

# ---------- train on NSFNET edges ----------
clf_edge = LogisticRegression(max_iter=2000, class_weight="balanced")
clf_edge.fit(train_edges[FEATS].values, train_edges["y"].values)

# zero-shot eval on GEANT2 held-out failures
m0 = eval_localization(heldout_edges, loc_heldout_paths, clf_edge, FEATS)
print("Zero-shot  :", {k: (round(v,3) if isinstance(v,float) else v) for k,v in m0.items()})

# ---------- fine-tune by re-training on (train + few-shot) ----------
if len(few_edges) > 0:
    combo_X = pd.concat([train_edges[FEATS], few_edges[FEATS]], axis=0)
    combo_y = pd.concat([train_edges["y"],   few_edges["y"]],   axis=0)
    clf_edge_ft = LogisticRegression(max_iter=2000, class_weight="balanced")
    clf_edge_ft.fit(combo_X.values, combo_y.values)
    mA = eval_localization(heldout_edges, loc_heldout_paths, clf_edge_ft, FEATS)
    print("k-shot     :", {k: (round(v,3) if isinstance(v,float) else v) for k,v in mA.items()})
    print("Δ (k-0)    :", {k: (round(mA[k]-m0[k],3) if isinstance(mA[k],float) else mA[k]) for k in m0})
else:
    print("No few-shot failure rows available → skipping fine-tune.")


Edge samples: train=1556, held-out=2825, few-shot=423  (k_fail=100)
Zero-shot  : {'paths': 690, 'top1': 0.98, 'top3': 1.0, 'mhe': 0.032}
k-shot     : {'paths': 690, 'top1': 0.994, 'top3': 1.0, 'mhe': 0.014}
Δ (k-0)    : {'paths': 690, 'top1': 0.014, 'top3': 0.0, 'mhe': -0.017}


In [6]:
# QoT calibration & reliability visuals (Zero-shot vs TTA-lite) on GEANT2 (HARD features)
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score

# ---------- 1) Features (HARD: drop margin/req) ----------
NUM_HARD = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
CAT = ['modulation']

def make_xy_qot(df):
    X = df[NUM_HARD + CAT].copy()
    X['modulation'] = X['modulation'].astype('category').cat.codes
    y = df['qot_ok'].astype(int).values
    return X, y

# ---------- 2) Train on NSFNET; evaluate on GEANT2 ----------
train = paths[paths['split']=='train_source'].copy()
test  = paths[paths['split']=='test_target'].copy()

Xtr, ytr = make_xy_qot(train)
Xte, yte = make_xy_qot(test)

sc_src = StandardScaler()
Xtr_s = sc_src.fit_transform(Xtr)
Xte_s = sc_src.transform(Xte)

clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                    alpha=1e-4, learning_rate_init=1e-3,
                    max_iter=200, random_state=42)
clf.fit(Xtr_s, ytr)

# Zero-shot probs
p0 = clf.predict_proba(Xte_s)[:,1]

# ---------- 3) TTA-lite: BN-adapt + test-time augmentations (label-free) ----------
def bn_adapt_transform(X_raw_batch, sc_src, gamma=0.3):
    mu_src = sc_src.mean_
    sd_src = sc_src.scale_
    mu_b   = X_raw_batch.mean(axis=0)
    sd_b   = X_raw_batch.std(axis=0, ddof=0) + 1e-6
    mu_new = (1-gamma)*mu_src + gamma*mu_b
    sd_new = (1-gamma)*sd_src + gamma*sd_b
    return (X_raw_batch - mu_new) / sd_new

def jitter_batch(Xb, strength=0.03, rng=None):
    rng = rng or np.random.RandomState(7)
    Xn = Xb.copy()
    noise = rng.normal(0.0, strength, size=Xn[:,:-1].shape)
    Xn[:,:-1] = Xn[:,:-1] * (1 + noise)   # don't jitter modulation code (last col)
    return Xn

BATCH = 256; GAMMA = 0.3; AUG_N = 5; JSTRENGTH = 0.03
pA = np.zeros_like(p0)
for s in range(0, len(Xte), BATCH):
    e = min(len(Xte), s+BATCH)
    Xb_raw = Xte.iloc[s:e].values
    Xb_s   = bn_adapt_transform(Xb_raw, sc_src, gamma=GAMMA)
    probs  = []
    for _ in range(AUG_N):
        Xb_j = jitter_batch(Xb_s, strength=JSTRENGTH)
        probs.append(clf.predict_proba(Xb_j)[:,1])
    pA[s:e] = np.mean(probs, axis=0)

# ---------- 4) Metrics & Visuals ----------
from pathlib import Path
OUT = Path("./outputs/figs"); OUT.mkdir(parents=True, exist_ok=True)

# (a) ROC curves
fpr0, tpr0, _ = roc_curve(yte, p0)
fprA, tprA, _ = roc_curve(yte, pA)
auc0, aucA = roc_auc_score(yte, p0), roc_auc_score(yte, pA)

plt.figure()
plt.plot(fpr0, tpr0, label=f"Zero-shot (AUC={auc0:.3f})")
plt.plot(fprA, tprA, label=f"TTA-lite (AUC={aucA:.3f})")
plt.plot([0,1],[0,1],'--',linewidth=1)
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("QoT ROC — GEANT2 (HARD features)")
plt.legend(); 
plt.savefig(OUT/"qot_roc_zero_vs_tta.png", bbox_inches="tight"); plt.close()

# (b) Reliability diagram + ECE
def reliability_curve(y_true, p, n_bins=10):
    bins = np.linspace(0,1,n_bins+1)
    idx  = np.digitize(p, bins) - 1
    accs, confs, sizes = [], [], []
    for b in range(n_bins):
        mask = idx==b
        if not np.any(mask): 
            accs.append(np.nan); confs.append((bins[b]+bins[b+1])/2); sizes.append(0)
            continue
        acc  = np.mean((p[mask]>=0.5)==y_true[mask])
        conf = p[mask].mean()
        accs.append(acc); confs.append(conf); sizes.append(mask.sum())
    # Expected Calibration Error
    N = len(y_true)
    ece = np.nansum([ (sizes[i]/N)*abs(accs[i]-confs[i]) for i in range(n_bins) ])
    return np.array(confs), np.array(accs), ece

c0, a0, ece0 = reliability_curve(yte, p0, n_bins=10)
cA, aA, eceA = reliability_curve(yte, pA, n_bins=10)

plt.figure()
plt.plot([0,1],[0,1],'--',linewidth=1)
plt.plot(c0, a0, marker='o', label=f"Zero-shot (ECE={ece0:.3f})")
plt.plot(cA, aA, marker='o', label=f"TTA-lite (ECE={eceA:.3f})")
plt.xlabel("Confidence"); plt.ylabel("Empirical Accuracy"); 
plt.title("QoT Reliability — GEANT2 (HARD features)")
plt.legend()
plt.savefig(OUT/"qot_reliability_zero_vs_tta.png", bbox_inches="tight"); plt.close()

# (c) Risk-coverage curve (selective prediction by confidence)
def risk_coverage(y_true, p):
    conf = np.maximum(p, 1-p)  # confidence of predicted label
    order = np.argsort(-conf)  # high → low confidence
    yhat = (p>=0.5).astype(int)
    accs, covs = [], []
    for k in range(50, len(p)+1, max(1, len(p)//20)):
        sel = order[:k]
        accs.append( np.mean(yhat[sel]==y_true[sel]) )
        covs.append( k/len(p) )
    return np.array(covs), np.array(accs)

cov0, acc0 = risk_coverage(yte, p0)
covA, accA = risk_coverage(yte, pA)

plt.figure()
plt.plot(cov0, 1-acc0, label="Zero-shot")   # risk = 1-acc
plt.plot(covA, 1-accA, label="TTA-lite")
plt.gca().invert_yaxis()
plt.xlabel("Coverage"); plt.ylabel("Risk (1 - Accuracy)")
plt.title("QoT Risk–Coverage — GEANT2 (HARD features)")
plt.legend()
plt.savefig(OUT/"qot_risk_coverage_zero_vs_tta.png", bbox_inches="tight"); plt.close()

print("Saved figures →",
      str(OUT/"qot_roc_zero_vs_tta.png"),
      str(OUT/"qot_reliability_zero_vs_tta.png"),
      str(OUT/"qot_risk_coverage_zero_vs_tta.png"), sep="\n")
print({"AUC_zero": float(auc0), "AUC_tta": float(aucA), "ECE_zero": float(ece0), "ECE_tta": float(eceA)})


Saved figures →
outputs\figs\qot_roc_zero_vs_tta.png
outputs\figs\qot_reliability_zero_vs_tta.png
outputs\figs\qot_risk_coverage_zero_vs_tta.png
{'AUC_zero': 0.9983525503074555, 'AUC_tta': 0.9986583012633958, 'ECE_zero': 0.2534282407499026, 'ECE_tta': 0.2520535671302439}


In [2]:
pip install matplotlib


   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   ----- ---------------------------------- 1.0/8.1 MB 3.7 MB/s eta 0:00:02
   ------- -------------------------------- 1.6/8.1 MB 3.6 MB/s eta 0:00:02
   ------------ --------------------------- 2.6/8.1 MB 3.7 MB/s eta 0:00:02
   ---------------- ----------------------- 3.4/8.1 MB 4.0 MB/s eta 0:00:02
   -------------------- ------------------- 4.2/8.1 MB 3.7 MB/s eta 0:00:02
   ----------------------- ---------------- 4.7/8.1 MB 3.6 MB/s eta 0:00:01
   --------------------------- ------------ 5.5/8.1 MB 3.7 MB/s eta 0:00:01
   -------------------------------- ------- 6.6/8.1 MB 3.7 MB/s eta 0:00:01
   ------------------------------------ --- 7.3/8.1 MB 3.7 MB/s eta 0:00:01
   ------------------------------------- -- 7.6/8.1 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 3.6 MB/s  0:00:02
   --------------------------

In [7]:
# Build per-path graph-aware features by joining link telemetry for the same day/topology.
# Output: ./outputs/paths_graph_enriched.csv  (+ a quick preview)

from pathlib import Path
import numpy as np, pandas as pd

# --- load fallbacks in case the kernel was restarted ---
DATA_DIR = Path(".")
if 'paths' not in globals():
    paths = pd.read_csv(DATA_DIR/"eon_paths_timeseries.csv")
if 'links' not in globals():
    links = pd.read_csv(DATA_DIR/"eon_links_timeseries.csv")

OUT_DIR = Path("./outputs"); (OUT_DIR/"figs").mkdir(parents=True, exist_ok=True)

# Fast lookup for link telemetry
lk = links.set_index(["topology","day","edge_id"])

def edge_ids_from_path(path_str):
    ns = [int(x) for x in str(path_str).split("->")]
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(ns[:-1], ns[1:])]

def compute_graph_feats(row):
    topo = row["topology"]; day = row["day"]; p = row["path"]
    eids = edge_ids_from_path(p)
    recs = []
    for eid in eids:
        try:
            rec = lk.loc[(topo, day, eid)]
        except KeyError:
            return pd.Series({"gf_ok":0})  # mark join failure; will drop later
        recs.append(dict(
            eid=eid,
            osnr=float(rec["osnr_db"]),
            snr=float(rec["snr_db"]),
            util=float(rec["bandwidth_utilization"]),
            shift=float(rec["center_freq_offset_ghz"]),
            scale=float(rec["filter_bw_scale"]),
            length=float(rec["length_km"]),
            latency=float(rec["latency_ms"])
        ))
    # aggregate
    hops = len(recs)
    osnrs = np.array([r["osnr"] for r in recs])
    utils = np.array([r["util"] for r in recs])
    shifts = np.array([r["shift"] for r in recs])
    scales = np.array([r["scale"] for r in recs])
    lens = np.array([r["length"] for r in recs])
    lats = np.array([r["latency"] for r in recs])

    # bottleneck (min OSNR) index & normalized position
    bot_idx = int(np.argmin(osnrs))
    bot_pos = bot_idx / max(1, hops-1)

    # fractions of “shifted” and “tightened” edges
    frac_shifted = float((shifts > 0).mean())
    frac_tight   = float((scales < 1.0).mean())

    return pd.Series({
        "gf_ok": 1,
        "gf_len_sum_km": float(lens.sum()),
        "gf_lat_sum_ms": float(lats.sum()),
        "gf_osnr_min": float(osnrs.min()),
        "gf_osnr_mean": float(osnrs.mean()),
        "gf_osnr_var": float(osnrs.var()) if hops>1 else 0.0,
        "gf_snr_min": float(np.min([r["snr"] for r in recs])),
        "gf_util_mean": float(utils.mean()),
        "gf_util_max": float(utils.max()),
        "gf_shift_max": float(shifts.max()),
        "gf_scale_min": float(scales.min()),
        "gf_frac_shifted": frac_shifted,
        "gf_frac_tight": frac_tight,
        "gf_bot_pos": float(bot_pos)  # where along the path the bottleneck sits (0…1)
    })

# Compute features (this may take ~seconds on 10k rows)
enriched = paths.join(paths.apply(compute_graph_feats, axis=1))

# Drop rows where join failed (gf_ok!=1)
before = len(enriched)
enriched = enriched[enriched["gf_ok"]==1].drop(columns=["gf_ok"])
after = len(enriched)
print(f"Enriched rows: {after}/{before} (dropped {before-after} due to missing link joins)")

# Save & preview
out_csv = OUT_DIR/"paths_graph_enriched.csv"
enriched.to_csv(out_csv, index=False)
print("Saved:", out_csv)
display(enriched.head(3)[[
    "topology","day","src","dst","path","hops",
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_shift_max","gf_scale_min",
    "gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]])


Enriched rows: 10800/10800 (dropped 0 due to missing link joins)
Saved: outputs\paths_graph_enriched.csv


,topology,day,src,dst,path,hops,gf_osnr_min,gf_osnr_var,gf_util_mean,gf_shift_max,gf_scale_min,gf_frac_shifted,gf_frac_tight,gf_bot_pos
0,NSFNET,0,1,13,1->8->10->11->12->13,5,22.537527,1.362599,0.787454,0.0,1.0,0.0,0.0,0.500000
1,NSFNET,0,14,1,14->11->10->8->1,4,22.537527,1.538080,0.795083,0.0,1.0,0.0,0.0,0.333333
2,NSFNET,0,5,8,5->7->8,2,23.277962,4.136134,0.636687,0.0,1.0,0.0,0.0,0.000000


In [8]:
# Compare zero-shot performance with/without graph-aware features (HARD setup) on GEANT2.
# Requires: you already ran Step 14 and have ./outputs/paths_graph_enriched.csv

from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# --- Load enriched file (contains original + gf_* columns)
ENRICHED_CSV = Path("./outputs/paths_graph_enriched.csv")
assert ENRICHED_CSV.exists(), "Run Step 14 first to create paths_graph_enriched.csv"
df = pd.read_csv(ENRICHED_CSV)

# --- Splits
train = df[df["split"]=="train_source"].copy()    # NSFNET
test  = df[df["split"]=="test_target"].copy()     # GEANT2

# --- Feature sets
# QoT (HARD): drop req/margin (decision-time realistic), keep other numerics
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
# Enriched = Base + graph-aware summaries
GF_COLS = [
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
    "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]
ENR_QOT = BASE_QOT + GF_COLS

# Failure Detection (HARD): drop direct fingerprints in base (offset/scale)
BASE_FAIL = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'symbol_rate_gbaud','bitrate_gbps'   # note: no offset/scale here
]
ENR_FAIL = BASE_FAIL + GF_COLS

CAT = ['modulation']

def run_task(task="qot", cols=None):
    Xtr = train[cols + CAT].copy()
    Xte = test[cols + CAT].copy()
    # encode modulation
    Xtr['modulation'] = Xtr['modulation'].astype('category').cat.codes
    Xte['modulation'] = Xte['modulation'].astype('category').cat.codes

    scaler = StandardScaler()
    Xtr_s = scaler.fit_transform(Xtr)
    Xte_s = scaler.transform(Xte)

    clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                        alpha=1e-4, learning_rate_init=1e-3,
                        max_iter=200, random_state=42)
    if task=="qot":
        ytr = train['qot_ok'].astype(int).values
        yte = test['qot_ok'].astype(int).values
        clf.fit(Xtr_s, ytr)
        p   = clf.predict_proba(Xte_s)[:,1]
        y   = (p>=0.5).astype(int)
        out = dict(acc=accuracy_score(yte,y),
                   f1 =f1_score(yte,y,average='macro'),
                   auc=roc_auc_score(yte,p))
    elif task=="fail":
        ytr = train['failure_present'].astype(int).values
        yte = test['failure_present'].astype(int).values
        clf.fit(Xtr_s, ytr)
        y   = clf.predict(Xte_s)
        out = dict(acc=accuracy_score(yte,y),
                   f1 =f1_score(yte,y,average='macro'))
    else:
        raise ValueError("task must be 'qot' or 'fail'")
    return out

def pretty(delta, keys):
    return {k: round(delta[k],4) for k in keys if k in delta}

# --- QoT: base vs enriched
m_qot_base = run_task("qot", BASE_QOT)
m_qot_enr  = run_task("qot", ENR_QOT)
dq = {k: m_qot_enr[k]-m_qot_base[k] for k in m_qot_enr}
print("\nQoT (HARD) — Zero-shot GEANT2")
print("Base     :", {k: round(v,4) for k,v in m_qot_base.items()})
print("+GraphFea:", {k: round(v,4) for k,v in m_qot_enr.items()})
print("Δ (enr-base):", pretty(dq, ["acc","f1","auc"]))

# --- Failure Detection: base vs enriched
m_fd_base = run_task("fail", BASE_FAIL)
m_fd_enr  = run_task("fail", ENR_FAIL)
dfd = {k: m_fd_enr[k]-m_fd_base[k] for k in m_fd_enr}
print("\nFailure Detection (HARD) — Zero-shot GEANT2")
print("Base     :", {k: round(v,4) for k,v in m_fd_base.items()})
print("+GraphFea:", {k: round(v,4) for k,v in m_fd_enr.items()})
print("Δ (enr-base):", pretty(dfd, ["acc","f1"]))



QoT (HARD) — Zero-shot GEANT2
Base     : {'acc': 0.9809, 'f1': 0.9754, 'auc': 0.9984}
+GraphFea: {'acc': 0.9598, 'f1': 0.9473, 'auc': 0.9938}
Δ (enr-base): {'acc': -0.0211, 'f1': -0.028, 'auc': -0.0046}


C:\Users\devon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(



Failure Detection (HARD) — Zero-shot GEANT2
Base     : {'acc': 0.8415, 'f1': 0.4935}
+GraphFea: {'acc': 0.9931, 'f1': 0.9862}
Δ (enr-base): {'acc': 0.1517, 'f1': 0.4927}


In [12]:
# Step 16 (fixed): QoT-guided rerouting — BASE vs +GraphFea (zero-shot, HARD)
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings("ignore")

# ---- deps
try:
    import networkx as nx
except ImportError:
    raise ImportError("Please install networkx: pip install networkx")

# ---- load data
ENRICHED_CSV = Path("./outputs/paths_graph_enriched.csv")
assert ENRICHED_CSV.exists(), "Run Step 14 first to create outputs/paths_graph_enriched.csv"
df = pd.read_csv(ENRICHED_CSV)

try:
    links
except NameError:
    links = pd.read_csv("eon_links_timeseries.csv")

# ---- splits
train = df[df["split"]=="train_source"].copy()
test  = df[(df["split"]=="test_target") & (df["topology"]=="GEANT2")].copy()

# ---- features (HARD setup)
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
GF_COLS = [
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
    "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]
ENR_QOT = BASE_QOT + GF_COLS
CAT = ['modulation']

# consistent modulation encoding based on train categories
train_mod_cats = train['modulation'].astype('category').cat.categories.tolist()
def encode_mod(series):
    return pd.Categorical(series, categories=train_mod_cats).codes

def to_ordered_df(df_in, col_order):
    """Return df with columns exactly in col_order; raises if a col is missing."""
    missing = [c for c in col_order if c not in df_in.columns]
    if missing:
        raise ValueError(f"Missing columns for model: {missing}")
    df_out = df_in[col_order].copy()
    return df_out

# ---- train helper returning scaler + exact col order used
def fit_qot(Xtr_raw, ytr, col_order):
    sc = StandardScaler()
    Xtr = to_ordered_df(Xtr_raw, col_order)
    Xs  = sc.fit_transform(Xtr)
    clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                        alpha=1e-4, learning_rate_init=1e-3,
                        max_iter=200, random_state=42)
    clf.fit(Xs, ytr)
    return clf, sc, col_order

# ---- train BASE model
col_order_base = BASE_QOT + CAT
Xtr_base = train[col_order_base].copy()
Xtr_base['modulation'] = encode_mod(Xtr_base['modulation'])
ytr = train['qot_ok'].astype(int).values
clf_base, sc_base, col_order_base = fit_qot(Xtr_base, ytr, col_order_base)

# ---- train +GraphFea model
col_order_enr = ENR_QOT + CAT
Xtr_enr = train[col_order_enr].copy()
Xtr_enr['modulation'] = encode_mod(Xtr_enr['modulation'])
clf_enr, sc_enr, col_order_enr = fit_qot(Xtr_enr, ytr, col_order_enr)

# ---- graph build + utilities
def build_graph_for_day(day):
    sub = links[(links['topology']=='GEANT2') & (links['day']==day)]
    G = nx.Graph()
    for _, r in sub.iterrows():
        u, v = int(r['u']), int(r['v'])
        G.add_edge(u, v,
                   edge_id=f"{min(u,v)}-{max(u,v)}",
                   length_km=float(r['length_km']),
                   latency_ms=float(r['latency_ms']))
    return G, sub.set_index('edge_id')

def edge_ids_from_nodes(nodes):
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(nodes[:-1], nodes[1:])]

def path_aggs(eids, idx):
    rows = idx.loc[eids]
    hops = len(eids)
    base = {
        "hops": hops,
        "distance_km": float(rows['length_km'].sum()),
        "latency_ms":  float(rows['latency_ms'].sum()),
        "avg_utilization": float(rows['bandwidth_utilization'].mean()),
        "min_osnr_db": float(rows['osnr_db'].min()),
        "min_snr_db":  float(rows['snr_db'].min()),
        "max_center_offset_ghz": float(rows['center_freq_offset_ghz'].max()),
        "min_filter_bw_scale":   float(rows['filter_bw_scale'].min()),
    }
    osnrs = rows['osnr_db'].values
    utils = rows['bandwidth_utilization'].values
    shifts= rows['center_freq_offset_ghz'].values
    scales= rows['filter_bw_scale'].values
    gf = {
        "gf_osnr_min": float(osnrs.min()),
        "gf_osnr_var": float(np.var(osnrs)) if hops>1 else 0.0,
        "gf_util_mean": float(utils.mean()),
        "gf_util_max": float(utils.max()),
        "gf_shift_max": float(shifts.max()),
        "gf_scale_min": float(scales.min()),
        "gf_frac_shifted": float((shifts>0).mean()),
        "gf_frac_tight": float((scales<1.0).mean()),
        "gf_bot_pos": float(np.argmin(osnrs)/max(1,hops-1))
    }
    return base, gf

def features_for_candidate(eids, day, tx_row, idx, use_graph):
    base, gf = path_aggs(eids, idx)
    row = {
        **base,
        "symbol_rate_gbaud": float(tx_row['symbol_rate_gbaud']),
        "bitrate_gbps": float(tx_row['bitrate_gbps']),
        "modulation": tx_row['modulation']
    }
    if use_graph:
        row.update(gf)
    return row

# ---- core evaluation
def reroute_eval(model_tag, feat_cols, clf, scaler, col_order, K=3, sample_n=600, seed=7):
    rng = np.random.RandomState(seed)
    sample = test.sample(n=min(sample_n, len(test)), random_state=seed).reset_index(drop=True)

    # original predictions (who needs reroute?)
    Xtest = sample[feat_cols + CAT].copy()
    Xtest['modulation'] = encode_mod(Xtest['modulation'])
    Xtest_ord = to_ordered_df(Xtest, col_order)
    p = clf.predict_proba(scaler.transform(Xtest_ord))[:,1]
    yhat = (p>=0.5).astype(int)

    considered = salvaged = 0
    extra_dist, extra_lat = [], []

    for i, row in sample.iterrows():
        if int(yhat[i]) == 1:
            continue  # already predicted feasible

        day = int(row['day'])
        try:
            G, idx = build_graph_for_day(day)
        except Exception:
            continue

        nodes = [int(x) for x in row['path'].split('->')]
        try:
            alt_iter = nx.shortest_simple_paths(G, nodes[0], nodes[-1], weight=lambda u,v,d: d['length_km'])
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue

        orig_eids = edge_ids_from_nodes(nodes)
        considered += 1
        tried = 0
        for cand_nodes in alt_iter:
            if cand_nodes == nodes:
                continue
            tried += 1
            if tried > K:
                break
            cand_eids = edge_ids_from_nodes(cand_nodes)
            # candidate features
            try:
                row_feat = features_for_candidate(cand_eids, day, row, idx, use_graph=(model_tag=="+GraphFea"))
            except KeyError:
                continue  # missing edge stats
            Xc = pd.DataFrame([row_feat])
            Xc['modulation'] = encode_mod(Xc['modulation'])
            # reorder columns EXACTLY as in fit
            Xc_ord = to_ordered_df(Xc, col_order)
            pred_good = int(clf.predict(scaler.transform(Xc_ord))[0]) == 1
            if pred_good:
                # overhead stats
                try:
                    orig_dist = float(idx.loc[orig_eids]['length_km'].sum())
                    cand_dist = float(idx.loc[cand_eids]['length_km'].sum())
                    orig_lat  = float(idx.loc[orig_eids]['latency_ms'].sum())
                    cand_lat  = float(idx.loc[cand_eids]['latency_ms'].sum())
                except KeyError:
                    orig_dist = cand_dist = orig_lat = cand_lat = np.nan
                extra_dist.append(cand_dist - orig_dist if (not np.isnan(cand_dist) and not np.isnan(orig_dist)) else np.nan)
                extra_lat.append(cand_lat - orig_lat if (not np.isnan(cand_lat) and not np.isnan(orig_lat)) else np.nan)
                salvaged += 1
                break

    # summarize
    rate = 100.0*salvaged/max(1,considered)
    # drop NaNs for means
    ed = [x for x in extra_dist if not (isinstance(x,float) and np.isnan(x))]
    el = [x for x in extra_lat  if not (isinstance(x,float) and np.isnan(x))]
    od_km = float(np.mean(ed)) if ed else float("nan")
    ol_ms = float(np.mean(el)) if el else float("nan")
    return dict(model=model_tag, considered=considered, salvaged=salvaged,
                salvage_rate_pct=rate, avg_extra_km=od_km, avg_extra_ms=ol_ms)

# ---- run both models
res_base = reroute_eval("BASE", BASE_QOT, clf_base, sc_base, col_order_base, K=3, sample_n=600)
res_enr  = reroute_eval("+GraphFea", ENR_QOT, clf_enr, sc_enr, col_order_enr, K=3, sample_n=600)

# ---- print paper-ready summary
def r4(d): return {k:(round(v,3) if isinstance(v,(int,float)) else v) for k,v in d.items()}
print("QoT-guided Rerouting (GEANT2, zero-shot, HARD)")
print("BASE     :", r4(res_base))
print("+GraphFea:", r4(res_enr))
print("Δ (enr - base):", {
    "salvage_rate_pct": round(res_enr["salvage_rate_pct"]-res_base["salvage_rate_pct"], 2),
    "avg_extra_km":     (round(res_enr["avg_extra_km"]-res_base["avg_extra_km"], 3)
                          if all(isinstance(x,(int,float)) and not np.isnan(x) 
                                 for x in [res_enr["avg_extra_km"], res_base["avg_extra_km"]]) else "n/a"),
    "avg_extra_ms":     (round(res_enr["avg_extra_ms"]-res_base["avg_extra_ms"], 3)
                          if all(isinstance(x,(int,float)) and not np.isnan(x) 
                                 for x in [res_enr["avg_extra_ms"], res_base["avg_extra_ms"]]) else "n/a"),
})
# save JSON for tables
import json; OUTM = Path("./outputs/metrics"); OUTM.mkdir(parents=True, exist_ok=True)
with open(OUTM/"reroute_base.json","w") as f: json.dump(res_base, f, indent=2)
with open(OUTM/"reroute_graph.json","w") as f: json.dump(res_enr,  f, indent=2)


QoT-guided Rerouting (GEANT2, zero-shot, HARD)
BASE     : {'model': 'BASE', 'considered': 166, 'salvaged': 38, 'salvage_rate_pct': 22.892, 'avg_extra_km': 212.433, 'avg_extra_ms': 1.062}
+GraphFea: {'model': '+GraphFea', 'considered': 158, 'salvaged': 43, 'salvage_rate_pct': 27.215, 'avg_extra_km': 217.185, 'avg_extra_ms': 1.086}
Δ (enr - base): {'salvage_rate_pct': 4.32, 'avg_extra_km': 4.751, 'avg_extra_ms': 0.024}


In [10]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Step 17 (fixed): k-shot curves (QoT & Failure) with graph-enriched features
# This cell computes gf_* for few-shot rows (if missing), then runs k in {0,10,40,100,200}.
# Outputs:
#   ./outputs/metrics/kshot_qot_enriched.csv
#   ./outputs/metrics/kshot_fail_enriched.csv
#   ./outputs/figs/kshot_qot_enriched.png
#   ./outputs/figs/kshot_fail_enriched.png

import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# ----------------- Load data -----------------
ENRICHED = Path("./outputs/paths_graph_enriched.csv")
FEWSHOT  = Path("./eon_target_fewshot.csv")
LINKS    = Path("./eon_links_timeseries.csv")
assert ENRICHED.exists(), "Run Step 14 first to generate paths_graph_enriched.csv"
assert FEWSHOT.exists(),  "Few-shot CSV not found: eon_target_fewshot.csv"
assert LINKS.exists(),    "Missing eon_links_timeseries.csv in working folder"

df  = pd.read_csv(ENRICHED)       # already has gf_* columns
few = pd.read_csv(FEWSHOT)
links = pd.read_csv(LINKS)        # used to compute gf_* for few-shot

# Splits from enriched paths
train = df[df["split"]=="train_source"].copy()   # NSFNET
test  = df[df["split"]=="test_target"].copy()    # GEANT2 (has gf_*)

# ----------------- Graph feature builder (for few-shot rows) -----------------
lk_idx = links.set_index(["topology","day","edge_id"])

def edge_ids_from_path(path_str):
    ns = [int(x) for x in str(path_str).split("->")]
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(ns[:-1], ns[1:])]

GF_COLS = [
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
    "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]

def compute_graph_feats_for_df(df_in):
    """Return df_in joined with gf_* columns; drops rows where link join fails."""
    rows = []
    miss = 0
    for _, r in df_in.iterrows():
        topo, day, path = r["topology"], r["day"], r["path"]
        eids = edge_ids_from_path(path)
        try:
            rows_link = lk_idx.loc[(topo, day, eids)]
        except KeyError:
            miss += 1
            continue
        hops = len(eids)
        osnrs = rows_link["osnr_db"].values
        utils = rows_link["bandwidth_utilization"].values
        shifts= rows_link["center_freq_offset_ghz"].values
        scales= rows_link["filter_bw_scale"].values
        gf = dict(
            gf_osnr_min = float(osnrs.min()),
            gf_osnr_var = float(np.var(osnrs)) if hops>1 else 0.0,
            gf_util_mean= float(utils.mean()),
            gf_util_max = float(utils.max()),
            gf_shift_max= float(shifts.max()),
            gf_scale_min= float(scales.min()),
            gf_frac_shifted = float((shifts>0).mean()),
            gf_frac_tight   = float((scales<1.0).mean()),
            gf_bot_pos = float(np.argmin(osnrs)/max(1,hops-1))
        )
        base_row = r.to_dict(); base_row.update(gf); rows.append(base_row)
    if miss:
        print(f"[info] compute_graph_feats_for_df: dropped {miss} rows due to missing link joins.")
    return pd.DataFrame(rows) if rows else df_in.assign(**{c: np.nan for c in GF_COLS})

def ensure_gf_columns(df_subset):
    """If gf_* missing in df_subset, compute & return a copy with gf_* present."""
    if all(c in df_subset.columns for c in GF_COLS):
        return df_subset.copy()
    res = compute_graph_feats_for_df(df_subset)
    # If some gf_* still missing (shouldn't), fill with NaN then drop
    for c in GF_COLS:
        if c not in res.columns:
            res[c] = np.nan
    # drop rows with NaNs in gf_* to keep training clean
    res = res.dropna(subset=GF_COLS)
    return res

# ----------------- Features (HARD setup) -----------------
# QoT (HARD): drop req/margin; add gf_*
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
ENR_QOT = BASE_QOT + GF_COLS

# Failure (HARD): base without direct fingerprints; add gf_*
BASE_FAIL = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'symbol_rate_gbaud','bitrate_gbps'
]
ENR_FAIL = BASE_FAIL + GF_COLS

CAT = ['modulation']

# Consistent modulation encoding from train
train_mod_cats = train['modulation'].astype('category').cat.categories.tolist()
def enc_mod(s):
    return pd.Categorical(s, categories=train_mod_cats).codes

def ordered(df_in, cols):
    missing = [c for c in cols if c not in df_in.columns]
    if missing: 
        raise KeyError(f"Missing columns: {missing}")
    return df_in[cols].copy()

# ----------------- Few-shot held-out split -----------------
KEY = ['topology','day','src','dst','path','hops','distance_km']
def make_heldout(few_subset):
    # remove few_subset keys from test to create held-out
    test_all = test.copy()
    test_all['__key__'] = test_all[KEY].astype(str).agg('|'.join, axis=1)
    keys = set(few_subset[KEY].astype(str).agg('|'.join, axis=1))
    return test_all[~test_all['__key__'].isin(keys)].copy()

# ----------------- Generic runner -----------------
def run_kshot(task="qot", k_list=(0,10,40,100,200), seed=42):
    rng = np.random.RandomState(seed)
    results = []
    if task=="qot":
        FEATS = ENR_QOT
        ycol  = 'qot_ok'
        metrics = ["acc","f1","auc"]
    elif task=="fail":
        FEATS = ENR_FAIL
        ycol  = 'failure_present'
        metrics = ["acc","f1"]
    else:
        raise ValueError("task must be 'qot' or 'fail'")

    # Train on NSFNET (already enriched df)
    Xtr = train[FEATS + CAT].copy()
    Xtr['modulation'] = enc_mod(Xtr['modulation'])
    ytr = train[ycol].astype(int).values
    scaler = StandardScaler()
    Xtr_s  = scaler.fit_transform(ordered(Xtr, FEATS + CAT))
    base_clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                             alpha=1e-4, learning_rate_init=1e-3,
                             max_iter=200, random_state=seed)
    base_clf.fit(Xtr_s, ytr)

    # shuffle few-shot pool (balanced file)
    few_pool = few.sample(frac=1.0, random_state=seed).reset_index(drop=True)

    for k in k_list:
        few_k = few_pool.iloc[:min(k, len(few_pool))].copy()
        if k > 0:
            # ensure gf_* present for few-shot subset
            few_k = ensure_gf_columns(few_k)

        held = make_heldout(few_k)  # held from enriched df (already has gf_*)

        # Held-out matrices
        Xho = held[FEATS + CAT].copy()
        Xho['modulation'] = enc_mod(Xho['modulation'])
        yho = held[ycol].astype(int).values
        Xho_s = scaler.transform(ordered(Xho, FEATS + CAT))

        if k == 0:
            clf = base_clf
        else:
            # Build few-shot matrix
            Xfs = few_k[FEATS + CAT].copy()
            Xfs['modulation'] = enc_mod(Xfs['modulation'])
            yfs = few_k[ycol].astype(int).values
            Xfs_s = scaler.transform(ordered(Xfs, FEATS + CAT))
            # Warm-start refit on source + few-shot (stable alternative to partial_fit)
            X_comb = np.vstack([Xtr_s, Xfs_s])
            y_comb = np.concatenate([ytr,  yfs])
            clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                                alpha=1e-4, learning_rate_init=1e-3,
                                max_iter=200, random_state=seed)
            clf.fit(X_comb, y_comb)

        if task=="qot":
            probs = clf.predict_proba(Xho_s)[:,1]
            yhat  = (probs>=0.5).astype(int)
            row = dict(task=task, k=k,
                       acc=accuracy_score(yho, yhat),
                       f1 =f1_score(yho, yhat, average='macro'),
                       auc=roc_auc_score(yho, probs))
        else:
            yhat = clf.predict(Xho_s)
            row = dict(task=task, k=k,
                       acc=accuracy_score(yho, yhat),
                       f1 =f1_score(yho, yhat, average='macro'))
        results.append(row)
        print(f"{task.upper()}  k={k:3d}  ->  " + "  ".join([f"{m}={row[m]:.4f}" for m in metrics]))
    return pd.DataFrame(results)

# ----------------- Run curves & save -----------------
OUTM = Path("./outputs/metrics"); OUTF = Path("./outputs/figs")
OUTM.mkdir(parents=True, exist_ok=True); OUTF.mkdir(parents=True, exist_ok=True)

df_qot  = run_kshot("qot",  k_list=[0,10,40,100,200], seed=42)
df_fail = run_kshot("fail", k_list=[0,10,40,100,200], seed=42)

df_qot.to_csv(OUTM/"kshot_qot_enriched.csv", index=False)
df_fail.to_csv(OUTM/"kshot_fail_enriched.csv", index=False)

# ----------------- Plots -----------------
plt.figure()
plt.plot(df_qot['k'], df_qot['f1'], marker='o', label='QoT F1')
plt.plot(df_qot['k'], df_qot['auc'], marker='s', label='QoT AUC')
plt.xlabel("k (few-shot labels)"); plt.ylabel("score"); plt.title("QoT — k-shot on GEANT2 (graph-enriched, HARD)")
plt.legend(); plt.grid(True, alpha=0.3)
plt.savefig(OUTF/"kshot_qot_enriched.png", bbox_inches="tight"); plt.close()

plt.figure()
plt.plot(df_fail['k'], df_fail['f1'], marker='o', label='Failure F1')
plt.xlabel("k (few-shot labels)"); plt.ylabel("F1"); plt.title("Failure Detection — k-shot on GEANT2 (graph-enriched, HARD)")
plt.legend(); plt.grid(True, alpha=0.3)
plt.savefig(OUTF/"kshot_fail_enriched.png", bbox_inches="tight"); plt.close()

print("Saved:")
print(" -", OUTM/"kshot_qot_enriched.csv")
print(" -", OUTM/"kshot_fail_enriched.csv")
print(" -", OUTF/"kshot_qot_enriched.png")
print(" -", OUTF/"kshot_fail_enriched.png")


QOT  k=  0  ->  acc=0.9598  f1=0.9473  auc=0.9938
QOT  k= 10  ->  acc=0.9662  f1=0.9559  auc=0.9959
QOT  k= 40  ->  acc=0.9681  f1=0.9577  auc=0.9954
QOT  k=100  ->  acc=0.9759  f1=0.9684  auc=0.9976
QOT  k=200  ->  acc=0.9819  f1=0.9758  auc=0.9986
FAIL  k=  0  ->  acc=0.9931  f1=0.9862
FAIL  k= 10  ->  acc=0.9939  f1=0.9876
FAIL  k= 40  ->  acc=0.9951  f1=0.9902
FAIL  k=100  ->  acc=0.9949  f1=0.9895
FAIL  k=200  ->  acc=0.9956  f1=0.9905
Saved:
 - outputs\metrics\kshot_qot_enriched.csv
 - outputs\metrics\kshot_fail_enriched.csv
 - outputs\figs\kshot_qot_enriched.png
 - outputs\figs\kshot_fail_enriched.png


In [3]:
# Step 18: Ablations — Base vs +Graph; telemetry masks; shift/tighten breakdown; optional k=100
# Outputs:
#   ./outputs/metrics/ablations_summary.csv
#   ./outputs/figs/abl_qot.png
#   ./outputs/figs/abl_fail.png
#   ./outputs/figs/abl_fail_shift_tight.png

import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# ---------- Load ----------
ENRICHED = Path("./outputs/paths_graph_enriched.csv")
FEWSHOT  = Path("./eon_target_fewshot.csv")
LINKS    = Path("./eon_links_timeseries.csv")
assert ENRICHED.exists(), "Run Step 14 first to generate paths_graph_enriched.csv"
assert FEWSHOT.exists(),  "Missing eon_target_fewshot.csv"
assert LINKS.exists(),    "Missing eon_links_timeseries.csv"

df   = pd.read_csv(ENRICHED)     # has gf_* columns
few  = pd.read_csv(FEWSHOT)
links= pd.read_csv(LINKS)        # only needed if we must recompute gf_* for few-shot

OUTM = Path("./outputs/metrics"); OUTF = Path("./outputs/figs")
OUTM.mkdir(parents=True, exist_ok=True); OUTF.mkdir(parents=True, exist_ok=True)

# ---------- Splits ----------
train = df[df["split"]=="train_source"].copy()   # NSFNET
test  = df[df["split"]=="test_target"].copy()    # GEANT2 (target)

# ---------- Graph features list ----------
GF_COLS = [
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
    "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]

# ---------- Feature sets (HARD) ----------
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
ENR_QOT = BASE_QOT + GF_COLS

BASE_FAIL = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'symbol_rate_gbaud','bitrate_gbps'
]
ENR_FAIL = BASE_FAIL + GF_COLS

# Telemetry masks:
# - No fingerprints: drop offset/scale summaries/signals
NO_FP_DROP = ['max_center_offset_ghz','min_filter_bw_scale','gf_shift_max','gf_scale_min','gf_frac_shifted','gf_frac_tight']
# - No OSNR: drop OSNR & its gf summaries
NO_OSNR_DROP = ['min_osnr_db','gf_osnr_min','gf_osnr_var']

CAT = ['modulation']

# ---------- Helpers ----------
train_mod_cats = train['modulation'].astype('category').cat.categories.tolist()
def enc_mod(s): return pd.Categorical(s, categories=train_mod_cats).codes
def ordered(df_in, cols):
    miss = [c for c in cols if c not in df_in.columns]
    if miss: raise KeyError(f"Missing columns: {miss}")
    return df_in[cols].copy()

KEY = ['topology','day','src','dst','path','hops','distance_km']
def heldout_after_few(few_subset):
    test_all = test.copy()
    test_all['__key__'] = test_all[KEY].astype(str).agg('|'.join, axis=1)
    keys = set(few_subset[KEY].astype(str).agg('|'.join, axis=1))
    return test_all[~test_all['__key__'].isin(keys)].copy()

def ensure_gf_for_few(df_subset):
    """few-shot CSV may not have gf_*; join from enriched df by composite key."""
    if all(c in df_subset.columns for c in GF_COLS):
        return df_subset.copy()
    join_cols = KEY + ['modulation','qot_ok','failure_present','failure_type','symbol_rate_gbaud','bitrate_gbps',
                       'avg_utilization','min_snr_db','max_center_offset_ghz','min_filter_bw_scale']
    # build key in both, then merge gf_* from enriched df
    tmp_f = df_subset.copy()
    tmp_f['__key__'] = tmp_f[KEY].astype(str).agg('|'.join, axis=1)
    tmp_d = df.copy()
    tmp_d['__key__'] = tmp_d[KEY].astype(str).agg('|'.join, axis=1)
    gf_cols_present = [c for c in GF_COLS if c in tmp_d.columns]
    merged = tmp_f.merge(tmp_d[['__key__']+gf_cols_present], on='__key__', how='left')
    # drop rows where gf_* could not be found
    merged = merged.dropna(subset=gf_cols_present)
    # keep original columns + gf_*
    for c in gf_cols_present:
        if c not in merged.columns: merged[c] = np.nan
    return merged

def train_and_eval(task, feat_list, k=0, subset=None, seed=42):
    """Train on NSFNET; optional k-shot on GEANT2; evaluate on GEANT2 held-out.
       subset: None / 'shift' / 'tighten' (only for failure task)."""
    rng = np.random.RandomState(seed)
    ycol = 'qot_ok' if task=='qot' else 'failure_present'
    # prepare training
    Xtr = train[feat_list + CAT].copy()
    Xtr['modulation'] = enc_mod(Xtr['modulation'])
    ytr = train[ycol].astype(int).values
    scaler = StandardScaler()
    Xtr_s = scaler.fit_transform(ordered(Xtr, feat_list + CAT))
    base_clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                             alpha=1e-4, learning_rate_init=1e-3,
                             max_iter=200, random_state=seed)
    base_clf.fit(Xtr_s, ytr)
    # few-shot subset (balanced file); ensure gf_* present if using ENR features
    few_pool = few.sample(frac=1.0, random_state=seed).reset_index(drop=True)
    few_k = few_pool.iloc[:min(k, len(few_pool))].copy()
    if k>0 and any(c.startswith('gf_') for c in feat_list):
        few_k = ensure_gf_for_few(few_k)
    held = heldout_after_few(few_k)
    # optional subset filter (only for failure task)
    if subset in ('shift','tighten'):
        held = held[(held['failure_present']==1) & (held['failure_type']==subset)].copy()
        if len(held)==0:
            return dict(acc=np.nan, f1=np.nan, auc=(np.nan if task=='qot' else None), n=0)
    # build held matrices
    Xho = held[feat_list + CAT].copy()
    Xho['modulation'] = enc_mod(Xho['modulation'])
    yho = held[ycol].astype(int).values
    Xho_s = scaler.transform(ordered(Xho, feat_list + CAT))
    # choose classifier: base or refit with few-shot
    clf = base_clf
    if k>0:
        Xfs = few_k[feat_list + CAT].copy()
        Xfs['modulation'] = enc_mod(Xfs['modulation'])
        yfs = few_k[ycol].astype(int).values
        Xfs_s = scaler.transform(ordered(Xfs, feat_list + CAT))
        Xc = np.vstack([Xtr_s, Xfs_s])
        yc = np.concatenate([ytr, yfs])
        clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                            alpha=1e-4, learning_rate_init=1e-3,
                            max_iter=200, random_state=seed)
        clf.fit(Xc, yc)
    # eval
    if task=='qot':
        prob = clf.predict_proba(Xho_s)[:,1]
        yhat = (prob>=0.5).astype(int)
        return dict(acc=accuracy_score(yho,yhat), f1=f1_score(yho,yhat,average='macro'),
                    auc=roc_auc_score(yho,prob), n=len(held))
    else:
        yhat = clf.predict(Xho_s)
        return dict(acc=accuracy_score(yho,yhat), f1=f1_score(yho,yhat,average='macro'),
                    n=len(held))

# ---------- Build scenarios ----------
scenarios = []

# QoT: Base / +Graph / +Graph-NoFP / +Graph-NoOSNR (k=0); +Graph (k=100)
qot_sets = {
    "base": ENR_QOT.copy(),   # start from enriched list to make drop logic easy, then remove gf_* for base
}
# For QoT base we want BASE_QOT only:
qot_sets["base"] = BASE_QOT
qot_sets["+graph"] = ENR_QOT
qot_sets["+graph_noFP"] = [c for c in ENR_QOT if c not in NO_FP_DROP]
qot_sets["+graph_noOSNR"] = [c for c in ENR_QOT if c not in NO_OSNR_DROP]

for name, feats in qot_sets.items():
    m = train_and_eval("qot", feats, k=0, subset=None)
    scenarios.append(dict(task="qot", scenario=name, k=0, **m))
# few-shot k=100 on +graph
m = train_and_eval("qot", ENR_QOT, k=100, subset=None)
scenarios.append(dict(task="qot", scenario="+graph", k=100, **m))

# Failure: Base / +Graph / +Graph-NoFP / +Graph-NoOSNR (k=0); +Graph (k=100)
fail_sets = {
    "base": BASE_FAIL,
    "+graph": ENR_FAIL,
    "+graph_noFP": [c for c in ENR_FAIL if c not in NO_FP_DROP],
    "+graph_noOSNR": [c for c in ENR_FAIL if c not in NO_OSNR_DROP]
}
for name, feats in fail_sets.items():
    m = train_and_eval("fail", feats, k=0, subset=None)
    scenarios.append(dict(task="fail", scenario=name, k=0, auc=None, **m))
# few-shot k=100 on +graph
m = train_and_eval("fail", ENR_FAIL, k=100, subset=None)
scenarios.append(dict(task="fail", scenario="+graph", k=100, auc=None, **m))

# Failure per-class breakdown (shift vs tighten) for +graph (k=0 and k=100)
for kshot in (0,100):
    for cls in ("shift","tighten"):
        m = train_and_eval("fail", ENR_FAIL, k=kshot, subset=cls)
        scenarios.append(dict(task="fail", scenario=f"+graph_{cls}", k=kshot, auc=None, **m))

# ---------- Save summary ----------
df_sum = pd.DataFrame(scenarios)
df_sum.to_csv(OUTM/"ablations_summary.csv", index=False)
print("Saved:", OUTM/"ablations_summary.csv")
display(df_sum)

# ---------- Quick plots (paper-friendly) ----------
# (1) QoT F1/AUC bars
plt.figure()
sub = df_sum[(df_sum.task=="qot") & (df_sum.k==0)]
x = np.arange(len(sub))
plt.bar(x-0.15, sub['f1'], width=0.3, label='F1')
plt.bar(x+0.15, sub['auc'], width=0.3, label='AUC')
plt.xticks(x, sub['scenario'], rotation=15); plt.ylabel("score"); plt.title("QoT (GEANT2, zero-shot, HARD)")
plt.legend(); plt.grid(axis='y', alpha=0.3)
plt.savefig(OUTF/"abl_qot.png", bbox_inches="tight"); plt.close()

# (2) Failure F1 bars
plt.figure()
sub = df_sum[(df_sum.task=="fail") & (df_sum.k==0) & (~df_sum['scenario'].str.contains("_"))]
x = np.arange(len(sub))
plt.bar(x, sub['f1'], width=0.5, label='F1')
plt.xticks(x, sub['scenario'], rotation=15); plt.ylabel("F1"); plt.title("Failure Detection (GEANT2, zero-shot, HARD)")
plt.grid(axis='y', alpha=0.3)
plt.savefig(OUTF/"abl_fail.png", bbox_inches="tight"); plt.close()

# (3) Shift vs Tighten (Failure) — +graph @ k=0 vs k=100
plt.figure()
sub0 = df_sum[(df_sum.task=="fail") & (df_sum.k==0) & (df_sum['scenario'].isin(['+graph_shift','+graph_tighten']))]
sub1 = df_sum[(df_sum.task=="fail") & (df_sum.k==100) & (df_sum['scenario'].isin(['+graph_shift','+graph_tighten']))]
labels = ['shift','tighten']
x = np.arange(len(labels))
f1_0 = [float(sub0[sub0['scenario']==f'+graph_{lab}']['f1']) if not sub0[sub0['scenario']==f'+graph_{lab}'].empty else np.nan for lab in labels]
f1_1 = [float(sub1[sub1['scenario']==f'+graph_{lab}']['f1']) if not sub1[sub1['scenario']==f'+graph_{lab}'].empty else np.nan for lab in labels]
plt.bar(x-0.15, f1_0, width=0.3, label='k=0')
plt.bar(x+0.15, f1_1, width=0.3, label='k=100')
plt.xticks(x, labels); plt.ylabel("F1"); plt.title("Failure Detection — shift vs tighten (+graph)")
plt.legend(); plt.grid(axis='y', alpha=0.3)
plt.ylim(0,1.0)
plt.savefig(OUTF/"abl_fail_shift_tight.png", bbox_inches="tight"); plt.close()

print("Saved plots:")
print(" -", OUTF/"abl_qot.png")
print(" -", OUTF/"abl_fail.png")
print(" -", OUTF/"abl_fail_shift_tight.png")


C:\Users\devon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\devon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Saved: outputs\metrics\ablations_summary.csv


,task,scenario,k,acc,f1,auc,n
0,qot,base,0,0.980926,0.975380,0.998353,5400
1,qot,+graph,0,0.959815,0.947331,0.993752,5400
2,qot,+graph_noFP,0,0.961667,0.947639,0.995775,5400
3,qot,+graph_noOSNR,0,0.959444,0.944633,0.993406,5400
4,qot,+graph,100,0.974533,0.965945,0.995881,5301
5,fail,base,0,0.841481,0.493459,NaN,5400
6,fail,+graph,0,0.993148,0.986161,NaN,5400
7,fail,+graph_noFP,0,0.818148,0.488958,NaN,5400
8,fail,+graph_noOSNR,0,0.996111,0.992187,NaN,5400
9,fail,+graph,100,0.995284,0.990244,NaN,5301


Saved plots:
 - outputs\figs\abl_qot.png
 - outputs\figs\abl_fail.png
 - outputs\figs\abl_fail_shift_tight.png


C:\Users\devon\AppData\Local\Temp\ipykernel_29000\3360143247.py:225: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  f1_0 = [float(sub0[sub0['scenario']==f'+graph_{lab}']['f1']) if not sub0[sub0['scenario']==f'+graph_{lab}'].empty else np.nan for lab in labels]
C:\Users\devon\AppData\Local\Temp\ipykernel_29000\3360143247.py:226: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  f1_1 = [float(sub1[sub1['scenario']==f'+graph_{lab}']['f1']) if not sub1[sub1['scenario']==f'+graph_{lab}'].empty else np.nan for lab in labels]


In [5]:
# Step 19 — Build a paper_pack/ with metrics + figures + summary README
import os, json, shutil
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path(".")
SRC_MET = ROOT/"outputs/metrics"
SRC_FIG = ROOT/"outputs/figs"

PACK = ROOT/"paper_pack"
PMET  = PACK/"metrics"
PFIG  = PACK/"figs"
PACK.mkdir(exist_ok=True); PMET.mkdir(parents=True, exist_ok=True); PFIG.mkdir(parents=True, exist_ok=True)

# ---- files to collect (copy only if present) ----
metrics_files = [
    "reroute_base.json",
    "reroute_graph.json",
    "reroute_summary.csv",
    "kshot_qot_enriched.csv",
    "kshot_fail_enriched.csv",
    "ablations_summary.csv",
]
figs_files = [
    "qot_roc_zero_vs_tta.png",
    "qot_reliability_zero_vs_tta.png",
    "qot_risk_coverage_zero_vs_tta.png",
    "kshot_qot_enriched.png",
    "kshot_fail_enriched.png",
    "abl_qot.png",
    "abl_fail.png",
    "abl_fail_shift_tight.png",
]

copied = {"metrics": [], "figs": [], "missing": []}

def try_copy(src_dir, name, dst_dir):
    p = src_dir/name
    if p.exists():
        shutil.copy2(p, dst_dir/name)
        return True
    else:
        copied["missing"].append(str(p))
        return False

for f in metrics_files:
    if try_copy(SRC_MET, f, PMET):
        copied["metrics"].append(f)

for f in figs_files:
    if try_copy(SRC_FIG, f, PFIG):
        copied["figs"].append(f)

# ---- summarize key results into a manifest ----
manifest = {"files": copied, "highlights": {}}

# Rerouting summary
rer_base = PMET/"reroute_base.json"
rer_graph= PMET/"reroute_graph.json"
if rer_base.exists() and rer_graph.exists():
    jb = json.loads(rer_base.read_text())
    jg = json.loads(rer_graph.read_text())
    manifest["highlights"]["rerouting"] = {
        "considered_base": jb.get("considered"),
        "considered_graph": jg.get("considered"),
        "salvage_rate_base_pct": round(jb.get("salvage_rate_pct", float("nan")), 3),
        "salvage_rate_graph_pct": round(jg.get("salvage_rate_pct", float("nan")), 3),
        "delta_salvage_pp": round(jg.get("salvage_rate_pct",0) - jb.get("salvage_rate_pct",0), 3),
        "avg_extra_km_base": round(jb.get("avg_extra_km", float("nan")), 3) if isinstance(jb.get("avg_extra_km"), (int,float)) else None,
        "avg_extra_km_graph": round(jg.get("avg_extra_km", float("nan")), 3) if isinstance(jg.get("avg_extra_km"), (int,float)) else None,
        "avg_extra_ms_base": round(jb.get("avg_extra_ms", float("nan")), 3) if isinstance(jb.get("avg_extra_ms"), (int,float)) else None,
        "avg_extra_ms_graph": round(jg.get("avg_extra_ms", float("nan")), 3) if isinstance(jg.get("avg_extra_ms"), (int,float)) else None,
    }

# k-shot curves summary
kq = PMET/"kshot_qot_enriched.csv"
kf = PMET/"kshot_fail_enriched.csv"
def kshot_summary(csv_path, cols):
    if not csv_path.exists(): return None
    df = pd.read_csv(csv_path)
    df = df.sort_values("k")
    return df[["k"]+cols].to_dict(orient="records")

manifest["highlights"]["kshot_qot"]  = kshot_summary(kq, ["f1","auc"]) if kq.exists() else None
manifest["highlights"]["kshot_fail"] = kshot_summary(kf, ["f1"])       if kf.exists() else None

# Ablations quick picks
abl = PMET/"ablations_summary.csv"
if abl.exists():
    df = pd.read_csv(abl)
    def pick(task, scenario, k=0, cols=("acc","f1","auc","n")):
        row = df[(df.task==task) & (df.scenario==scenario) & (df.k==k)]
        if row.empty: return None
        r = row.iloc[0].to_dict()
        return {c: (float(r[c]) if c in r and pd.notna(r[c]) else None) for c in cols}
    manifest["highlights"]["ablations"] = {
        "qot_base_k0":   pick("qot","base",0),
        "qot_graph_k0":  pick("qot","+graph",0),
        "qot_graph_k100":pick("qot","+graph",100),
        "fail_base_k0":  pick("fail","base",0, cols=("acc","f1","n")),
        "fail_graph_k0": pick("fail","+graph",0, cols=("acc","f1","n")),
        "fail_graph_noFP_k0": pick("fail","+graph_noFP",0, cols=("acc","f1","n")),
        "fail_graph_noOSNR_k0": pick("fail","+graph_noOSNR",0, cols=("acc","f1","n")),
        "fail_graph_k100": pick("fail","+graph",100, cols=("acc","f1","n")),
    }

# Save manifest
with open(PACK/"manifest.json","w") as f:
    json.dump(manifest, f, indent=2)

# ---- Write a compact README.md with headline numbers ----
lines = []
lines.append("# Paper Pack")
lines.append("")
lines.append("This folder aggregates metrics and figures generated in Steps 13–18.")
lines.append("")
if "rerouting" in manifest["highlights"]:
    rr = manifest["highlights"]["rerouting"]
    lines += [
        "## QoT-guided Rerouting (GEANT2, zero-shot, HARD)",
        f"- BASE salvage: **{rr['salvage_rate_base_pct']}%** ; +GraphFea: **{rr['salvage_rate_graph_pct']}%** ; Δ = **{rr['delta_salvage_pp']} pp**",
        f"- Overhead (avg): BASE **{rr['avg_extra_km_base']} km / {rr['avg_extra_ms_base']} ms**, +GraphFea **{rr['avg_extra_km_graph']} km / {rr['avg_extra_ms_graph']} ms**",
        ""
    ]
if manifest["highlights"].get("kshot_qot"):
    lines.append("## k-shot (QoT, graph-enriched, HARD)")
    lines.append("| k | F1 | AUC |"); lines.append("|---:|---:|---:|")
    for r in manifest["highlights"]["kshot_qot"]:
        lines.append(f"| {r['k']} | {r['f1']:.4f} | {r['auc']:.4f} |")
    lines.append("")
if manifest["highlights"].get("kshot_fail"):
    lines.append("## k-shot (Failure detection, graph-enriched, HARD)")
    lines.append("| k | F1 |"); lines.append("|---:|---:|")
    for r in manifest["highlights"]["kshot_fail"]:
        lines.append(f"| {r['k']} | {r['f1']:.4f} |")
    lines.append("")
if manifest["highlights"].get("ablations"):
    ab = manifest["highlights"]["ablations"]
    lines += [
        "## Ablations (Zero-shot unless noted)",
        f"- QoT: Base k=0 F1 **{ab['qot_base_k0']['f1']:.4f}**, +Graph k=0 F1 **{ab['qot_graph_k0']['f1']:.4f}**, +Graph k=100 F1 **{ab['qot_graph_k100']['f1']:.4f}**.",
        f"- Failure: Base k=0 F1 **{ab['fail_base_k0']['f1']:.4f}**, +Graph k=0 F1 **{ab['fail_graph_k0']['f1']:.4f}**, +Graph(noFP) k=0 F1 **{ab['fail_graph_noFP_k0']['f1']:.4f}**, +Graph(noOSNR) k=0 F1 **{ab['fail_graph_noOSNR_k0']['f1']:.4f}**; +Graph k=100 F1 **{ab['fail_graph_k100']['f1']:.4f}**.",
        ""
    ]
lines += [
    "## Figures",
    "- ROC: `figs/qot_roc_zero_vs_tta.png`, Reliability: `figs/qot_reliability_zero_vs_tta.png`, Risk–Coverage: `figs/qot_risk_coverage_zero_vs_tta.png`",
    "- k-shot: `figs/kshot_qot_enriched.png`, `figs/kshot_fail_enriched.png`",
    "- Ablations: `figs/abl_qot.png`, `figs/abl_fail.png`, `figs/abl_fail_shift_tight.png`",
    "",
    "## Files included",
    f"- Metrics: {', '.join(copied['metrics'])}",
    f"- Figures: {', '.join(copied['figs'])}",
    "",
    "_Auto-generated by Step 19._"
]
(PACK/"README.md").write_text("\n".join(lines), encoding="utf-8")

print("Pack created at:", PACK.resolve())
print("Copied metrics:", copied['metrics'])
print("Copied figures:", copied['figs'])
if copied["missing"]:
    print("\nMissing (not found, skipped):")
    for m in copied["missing"]:
        print(" -", m)
else:
    print("\nAll expected files found.")


Pack created at: C:\devonboard\research\daily taskk\EACE2025\by gpt\paper_pack
Copied metrics: ['reroute_base.json', 'reroute_graph.json', 'reroute_summary.csv', 'kshot_qot_enriched.csv', 'kshot_fail_enriched.csv', 'ablations_summary.csv']
Copied figures: ['qot_roc_zero_vs_tta.png', 'qot_reliability_zero_vs_tta.png', 'qot_risk_coverage_zero_vs_tta.png', 'kshot_qot_enriched.png', 'kshot_fail_enriched.png', 'abl_qot.png', 'abl_fail.png', 'abl_fail_shift_tight.png']

All expected files found.


In [6]:
# Step 20: Case-study time series (2-day window) with Zero-shot vs TTA-lite QoT predictions
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# ----------------- Load enriched data -----------------
ENRICHED = Path("./outputs/paths_graph_enriched.csv")
assert ENRICHED.exists(), "Run Step 14 first to create paths_graph_enriched.csv"
df = pd.read_csv(ENRICHED)

# Focus on target domain
test = df[(df["split"]=="test_target") & (df["topology"]=="GEANT2")].copy()
train = df[df["split"]=="train_source"].copy()  # NSFNET for training

# ----------------- Feature schema (HARD + graph-aware) -----------------
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
GF_COLS = [
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
    "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]
FEATS = BASE_QOT + GF_COLS
CAT   = ['modulation']
COL_ORDER = FEATS + CAT

# Consistent modulation encoding (fit on train)
train_mod_cats = train['modulation'].astype('category').cat.categories.tolist()
def enc_mod(s): return pd.Categorical(s, categories=train_mod_cats).codes
def ordered(df_in): return df_in[COL_ORDER].copy()

# ----------------- Train base QoT model on NSFNET -----------------
Xtr = train[COL_ORDER].copy()
Xtr['modulation'] = enc_mod(Xtr['modulation'])
ytr = train['qot_ok'].astype(int).values

sc_src = StandardScaler()
Xtr_s  = sc_src.fit_transform(ordered(Xtr))

clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                    alpha=1e-4, learning_rate_init=1e-3,
                    max_iter=200, random_state=42)
clf.fit(Xtr_s, ytr)

# ----------------- Pick a "loud" path (max telemetry variance & failures) -----------------
grp = (test
       .assign(var_shift=test['gf_shift_max'].fillna(0.0).groupby([test['src'],test['dst'],test['path']]).transform('var'),
               var_scale=test['gf_scale_min'].fillna(1.0).groupby([test['src'],test['dst'],test['path']]).transform('var'),
               fail_cnt=test['failure_present'].groupby([test['src'],test['dst'],test['path']]).transform('sum'))
       .copy())

# score = more failures + more variability
grp['score'] = grp['fail_cnt'].fillna(0) + grp['var_shift'].fillna(0) + grp['var_scale'].fillna(0)
key_cols = ['src','dst','path']
best_key = (grp.sort_values('score', ascending=False)[key_cols].iloc[0].to_dict())

case = test[(test['src']==best_key['src']) & (test['dst']==best_key['dst']) & (test['path']==best_key['path'])].copy()
case_days = sorted(case['day'].unique())
# pick a 2-day window around the first failure day if possible
fail_days = sorted(case[case['failure_present']==1]['day'].unique())
if fail_days:
    d0 = fail_days[0]
else:
    # no failure: just take earliest two consecutive days
    d0 = case_days[0]
d1 = d0 + 1
if d1 not in case_days:  # fallbacks
    d1 = d0
    d0 = max(min(case_days), d1-1)

window = case[case['day'].isin([d0, d1])].sort_values('day').copy()
title_id = f"{int(best_key['src'])}->{int(best_key['dst'])}  path={best_key['path']}  days {d0}-{d1}"

# ----------------- Zero-shot vs TTA-lite predictions across the window -----------------
def bn_adapt_batch(X_raw, sc, gamma=0.3):
    mu_src, sd_src = sc.mean_, sc.scale_
    mu_b  = X_raw.mean(axis=0)
    sd_b  = X_raw.std(axis=0, ddof=0) + 1e-6
    mu = (1-gamma)*mu_src + gamma*mu_b
    sd = (1-gamma)*sd_src + gamma*sd_b
    return (X_raw - mu)/sd

def tta_probs_for_day(day_rows, day_batch_all, aug_n=5, jitter=0.03, gamma=0.3):
    # day_rows: DataFrame (subset of the case path for this day)
    # day_batch_all: ALL GEANT2 rows for the same day (for batch stats)
    Xb_raw = day_batch_all[COL_ORDER].copy()
    Xb_raw['modulation'] = enc_mod(Xb_raw['modulation'])
    Xb_raw = ordered(Xb_raw).values
    Xb_s   = bn_adapt_batch(Xb_raw, sc_src, gamma=gamma)

    # We need indices of our case rows inside batch to extract their probs
    Xday = day_rows[COL_ORDER].copy()
    Xday['modulation'] = enc_mod(Xday['modulation'])
    Xday = ordered(Xday).values

    # Map day_rows into the same normalized space using batch stats:
    # (recompute mean/std from Xb_raw to avoid mismatch)
    mu_src, sd_src = sc_src.mean_, sc_src.scale_
    mu_b  = Xb_raw.mean(axis=0); sd_b = Xb_raw.std(axis=0, ddof=0)+1e-6
    mu = (1-gamma)*mu_src + gamma*mu_b
    sd = (1-gamma)*sd_src + gamma*sd_b
    Xday_s = (Xday - mu)/sd

    # Monte-Carlo jitter + averaging
    rng = np.random.RandomState(7)
    probs = []
    for _ in range(aug_n):
        Xj = Xday_s.copy()
        noise = rng.normal(0.0, jitter, size=Xj[:,:-1].shape)  # don't jitter modulation (last col)
        Xj[:,:-1] = Xj[:,:-1]*(1+noise)
        probs.append(clf.predict_proba(Xj)[:,1])
    return np.mean(probs, axis=0)

# Build per-day frames
rows = []
for day in [d0, d1]:
    day_all = test[test['day']==day].copy()
    day_case = window[window['day']==day].copy()
    if len(day_case)==0: 
        continue
    # zero-shot
    Xz = day_case[COL_ORDER].copy()
    Xz['modulation'] = enc_mod(Xz['modulation'])
    Xz_s = sc_src.transform(ordered(Xz))
    p_zero = clf.predict_proba(Xz_s)[:,1]
    # TTA-lite (batch = all GEANT2 rows of that day)
    p_tta  = tta_probs_for_day(day_case, day_all, aug_n=5, jitter=0.03, gamma=0.3)
    for i,(ix,r) in enumerate(day_case.iterrows()):
        rows.append({
            "day": int(day),
            "gf_shift_max": float(r["gf_shift_max"]),
            "gf_scale_min": float(r["gf_scale_min"]),
            "gf_osnr_min":  float(r["gf_osnr_min"]),
            "failure_present": int(r["failure_present"]),
            "failure_type": str(r["failure_type"]),
            "p_zero": float(p_zero[i]),
            "p_tta":  float(p_tta[i]),
        })
ts = pd.DataFrame(rows).sort_values("day")

# ----------------- Save CSV + Plot -----------------
OUTF = Path("./outputs/figs"); OUTM = Path("./outputs/metrics")
OUTF.mkdir(parents=True, exist_ok=True); OUTM.mkdir(parents=True, exist_ok=True)
csv_path = OUTM/f"case_study_days_{d0}_{d1}_sd_{best_key['src']}_{best_key['dst']}.csv"
ts.to_csv(csv_path, index=False)

plt.figure(figsize=(9,7))

# (1) Shift / Scale
plt.subplot(3,1,1)
plt.plot(ts['day'], ts['gf_shift_max'], marker='o', label='gf_shift_max')
plt.plot(ts['day'], ts['gf_scale_min'], marker='s', label='gf_scale_min')
for d,fp in zip(ts['day'], ts['failure_present']):
    if fp==1: plt.axvspan(d-0.5, d+0.5, alpha=0.1)
plt.ylabel("shift / scale")
plt.title(f"Case Study: {title_id}")
plt.legend()

# (2) Bottleneck OSNR
plt.subplot(3,1,2)
plt.plot(ts['day'], ts['gf_osnr_min'], marker='o', label='gf_osnr_min')
for d,fp in zip(ts['day'], ts['failure_present']):
    if fp==1: plt.axvspan(d-0.5, d+0.5, alpha=0.1)
plt.ylabel("OSNR (min)")
plt.legend()

# (3) QoT probability: Zero-shot vs TTA-lite
plt.subplot(3,1,3)
plt.plot(ts['day'], ts['p_zero'], marker='o', label='Zero-shot P(QoT OK)')
plt.plot(ts['day'], ts['p_tta'], marker='s', label='TTA-lite P(QoT OK)')
plt.axhline(0.5, linestyle='--', linewidth=1)
for d,fp in zip(ts['day'], ts['failure_present']):
    if fp==1: plt.axvspan(d-0.5, d+0.5, alpha=0.1)
plt.xlabel("day"); plt.ylabel("probability")
plt.legend()

png_path = OUTF/f"case_study_days_{d0}_{d1}_sd_{best_key['src']}_{best_key['dst']}.png"
plt.tight_layout(); plt.savefig(png_path, bbox_inches="tight"); plt.close()

print("Saved case-study:")
print(" - CSV:", csv_path)
print(" - PNG:", png_path)
print("Picked path:", title_id)


Saved case-study:
 - CSV: outputs\metrics\case_study_days_57_58_sd_1_22.csv
 - PNG: outputs\figs\case_study_days_57_58_sd_1_22.png
Picked path: 1->22  path=1->8->5->4->22  days 57-58


In [7]:
# Step 21: Auto-generate a paper-ready "Results Summary" markdown from your saved metrics/figures.
from pathlib import Path
import json, pandas as pd, numpy as np
from datetime import datetime

ROOT = Path(".")
PACK = ROOT/"paper_pack"
PMET  = PACK/"metrics"
PFIG  = PACK/"figs"
OUTM  = ROOT/"outputs"/"metrics"
OUTF  = ROOT/"outputs"/"figs"

# ensure paper_pack exists (if Step 19 not run, create minimal structure)
PACK.mkdir(exist_ok=True); PMET.mkdir(exist_ok=True, parents=True); PFIG.mkdir(exist_ok=True, parents=True)

# -------- helpers --------
def try_read_json(p: Path):
    if p.exists():
        try:
            return json.loads(p.read_text())
        except Exception:
            return None
    return None

def try_read_csv(p: Path):
    if p.exists():
        try:
            return pd.read_csv(p)
        except Exception:
            return None
    return None

def fmt(x, nd=3, pct=False):
    if x is None or (isinstance(x,float) and (np.isnan(x) or np.isinf(x))):
        return "—"
    if pct:
        return f"{x:.{nd}f}%"
    return f"{x:.{nd}f}"

def pick_last(glob_pat, in_dir):
    cands = sorted(in_dir.glob(glob_pat), key=lambda p: p.stat().st_mtime, reverse=True)
    return cands[0] if cands else None

lines = []
lines.append(f"# Results Summary ({datetime.now().strftime('%Y-%m-%d %H:%M')})")
lines.append("")
lines.append("**Novelty (recap):** label-free test-time adaptation (TTA) that boosts zero-shot QoT; "
             "graph-aware localization using link fingerprints; and few-shot label efficiency on GEANT2.")
lines.append("")

# -------- Rerouting (from Step 16) --------
rer_base = PMET/"reroute_base.json"; rer_graph = PMET/"reroute_graph.json"
if not rer_base.exists(): rer_base = OUTM/"reroute_base.json"
if not rer_graph.exists(): rer_graph = OUTM/"reroute_graph.json"
jb, jg = try_read_json(rer_base), try_read_json(rer_graph)

if jb and jg:
    lines += [
        "## QoT-guided Re-routing (GEANT2, zero-shot, HARD)",
        f"- **Salvage rate:** BASE {fmt(jb.get('salvage_rate_pct'),2,pct=True)} → +GraphFea {fmt(jg.get('salvage_rate_pct'),2,pct=True)} "
        f"(Δ **{fmt(jg.get('salvage_rate_pct',0)-jb.get('salvage_rate_pct',0),2)} pp**)",
        f"- **Overhead (avg):** BASE {fmt(jb.get('avg_extra_km'))} km / {fmt(jb.get('avg_extra_ms'))} ms;  "
        f"+GraphFea {fmt(jg.get('avg_extra_km'))} km / {fmt(jg.get('avg_extra_ms'))} ms",
        ""
    ]

# -------- k-shot curves (from Step 17) --------
kq = PMET/"kshot_qot_enriched.csv"; kf = PMET/"kshot_fail_enriched.csv"
if not kq.exists(): kq = OUTM/"kshot_qot_enriched.csv"
if not kf.exists(): kf = OUTM/"kshot_fail_enriched.csv"
df_kq, df_kf = try_read_csv(kq), try_read_csv(kf)

if df_kq is not None:
    df_kq = df_kq.sort_values("k")
    lines += ["## Few-shot (QoT, graph-enriched, HARD)", "", "| k | F1 | AUC |", "|---:|---:|---:|"]
    for _, r in df_kq.iterrows():
        lines.append(f"| {int(r['k'])} | {fmt(r['f1'],4)} | {fmt(r['auc'],4)} |")
    lines.append("")
if df_kf is not None:
    df_kf = df_kf.sort_values("k")
    lines += ["## Few-shot (Failure detection, graph-enriched, HARD)", "", "| k | F1 |", "|---:|---:|"]
    for _, r in df_kf.iterrows():
        lines.append(f"| {int(r['k'])} | {fmt(r['f1'],4)} |")
    lines.append("")

# -------- Ablations (from Step 18) --------
abl = PMET/"ablations_summary.csv"
if not abl.exists(): abl = OUTM/"ablations_summary.csv"
df_abl = try_read_csv(abl)
if df_abl is not None:
    def pick(task, scenario, k=0, cols=("acc","f1","auc","n")):
        row = df_abl[(df_abl.task==task) & (df_abl.scenario==scenario) & (df_abl.k==k)]
        if row.empty: return None
        r = row.iloc[0].to_dict()
        return {c: (float(r[c]) if c in r and pd.notna(r[c]) else None) for c in cols}
    qb = pick("qot","base",0); qg0 = pick("qot","+graph",0); qg100 = pick("qot","+graph",100)
    fb = pick("fail","base",0, cols=("acc","f1","n")); fg0 = pick("fail","+graph",0, cols=("acc","f1","n"))
    fnfp = pick("fail","+graph_noFP",0, cols=("acc","f1","n"))
    fnos = pick("fail","+graph_noOSNR",0, cols=("acc","f1","n"))
    fg100 = pick("fail","+graph",100, cols=("acc","f1","n"))
    lines += [
        "## Ablations (Zero-shot unless noted)",
        f"- **QoT:** Base F1 {fmt(qb['f1'])}, +Graph F1 {fmt(qg0['f1'])}, +Graph (k=100) F1 {fmt(qg100['f1'])}.",
        f"- **Failure:** Base F1 {fmt(fb['f1'])}, +Graph F1 {fmt(fg0['f1'])}, "
        f"+Graph(noFP) F1 {fmt(fnfp['f1'])}, +Graph(noOSNR) F1 {fmt(fnos['f1'])}; "
        f"+Graph (k=100) F1 {fmt(fg100['f1'])}.",
        ""
    ]

# -------- Zero-shot vs TTA (from Step 13) --------
roc_png = pick_last("qot_roc_zero_vs_tta.png", OUTF) or pick_last("qot_roc_zero_vs_tta.png", PFIG)
rel_png = pick_last("qot_reliability_zero_vs_tta.png", OUTF) or pick_last("qot_reliability_zero_vs_tta.png", PFIG)
rc_png  = pick_last("qot_risk_coverage_zero_vs_tta.png", OUTF) or pick_last("qot_risk_coverage_zero_vs_tta.png", PFIG)
if roc_png or rel_png or rc_png:
    lines += ["## Label-free Test-Time Adaptation (QoT)",]
    if roc_png: lines.append(f"- ROC: `{roc_png}`")
    if rel_png: lines.append(f"- Reliability: `{rel_png}`")
    if rc_png:  lines.append(f"- Risk–Coverage: `{rc_png}`")
    lines.append("")

# -------- Case study (from Step 20) --------
case_csv = pick_last("case_study_days_*.csv", OUTM)
case_png = pick_last("case_study_days_*.png", OUTF)
if case_png:
    lines += ["## Case Study (2-day dynamics)", f"- Figure: `{case_png}`"]
    if case_csv:
        try:
            dcs = pd.read_csv(case_csv)
            delta = float(dcs["p_tta"].mean() - dcs["p_zero"].mean())
            lines.append(f"- Avg Δ P(QoT OK), TTA − Zero-shot: **{fmt(delta,3)}**")
        except Exception:
            pass
    lines.append("")

# -------- Write files --------
out_md = PACK/"results_summary.md"
out_md.write_text("\n".join(lines), encoding="utf-8")

print("Wrote:", out_md.resolve())
print("\nPreview (first 40 lines):\n")
print("\n".join(lines[:40]))


Wrote: C:\devonboard\research\daily taskk\EACE2025\by gpt\paper_pack\results_summary.md

Preview (first 40 lines):

# Results Summary (2025-08-22 15:37)

**Novelty (recap):** label-free test-time adaptation (TTA) that boosts zero-shot QoT; graph-aware localization using link fingerprints; and few-shot label efficiency on GEANT2.

## QoT-guided Re-routing (GEANT2, zero-shot, HARD)
- **Salvage rate:** BASE 22.89% → +GraphFea 27.22% (Δ **4.32 pp**)
- **Overhead (avg):** BASE 212.433 km / 1.062 ms;  +GraphFea 217.185 km / 1.086 ms

## Few-shot (QoT, graph-enriched, HARD)

| k | F1 | AUC |
|---:|---:|---:|
| 0 | 0.9473 | 0.9938 |
| 10 | 0.9559 | 0.9959 |
| 40 | 0.9577 | 0.9954 |
| 100 | 0.9684 | 0.9976 |
| 200 | 0.9758 | 0.9986 |

## Few-shot (Failure detection, graph-enriched, HARD)

| k | F1 |
|---:|---:|
| 0 | 0.9862 |
| 10 | 0.9876 |
| 40 | 0.9902 |
| 100 | 0.9895 |
| 200 | 0.9905 |

## Ablations (Zero-shot unless noted)
- **QoT:** Base F1 0.975, +Graph F1 0.947, +Graph (k=100) F1 0.966

In [9]:
# Step 22 (alt): Print results inline (no LaTeX) — rerouting, k-shot, ablations.
# It reads metrics from ./outputs/metrics or paper_pack/metrics and prints tidy tables.

from pathlib import Path
import pandas as pd, numpy as np, json

pd.set_option("display.max_colwidth", 120)
pd.set_option("display.width", 120)

ROOT = Path(".")
OUTM = ROOT/"outputs"/"metrics"
PPM  = ROOT/"paper_pack"/"metrics"
OUTM.mkdir(parents=True, exist_ok=True)

def pick(*paths: Path):
    for p in paths:
        if p.exists(): return p
    return None

def fmt_num(x, nd=3):
    if x is None: return np.nan
    try:
        xf = float(x)
        if np.isnan(xf) or np.isinf(xf): return np.nan
        return round(xf, nd)
    except Exception:
        return x

# ---------- 1) QoT-guided re-routing (BASE vs +GraphFea) ----------
rer_base_p  = pick(PPM/"reroute_base.json", OUTM/"reroute_base.json")
rer_graph_p = pick(PPM/"reroute_graph.json", OUTM/"reroute_graph.json")

if rer_base_p and rer_graph_p:
    jb = json.loads(rer_base_p.read_text()); jg = json.loads(rer_graph_p.read_text())
    rer_df = pd.DataFrame([
        dict(Model="BASE",
             Considered=jb.get("considered"), Salvaged=jb.get("salvaged"),
             Salvage_pct=fmt_num(jb.get("salvage_rate_pct"),2),
             Overhead_km=fmt_num(jb.get("avg_extra_km")), Overhead_ms=fmt_num(jb.get("avg_extra_ms"))),
        dict(Model="+GraphFea",
             Considered=jg.get("considered"), Salvaged=jg.get("salvaged"),
             Salvage_pct=fmt_num(jg.get("salvage_rate_pct"),2),
             Overhead_km=fmt_num(jg.get("avg_extra_km")), Overhead_ms=fmt_num(jg.get("avg_extra_ms"))),
        dict(Model="Δ (+GraphFea - BASE)",
             Considered="", Salvaged="",
             Salvage_pct=fmt_num(jg.get("salvage_rate_pct",0)-jb.get("salvage_rate_pct",0),2),
             Overhead_km=(fmt_num(jg.get("avg_extra_km")-jb.get("avg_extra_km")) if all(isinstance(x,(int,float)) for x in [jg.get("avg_extra_km"),jb.get("avg_extra_km")]) else ""),
             Overhead_ms=(fmt_num(jg.get("avg_extra_ms")-jb.get("avg_extra_ms")) if all(isinstance(x,(int,float)) for x in [jg.get("avg_extra_ms"),jb.get("avg_extra_ms")]) else "")),
    ])
    print("\n=== QoT-guided Re-routing (GEANT2, zero-shot, HARD) ===")
    display(rer_df)
else:
    print("Rerouting metrics not found. Run Step 16 + save step first.")

# ---------- 2) k-shot curves (QoT & Failure) ----------
kq_p = pick(PPM/"kshot_qot_enriched.csv", OUTM/"kshot_qot_enriched.csv")
kf_p = pick(PPM/"kshot_fail_enriched.csv", OUTM/"kshot_fail_enriched.csv")

if kq_p and kq_p.exists():
    kq = pd.read_csv(kq_p).sort_values("k")
    kq_out = kq[["k","f1","auc"]].copy()
    kq_out["f1"]  = kq_out["f1"].map(lambda x: round(x,4))
    kq_out["auc"] = kq_out["auc"].map(lambda x: round(x,4))
    print("\n=== Few-shot (QoT, graph-enriched, HARD) ===")
    display(kq_out)
else:
    print("\nQoT k-shot file not found.")

if kf_p and kf_p.exists():
    kf = pd.read_csv(kf_p).sort_values("k")
    kf_out = kf[["k","f1"]].copy()
    kf_out["f1"] = kf_out["f1"].map(lambda x: round(x,4))
    print("\n=== Few-shot (Failure detection, graph-enriched, HARD) ===")
    display(kf_out)
else:
    print("\nFailure k-shot file not found.")

# ---------- 3) Ablations summary (QoT & Failure) ----------
abl_p = pick(PPM/"ablations_summary.csv", OUTM/"ablations_summary.csv")
if abl_p and abl_p.exists():
    abl = pd.read_csv(abl_p)

    # QoT: k=0 rows + k=100 for +graph
    q0 = abl[(abl.task=="qot") & ((abl.k==0) | ((abl.k==100) & (abl.scenario=="+graph")))].copy()
    qtab = (q0[["scenario","k","acc","f1","auc"]]
            .assign(acc=lambda d: d["acc"].round(4),
                    f1=lambda d: d["f1"].round(4),
                    auc=lambda d: d["auc"].round(4)))
    print("\n=== QoT Ablations (GEANT2, HARD) ===")
    display(qtab.reset_index(drop=True))

    # Failure: k=0 rows + k=100 for +graph
    f0 = abl[(abl.task=="fail") & ((abl.k==0) | ((abl.k==100) & (abl.scenario=="+graph")))].copy()
    ftab = (f0[["scenario","k","acc","f1","n"]]
            .assign(acc=lambda d: d["acc"].round(4),
                    f1=lambda d: d["f1"].round(4)))
    print("\n=== Failure Ablations (GEANT2, HARD) ===")
    display(ftab.reset_index(drop=True))

    # (Optional) If you also want per-class rows that exist in the CSV:
    per_cls = abl[(abl.task=="fail") & (abl["scenario"].isin(["+graph_shift","+graph_tighten"]))].copy()
    if not per_cls.empty:
        pctab = (per_cls[["scenario","k","acc","f1","n"]]
                 .assign(acc=lambda d: d["acc"].round(4),
                         f1=lambda d: d["f1"].round(4)))
        print("\n=== Failure per-class (shift/tighten) — macro-F1 shown (see note) ===")
        display(pctab.reset_index(drop=True))
        print("Note: macro-F1 looks ~0.5 because these subsets contain only positives.\n"
              "If you want positive-class F1, run the small helper cell I shared earlier.")
else:
    print("\nAblations file not found. Run Step 18 first.")



=== QoT-guided Re-routing (GEANT2, zero-shot, HARD) ===


,Model,Considered,Salvaged,Salvage_pct,Overhead_km,Overhead_ms
0,BASE,166,38,22.89,212.433,1.062
1,+GraphFea,158,43,27.22,217.185,1.086
2,Δ (+GraphFea - BASE),,,4.32,4.751,0.024



=== Few-shot (QoT, graph-enriched, HARD) ===


,k,f1,auc
0,0,0.9473,0.9938
1,10,0.9559,0.9959
2,40,0.9577,0.9954
3,100,0.9684,0.9976
4,200,0.9758,0.9986



=== Few-shot (Failure detection, graph-enriched, HARD) ===


...


=== Failure Ablations (GEANT2, HARD) ===


,scenario,k,acc,f1,n
0,base,0,0.8415,0.4935,5400
1,+graph,0,0.9931,0.9862,5400
2,+graph_noFP,0,0.8181,0.4890,5400
3,+graph_noOSNR,0,0.9961,0.9922,5400
4,+graph,100,0.9953,0.9902,5301
5,+graph_shift,0,0.9791,0.4947,479
6,+graph_tighten,0,0.9453,0.4860,311



=== Failure per-class (shift/tighten) — macro-F1 shown (see note) ===


,scenario,k,acc,f1,n
0,+graph_shift,0,0.9791,0.4947,479
1,+graph_tighten,0,0.9453,0.4860,311
2,+graph_shift,100,0.9956,0.4989,452
3,+graph_tighten,100,0.9792,0.4948,289


Note: macro-F1 looks ~0.5 because these subsets contain only positives.
If you want positive-class F1, run the small helper cell I shared earlier.


In [10]:
# Step 23 — Bootstrap 95% CIs for key metrics: QoT AUC (Zero vs TTA), Failure F1 (+Graph), Rerouting salvage (BASE vs +Graph)
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.utils import resample
import warnings; warnings.filterwarnings("ignore")

# ------------------ Load data ------------------
ENRICHED = Path("./outputs/paths_graph_enriched.csv")
LINKS    = Path("./eon_links_timeseries.csv")
assert ENRICHED.exists(), "Run Step 14 first (paths_graph_enriched.csv)."
assert LINKS.exists(), "Missing eon_links_timeseries.csv in working folder."

df    = pd.read_csv(ENRICHED)
links = pd.read_csv(LINKS)

train = df[df["split"]=="train_source"].copy()                   # NSFNET
test  = df[(df["split"]=="test_target") & (df["topology"]=="GEANT2")].copy()  # GEANT2

# ------------------ Feature schemas (HARD) ------------------
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
GF_COLS = [
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
    "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]
ENR_QOT  = BASE_QOT + GF_COLS
BASE_FAIL= [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'symbol_rate_gbaud','bitrate_gbps'
]
ENR_FAIL = BASE_FAIL + GF_COLS
CAT = ['modulation']

# Consistent modulation encoding
train_mod_cats = train['modulation'].astype('category').cat.categories.tolist()
def enc_mod(s): return pd.Categorical(s, categories=train_mod_cats).codes
def ordered(df_in, cols): return df_in[cols].copy()

# ------------------ Train base models (NSFNET) ------------------
# QoT model (+GraphFea)
COL_QOT = ENR_QOT + CAT
Xtr_q = train[COL_QOT].copy(); Xtr_q['modulation'] = enc_mod(Xtr_q['modulation'])
ytr_q = train['qot_ok'].astype(int).values
sc_q  = StandardScaler(); Xtr_qs = sc_q.fit_transform(ordered(Xtr_q, COL_QOT))
clf_q = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                      alpha=1e-4, learning_rate_init=1e-3,
                      max_iter=200, random_state=42).fit(Xtr_qs, ytr_q)

# Failure model (+GraphFea)
COL_F = ENR_FAIL + CAT
Xtr_f = train[COL_F].copy(); Xtr_f['modulation'] = enc_mod(Xtr_f['modulation'])
ytr_f = train['failure_present'].astype(int).values
sc_f  = StandardScaler(); Xtr_fs = sc_f.fit_transform(ordered(Xtr_f, COL_F))
clf_f = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                      alpha=1e-4, learning_rate_init=1e-3,
                      max_iter=200, random_state=42).fit(Xtr_fs, ytr_f)

# ------------------ QoT: Zero-shot vs TTA-lite probs on GEANT2 ------------------
Xte_q = test[COL_QOT].copy(); Xte_q['modulation'] = enc_mod(Xte_q['modulation'])
yte_q = test['qot_ok'].astype(int).values
Xte_qs = sc_q.transform(ordered(Xte_q, COL_QOT))
p_zero = clf_q.predict_proba(Xte_qs)[:,1]

def bn_adapt_batch(X_raw, sc, gamma=0.3):
    mu_src, sd_src = sc.mean_, sc.scale_
    mu_b  = X_raw.mean(axis=0); sd_b = X_raw.std(axis=0, ddof=0) + 1e-6
    mu = (1-gamma)*mu_src + gamma*mu_b
    sd = (1-gamma)*sd_src + gamma*sd_b
    return (X_raw - mu)/sd

def tta_probs(X_raw, sc, clf, aug_n=5, jitter=0.03, gamma=0.3):
    Xb = bn_adapt_batch(X_raw, sc, gamma=gamma)
    rng = np.random.RandomState(7)
    probs = []
    for _ in range(aug_n):
        Xj = Xb.copy()
        noise = rng.normal(0.0, jitter, size=Xj[:,:-1].shape) # don't jitter modulation col
        Xj[:,:-1] = Xj[:,:-1]*(1+noise)
        probs.append(clf.predict_proba(Xj)[:,1])
    return np.mean(probs, axis=0)

p_tta = tta_probs(ordered(Xte_q, COL_QOT).values, sc_q, clf_q, aug_n=5, jitter=0.03, gamma=0.3)

# ------------------ Failure detection: predictions on GEANT2 (+GraphFea) ------------------
Xte_f = test[COL_F].copy(); Xte_f['modulation'] = enc_mod(Xte_f['modulation'])
yte_f = test['failure_present'].astype(int).values
Xte_fs= sc_f.transform(ordered(Xte_f, COL_F))
yhat_f = clf_f.predict(Xte_fs)

# ------------------ Rerouting: re-evaluate with per-demand outcomes for bootstrap ------------------
import networkx as nx

def build_graph_for_day(day):
    sub = links[(links['topology']=='GEANT2') & (links['day']==day)]
    G = nx.Graph()
    for _, r in sub.iterrows():
        u, v = int(r['u']), int(r['v'])
        G.add_edge(u, v, length_km=float(r['length_km']))
    return G, sub.set_index('edge_id')

def edge_ids_from_nodes(nodes):
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(nodes[:-1], nodes[1:])]

def path_aggs(eids, idx):
    rows = idx.loc[eids]
    hops = len(eids)
    base = {
        "hops": hops,
        "distance_km": float(rows['length_km'].sum()),
        "latency_ms":  float(rows['latency_ms'].sum()),
        "avg_utilization": float(rows['bandwidth_utilization'].mean()),
        "min_osnr_db": float(rows['osnr_db'].min()),
        "min_snr_db":  float(rows['snr_db'].min()),
        "max_center_offset_ghz": float(rows['center_freq_offset_ghz'].max()),
        "min_filter_bw_scale":   float(rows['filter_bw_scale'].min()),
    }
    osnrs = rows['osnr_db'].values
    utils = rows['bandwidth_utilization'].values
    shifts= rows['center_freq_offset_ghz'].values
    scales= rows['filter_bw_scale'].values
    gf = {
        "gf_osnr_min": float(osnrs.min()),
        "gf_osnr_var": float(np.var(osnrs)) if hops>1 else 0.0,
        "gf_util_mean": float(utils.mean()),
        "gf_util_max": float(utils.max()),
        "gf_shift_max": float(shifts.max()),
        "gf_scale_min": float(scales.min()),
        "gf_frac_shifted": float((shifts>0).mean()),
        "gf_frac_tight": float((scales<1.0).mean()),
        "gf_bot_pos": float(np.argmin(osnrs)/max(1,hops-1))
    }
    return base, gf

def cand_features(eids, tx_row, idx, use_graph):
    base, gf = path_aggs(eids, idx)
    row = {
        **base,
        "symbol_rate_gbaud": float(tx_row['symbol_rate_gbaud']),
        "bitrate_gbps": float(tx_row['bitrate_gbps']),
        "modulation": tx_row['modulation']
    }
    if use_graph:
        row.update(gf)
    return row

def reroute_eval_collect(feat_cols, clf, scaler, use_graph, K=3, sample_n=600, seed=7):
    rng = np.random.RandomState(seed)
    sample = test.sample(n=min(sample_n, len(test)), random_state=seed).reset_index(drop=True)

    # who needs reroute?
    Xtest = sample[feat_cols + CAT].copy(); Xtest['modulation'] = enc_mod(Xtest['modulation'])
    Xtest_ord = ordered(Xtest, feat_cols + CAT)
    need = (clf.predict_proba(scaler.transform(Xtest_ord))[:,1] < 0.5).astype(int)

    considered_idx = np.where(need==1)[0].tolist()
    outcomes = []   # 1 if salvaged, 0 if not
    for i in considered_idx:
        row = sample.iloc[i]
        day = int(row['day'])
        try:
            G, idx = build_graph_for_day(day)
        except Exception:
            continue
        nodes = [int(x) for x in row['path'].split('->')]
        try:
            alt_iter = nx.shortest_simple_paths(G, nodes[0], nodes[-1], weight='length_km')
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
        tried = 0; salv=False
        for cand_nodes in alt_iter:
            if cand_nodes == nodes: continue
            tried += 1
            if tried > K: break
            eids = edge_ids_from_nodes(cand_nodes)
            try:
                row_feat = cand_features(eids, row, idx, use_graph=use_graph)
            except KeyError:
                continue
            Xc = pd.DataFrame([row_feat]); Xc['modulation'] = enc_mod(Xc['modulation'])
            Xc_ord = ordered(Xc, feat_cols + CAT)
            if int(clf.predict(scaler.transform(Xc_ord))[0]) == 1:
                salv=True; break
        outcomes.append(1 if salv else 0)
    return outcomes  # list of 0/1 for considered demands

# Train two QoT models for rerouting
COL_BASE = BASE_QOT + CAT
Xtr_b = train[COL_BASE].copy(); Xtr_b['modulation'] = enc_mod(Xtr_b['modulation'])
ytr_b = train['qot_ok'].astype(int).values
sc_b  = StandardScaler(); Xtr_bs = sc_b.fit_transform(ordered(Xtr_b, COL_BASE))
clf_b = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                      alpha=1e-4, learning_rate_init=1e-3,
                      max_iter=200, random_state=42).fit(Xtr_bs, ytr_b)

COL_ENR = ENR_QOT + CAT
Xtr_g = train[COL_ENR].copy(); Xtr_g['modulation'] = enc_mod(Xtr_g['modulation'])
ytr_g = ytr_b
sc_g  = StandardScaler(); Xtr_gs = sc_g.fit_transform(ordered(Xtr_g, COL_ENR))
clf_g = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                      alpha=1e-4, learning_rate_init=1e-3,
                      max_iter=200, random_state=42).fit(Xtr_gs, ytr_g)

# Collect per-demand salvage outcomes (BASE / +GraphFea)
out_base = reroute_eval_collect(BASE_QOT, clf_b, sc_b, use_graph=False, K=3, sample_n=600, seed=7)
out_graph= reroute_eval_collect(ENR_QOT,  clf_g, sc_g, use_graph=True,  K=3, sample_n=600, seed=7)

def salvage_rate(lst): 
    return (np.mean(lst)*100.0 if len(lst)>0 else np.nan), len(lst)

# ------------------ Bootstrap helpers ------------------
def ci_percentile(samples, alpha=0.05):
    lo = np.percentile(samples, 100*alpha/2)
    hi = np.percentile(samples, 100*(1-alpha/2))
    return float(lo), float(hi)

def bootstrap_metric_binary(y, yhat, metric_fn, B=1000, seed=0):
    rng = np.random.RandomState(seed)
    N = len(y); vals=[]
    for _ in range(B):
        idx = rng.randint(0, N, size=N)
        vals.append(metric_fn(y[idx], yhat[idx]))
    return np.array(vals)

def bootstrap_auc(y, p, B=1000, seed=0):
    rng = np.random.RandomState(seed)
    N = len(y); vals=[]
    for _ in range(B):
        idx = rng.randint(0, N, size=N)
        try:
            vals.append(roc_auc_score(y[idx], p[idx]))
        except ValueError:
            continue
    return np.array(vals)

def bootstrap_salvage(outcomes, B=2000, seed=0):
    rng = np.random.RandomState(seed)
    n = len(outcomes); outs = np.array(outcomes, dtype=float)
    vals=[]
    for _ in range(B):
        idx = rng.randint(0, n, size=n)
        vals.append(outs[idx].mean()*100.0)
    return np.array(vals)

# ------------------ Compute CIs ------------------
# QoT AUC
auc0 = roc_auc_score(yte_q, p_zero)
aucA = roc_auc_score(yte_q, p_tta)
b0   = bootstrap_auc(yte_q, p_zero, B=1000, seed=1)
bA   = bootstrap_auc(yte_q, p_tta,  B=1000, seed=2)
dA   = bA[:min(len(bA),len(b0))] - b0[:min(len(bA),len(b0))]
ci0  = ci_percentile(b0); ciA = ci_percentile(bA); cid = ci_percentile(dA)

# Failure F1 (+GraphFea)
f1  = f1_score(yte_f, yhat_f, average='macro')
bf1 = bootstrap_metric_binary(yte_f, yhat_f, lambda yt, yh: f1_score(yt, yh, average='macro'), B=1000, seed=3)
cif = ci_percentile(bf1)

# Rerouting salvage
rate_b, n_b = salvage_rate(out_base)
rate_g, n_g = salvage_rate(out_graph)
bb = bootstrap_salvage(out_base, B=2000, seed=4)
bg = bootstrap_salvage(out_graph, B=2000, seed=5)
bd = bg[:min(len(bg),len(bb))] - bb[:min(len(bg),len(bb))]
cib = ci_percentile(bb); cig = ci_percentile(bg); cid_s = ci_percentile(bd)
p_boot = float(np.mean(bd <= 0.0))  # one-sided: +GraphFea <= BASE

# ------------------ Print tidy tables ------------------
def tbl(rows, cols):
    dfp = pd.DataFrame(rows, columns=cols)
    display(dfp)

print("\n=== QoT (GEANT2, HARD) — AUC: Zero-shot vs TTA-lite ===")
tbl([
    ["Zero-shot", round(auc0,4), f"[{ci0[0]:.4f}, {ci0[1]:.4f}]"],
    ["TTA-lite",  round(aucA,4), f"[{ciA[0]:.4f}, {ciA[1]:.4f}]"],
    ["Δ (TTA - Zero)", round(aucA-auc0,4), f"[{cid[0]:.4f}, {cid[1]:.4f}]"],
], ["Model","AUC","95% CI"])

print("\n=== Failure Detection (GEANT2, HARD, +GraphFea) — Macro-F1 ===")
tbl([
    ["+GraphFea", round(f1,4), f"[{cif[0]:.4f}, {cif[1]:.4f}]", len(yte_f)]
], ["Model","F1 (macro)","95% CI","N"])

print("\n=== QoT-guided Re-routing (GEANT2, HARD) — Salvage rate (%) ===")
tbl([
    ["BASE",      round(rate_b,3), f"[{cib[0]:.3f}, {cib[1]:.3f}]", n_b],
    ["+GraphFea", round(rate_g,3), f"[{cig[0]:.3f}, {cig[1]:.3f}]", n_g],
    ["Δ (+Graph - BASE)", round(rate_g-rate_b,3), f"[{cid_s[0]:.3f}, {cid_s[1]:.3f}]", ""],
    ["Bootstrap one-sided p(Δ<=0)", p_boot, "", ""],
], ["Model","Salvage %","95% CI","Considered"])



=== QoT (GEANT2, HARD) — AUC: Zero-shot vs TTA-lite ===


,Model,AUC,95% CI
0,Zero-shot,0.9938,"[0.9924, 0.9948]"
1,TTA-lite,0.9940,"[0.9928, 0.9952]"
2,Δ (TTA - Zero),0.0003,"[-0.0015, 0.0022]"



=== Failure Detection (GEANT2, HARD, +GraphFea) — Macro-F1 ===


,Model,F1 (macro),95% CI,N
0,+GraphFea,0.9862,"[0.9820, 0.9903]",5400



=== QoT-guided Re-routing (GEANT2, HARD) — Salvage rate (%) ===


,Model,Salvage %,95% CI,Considered
0,BASE,22.892,"[16.867, 29.518]",166
1,+GraphFea,27.215,"[20.886, 34.177]",158
2,Δ (+Graph - BASE),4.324,"[-4.866, 14.207]",
3,Bootstrap one-sided p(Δ<=0),0.180,,


In [2]:
# Step 24 — Reproducibility: seed sweep (median ± IQR) for QoT AUC, Failure F1, Rerouting salvage
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import warnings; warnings.filterwarnings("ignore")

# ---------- Optional deps for rerouting ----------
try:
    import networkx as nx
except Exception:
    nx = None

# ---------- Paths & data ----------
ENRICHED = Path("./outputs/paths_graph_enriched.csv")
LINKS    = Path("./eon_links_timeseries.csv")
assert ENRICHED.exists(), "Run Step 14 first (paths_graph_enriched.csv must exist)."
df = pd.read_csv(ENRICHED)

links = None
if LINKS.exists():
    links = pd.read_csv(LINKS)

train = df[df["split"]=="train_source"].copy()                     # NSFNET
test  = df[(df["split"]=="test_target") & (df["topology"]=="GEANT2")].copy()  # GEANT2

# ---------- Features (HARD) ----------
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
GF_COLS = [
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
    "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]
ENR_QOT  = BASE_QOT + GF_COLS

BASE_FAIL= [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'symbol_rate_gbaud','bitrate_gbps'
]
ENR_FAIL = BASE_FAIL + GF_COLS

CAT = ['modulation']
MOD_CATS = train['modulation'].astype('category').cat.categories.tolist()
def enc_mod(s): return pd.Categorical(s, categories=MOD_CATS).codes
def ordered(df_in, cols): 
    missing = [c for c in cols if c not in df_in.columns]
    if missing: 
        raise KeyError(f"Missing columns: {missing}")
    return df_in[cols].copy()

# ---------- TTA-lite helpers ----------
def bn_adapt_batch(X_raw, sc, gamma=0.3):
    mu_src, sd_src = sc.mean_, sc.scale_
    mu_b  = X_raw.mean(axis=0); sd_b = X_raw.std(axis=0, ddof=0) + 1e-6
    mu = (1-gamma)*mu_src + gamma*mu_b
    sd = (1-gamma)*sd_src + gamma*sd_b
    return (X_raw - mu)/sd

def tta_probs(X_raw, sc, clf, aug_n=5, jitter=0.03, gamma=0.3, seed=7):
    Xb = bn_adapt_batch(X_raw, sc, gamma=gamma)
    rng = np.random.RandomState(seed)
    probs = []
    for _ in range(aug_n):
        Xj = Xb.copy()
        noise = rng.normal(0.0, jitter, size=Xj[:,:-1].shape)  # don't jitter modulation col
        Xj[:,:-1] = Xj[:,:-1] * (1 + noise)
        probs.append(clf.predict_proba(Xj)[:,1])
    return np.mean(probs, axis=0)

# ---------- Rerouting helpers ----------
def build_graph_for_day(day, links_df):
    sub = links_df[(links_df['topology']=='GEANT2') & (links_df['day']==day)]
    G = nx.Graph()
    for _, r in sub.iterrows():
        u, v = int(r['u']), int(r['v'])
        G.add_edge(u, v, length_km=float(r['length_km']))
    return G, sub.set_index('edge_id')

def edge_ids_from_nodes(nodes):
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(nodes[:-1], nodes[1:])]

def path_aggs(eids, idx):
    rows = idx.loc[eids]
    hops = len(eids)
    base = {
        "hops": hops,
        "distance_km": float(rows['length_km'].sum()),
        "latency_ms":  float(rows['latency_ms'].sum()),
        "avg_utilization": float(rows['bandwidth_utilization'].mean()),
        "min_osnr_db": float(rows['osnr_db'].min()),
        "min_snr_db":  float(rows['snr_db'].min()),
        "max_center_offset_ghz": float(rows['center_freq_offset_ghz'].max()),
        "min_filter_bw_scale":   float(rows['filter_bw_scale'].min()),
    }
    osnrs = rows['osnr_db'].values
    utils = rows['bandwidth_utilization'].values
    shifts= rows['center_freq_offset_ghz'].values
    scales= rows['filter_bw_scale'].values
    gf = {
        "gf_osnr_min": float(osnrs.min()),
        "gf_osnr_var": float(np.var(osnrs)) if hops>1 else 0.0,
        "gf_util_mean": float(utils.mean()),
        "gf_util_max": float(utils.max()),
        "gf_shift_max": float(shifts.max()),
        "gf_scale_min": float(scales.min()),
        "gf_frac_shifted": float((shifts>0).mean()),
        "gf_frac_tight": float((scales<1.0).mean()),
        "gf_bot_pos": float(np.argmin(osnrs)/max(1,hops-1))
    }
    return base, gf

def cand_features(eids, tx_row, idx, use_graph):
    base, gf = path_aggs(eids, idx)
    row = {
        **base,
        "symbol_rate_gbaud": float(tx_row['symbol_rate_gbaud']),
        "bitrate_gbps": float(tx_row['bitrate_gbps']),
        "modulation": tx_row['modulation']
    }
    if use_graph:
        row.update(gf)
    return row

def reroute_eval_seed(feat_cols, clf, scaler, use_graph, K=3, sample_n=600, seed=7):
    """Return salvage rate (%) for given model under a fixed seed."""
    rng = np.random.RandomState(seed)
    sample = test.sample(n=min(sample_n, len(test)), random_state=seed).reset_index(drop=True)

    # who needs reroute?
    Xtest = sample[feat_cols + CAT].copy(); Xtest['modulation'] = enc_mod(Xtest['modulation'])
    Xtest_ord = ordered(Xtest, feat_cols + CAT)
    need = (clf.predict_proba(scaler.transform(Xtest_ord))[:,1] < 0.5).astype(int)

    considered = salvaged = 0
    for i, row in sample.iterrows():
        if int(need[i]) == 0:
            continue
        day = int(row['day'])
        try:
            G, idx = build_graph_for_day(day, links)
        except Exception:
            continue
        nodes = [int(x) for x in row['path'].split('->')]
        try:
            alt_iter = nx.shortest_simple_paths(G, nodes[0], nodes[-1], weight='length_km')
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
        tried = 0; considered += 1
        for cand_nodes in alt_iter:
            if cand_nodes == nodes: 
                continue
            tried += 1
            if tried > K: 
                break
            eids = edge_ids_from_nodes(cand_nodes)
            try:
                row_feat = cand_features(eids, row, idx, use_graph=use_graph)
            except KeyError:
                continue
            Xc = pd.DataFrame([row_feat]); Xc['modulation'] = enc_mod(Xc['modulation'])
            Xc_ord = ordered(Xc, feat_cols + CAT)
            if int(clf.predict(scaler.transform(Xc_ord))[0]) == 1:
                salvaged += 1
                break
    return (salvaged/considered*100.0) if considered>0 else np.nan

# ---------- Seed sweep ----------
SEEDS = [1,2,3,4,5,6,7]
rows_qot, rows_fail, rows_rer = [], [], []

for s in SEEDS:
    # --- QoT model (+GraphFea) trained on NSFNET
    COL_QOT = ENR_QOT + CAT
    Xtr = train[COL_QOT].copy(); Xtr['modulation'] = enc_mod(Xtr['modulation'])
    ytr = train['qot_ok'].astype(int).values
    sc_q  = StandardScaler(); Xtr_s = sc_q.fit_transform(ordered(Xtr, COL_QOT))
    clf_q = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                          alpha=1e-4, learning_rate_init=1e-3,
                          max_iter=200, random_state=s).fit(Xtr_s, ytr)
    # GEANT2 probs: zero vs TTA
    Xte = test[COL_QOT].copy(); Xte['modulation'] = enc_mod(Xte['modulation'])
    yte = test['qot_ok'].astype(int).values
    Xte_s = sc_q.transform(ordered(Xte, COL_QOT))
    p_zero = clf_q.predict_proba(Xte_s)[:,1]
    p_tta  = tta_probs(ordered(Xte, COL_QOT).values, sc_q, clf_q, aug_n=5, jitter=0.03, gamma=0.3, seed=s)
    rows_qot.append(dict(seed=s, auc_zero=roc_auc_score(yte, p_zero), auc_tta=roc_auc_score(yte, p_tta),
                         delta_auc=(roc_auc_score(yte, p_tta)-roc_auc_score(yte, p_zero))))

    # --- Failure model (+GraphFea) trained on NSFNET
    COL_F = ENR_FAIL + CAT
    Xtrf = train[COL_F].copy(); Xtrf['modulation'] = enc_mod(Xtrf['modulation'])
    ytrf = train['failure_present'].astype(int).values
    sc_f = StandardScaler(); Xtrf_s = sc_f.fit_transform(ordered(Xtrf, COL_F))
    clf_f= MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                         alpha=1e-4, learning_rate_init=1e-3,
                         max_iter=200, random_state=s).fit(Xtrf_s, ytrf)
    Xtef = test[COL_F].copy(); Xtef['modulation'] = enc_mod(Xtef['modulation'])
    ytef = test['failure_present'].astype(int).values
    Xtef_s = sc_f.transform(ordered(Xtef, COL_F))
    yhat_f = clf_f.predict(Xtef_s)
    rows_fail.append(dict(seed=s, f1_macro=f1_score(ytef, yhat_f, average='macro')))

    # --- Rerouting salvage (BASE vs +GraphFea) — only if deps present
    if (nx is not None) and (links is not None):
        # BASE model for rerouting decision
        COL_B = BASE_QOT + CAT
        Xtrb = train[COL_B].copy(); Xtrb['modulation'] = enc_mod(Xtrb['modulation'])
        ytrb = train['qot_ok'].astype(int).values
        sc_b = StandardScaler(); Xtrb_s = sc_b.fit_transform(ordered(Xtrb, COL_B))
        clf_b= MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                             alpha=1e-4, learning_rate_init=1e-3,
                             max_iter=200, random_state=s).fit(Xtrb_s, ytrb)
        # +GraphFea model for rerouting decision
        sc_g = sc_q; clf_g = clf_q  # reuse QoT +GraphFea
        salv_b = reroute_eval_seed(BASE_QOT, clf_b, sc_b, use_graph=False, K=3, sample_n=600, seed=s)
        salv_g = reroute_eval_seed(ENR_QOT,  clf_g, sc_g, use_graph=True,  K=3, sample_n=600, seed=s)
        rows_rer.append(dict(seed=s, salv_base=salv_b, salv_graph=salv_g, delta_salv=salv_g - salv_b))
    else:
        rows_rer.append(dict(seed=s, salv_base=np.nan, salv_graph=np.nan, delta_salv=np.nan))

# ---------- Build DataFrames ----------
df_qot  = pd.DataFrame(rows_qot)
df_fail = pd.DataFrame(rows_fail)
df_rer  = pd.DataFrame(rows_rer)

OUTM = Path("./outputs/metrics"); OUTM.mkdir(parents=True, exist_ok=True)
df_qot.to_csv(OUTM/"seed_sweep_qot.csv", index=False)
df_fail.to_csv(OUTM/"seed_sweep_fail.csv", index=False)
df_rer.to_csv(OUTM/"seed_sweep_reroute.csv", index=False)

# ---------- Summaries: median ± IQR ----------
def med_iqr(s):
    s = pd.Series(s).dropna()
    if s.empty: return np.nan, np.nan
    med = s.median()
    iqr = s.quantile(0.75) - s.quantile(0.25)
    return med, iqr

summ_rows = []
m, i = med_iqr(df_qot['auc_zero']); summ_rows.append(["QoT AUC (Zero)", round(m,4), round(i,4)])
m, i = med_iqr(df_qot['auc_tta']);  summ_rows.append(["QoT AUC (TTA)",  round(m,4), round(i,4)])
m, i = med_iqr(df_qot['delta_auc']);summ_rows.append(["QoT ΔAUC (TTA−Zero)", round(m,4), round(i,4)])
m, i = med_iqr(df_fail['f1_macro']);summ_rows.append(["Failure F1 (macro, +GraphFea)", round(m,4), round(i,4)])
m, i = med_iqr(df_rer['salv_base']);summ_rows.append(["Reroute Salvage % (BASE)", round(m,3), round(i,3)])
m, i = med_iqr(df_rer['salv_graph']);summ_rows.append(["Reroute Salvage % (+GraphFea)", round(m,3), round(i,3)])
m, i = med_iqr(df_rer['delta_salv']);summ_rows.append(["Δ Salvage pp (+Graph−BASE)", round(m,3), round(i,3)])

summary = pd.DataFrame(summ_rows, columns=["Metric","Median","IQR"])
summary.to_csv(OUTM/"seed_sweep_summary.csv", index=False)

# ---------- Print clean tables ----------
print("\n=== Seed sweep — QoT AUC (Zero vs TTA) ===")
display(df_qot.round(4))

print("\n=== Seed sweep — Failure Detection F1 (+GraphFea) ===")
display(df_fail.round(4))

print("\n=== Seed sweep — Rerouting Salvage % (BASE vs +GraphFea) ===")
display(df_rer.round(3))

print("\n=== Seed sweep — Median ± IQR (paper-ready) ===")
display(summary)
print("\nSaved:")
print(" -", OUTM/"seed_sweep_qot.csv")
print(" -", OUTM/"seed_sweep_fail.csv")
print(" -", OUTM/"seed_sweep_reroute.csv")
print(" -", OUTM/"seed_sweep_summary.csv")
if (nx is None) or (links is None):
    print("\n[Note] Rerouting was skipped (missing networkx or links csv). Install `networkx` and ensure `eon_links_timeseries.csv` exists to enable it.")




=== Seed sweep — QoT AUC (Zero vs TTA) ===


,seed,auc_zero,auc_tta,delta_auc
0,1,0.9946,0.9951,0.0005
1,2,0.9954,0.9957,0.0003
2,3,0.9933,0.9939,0.0006
3,4,0.9939,0.9939,0.0001
4,5,0.9944,0.9939,-0.0004
5,6,0.9944,0.9946,0.0002
6,7,0.9940,0.9942,0.0001



=== Seed sweep — Failure Detection F1 (+GraphFea) ===


,seed,f1_macro
0,1,0.9861
1,2,0.9895
2,3,0.9880
3,4,0.9850
4,5,0.9865
5,6,0.9827
6,7,0.9873



=== Seed sweep — Rerouting Salvage % (BASE vs +GraphFea) ===


,seed,salv_base,salv_graph,delta_salv
0,1,22.013,26.036,4.023
1,2,25.175,26.000,0.825
2,3,19.463,20.859,1.396
3,4,25.000,24.161,-0.839
4,5,22.297,19.286,-3.012
5,6,19.333,20.915,1.582
6,7,19.231,17.834,-1.396



=== Seed sweep — Median ± IQR (paper-ready) ===


,Metric,Median,IQR
0,QoT AUC (Zero),0.9944,0.0005
1,QoT AUC (TTA),0.9942,0.0009
2,QoT ΔAUC (TTA−Zero),0.0002,0.0003
3,"Failure F1 (macro, +GraphFea)",0.9865,0.0021
4,Reroute Salvage % (BASE),22.0130,4.2500
5,Reroute Salvage % (+GraphFea),20.9150,5.0080
6,Δ Salvage pp (+Graph−BASE),0.8250,2.6060



Saved:
 - outputs\metrics\seed_sweep_qot.csv
 - outputs\metrics\seed_sweep_fail.csv
 - outputs\metrics\seed_sweep_reroute.csv
 - outputs\metrics\seed_sweep_summary.csv


In [1]:
# Step 25 (fixed) — Temperature scaling (calibration) + risk–coverage for QoT
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss, brier_score_loss, roc_auc_score, accuracy_score

# ---------- Load ----------
ENRICHED = Path("./outputs/paths_graph_enriched.csv")
FEWSHOT  = Path("./eon_target_fewshot.csv")
LINKS    = Path("./eon_links_timeseries.csv")
assert ENRICHED.exists(), "Run Step 14 first."
assert FEWSHOT.exists(),  "Missing eon_target_fewshot.csv"
assert LINKS.exists(),    "Missing eon_links_timeseries.csv"

df   = pd.read_csv(ENRICHED)   # already has gf_* columns
few  = pd.read_csv(FEWSHOT)
links= pd.read_csv(LINKS)

# ---------- Splits ----------
train = df[df["split"]=="train_source"].copy()                                     # NSFNET
test  = df[(df["split"]=="test_target") & (df["topology"]=="GEANT2")].copy()       # GEANT2

# ---------- Graph features for few-shot (if missing) ----------
GF_COLS = ["gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
           "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"]

lk_idx = links.set_index(["topology","day","edge_id"])
def edge_ids_from_path(path_str):
    ns = [int(x) for x in str(path_str).split("->")]
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(ns[:-1], ns[1:])]

def ensure_gf_columns(df_in):
    if all(c in df_in.columns for c in GF_COLS):
        return df_in.copy()
    rows, miss = [], 0
    for _, r in df_in.iterrows():
        topo, day, path = r["topology"], r["day"], r["path"]
        eids = edge_ids_from_path(path)
        try:
            rows_link = lk_idx.loc[(topo, day, eids)]
        except KeyError:
            miss += 1; continue
        hops   = len(eids)
        osnrs  = rows_link["osnr_db"].values
        utils  = rows_link["bandwidth_utilization"].values
        shifts = rows_link["center_freq_offset_ghz"].values
        scales = rows_link["filter_bw_scale"].values
        gf = dict(
            gf_osnr_min=float(osnrs.min()),
            gf_osnr_var=float(np.var(osnrs)) if hops>1 else 0.0,
            gf_util_mean=float(utils.mean()),
            gf_util_max=float(utils.max()),
            gf_shift_max=float(shifts.max()),
            gf_scale_min=float(scales.min()),
            gf_frac_shifted=float((shifts>0).mean()),
            gf_frac_tight=float((scales<1.0).mean()),
            gf_bot_pos=float(np.argmin(osnrs)/max(1,hops-1)),
        )
        row = r.to_dict(); row.update(gf); rows.append(row)
    out = pd.DataFrame(rows)
    if miss: print(f"[info] few-shot gf_: dropped {miss} rows where link join failed.")
    return out

few = ensure_gf_columns(few)

# ---------- Held-out = GEANT2 \ few-shot keys ----------
KEY = ['topology','day','src','dst','path','hops','distance_km']
test['_key'] = test[KEY].astype(str).agg('|'.join, axis=1)
few['_key']  = few[KEY].astype(str).agg('|'.join, axis=1)
held = test[~test['_key'].isin(set(few['_key']))].copy()

# ---------- Feature schema (QoT, HARD + graph-aware) ----------
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
ENR_QOT = BASE_QOT + GF_COLS
CAT     = ['modulation']
COLS    = ENR_QOT + CAT

# ---------- Helpers ----------
train_mod_cats = train['modulation'].astype('category').cat.categories.tolist()
def enc_mod(s): return pd.Categorical(s, categories=train_mod_cats).codes
def ordered(df_in, cols): 
    missing = [c for c in cols if c not in df_in.columns]
    if missing: raise KeyError(f"Missing: {missing}")
    return df_in[cols].copy()
def sigmoid(z): return 1.0/(1.0 + np.exp(-z))
def logit(p):
    p = np.clip(p, 1e-6, 1-1e-6)
    return np.log(p/(1-p))

# ---------- Train base QoT model on NSFNET ----------
Xtr = train[COLS].copy(); Xtr['modulation'] = enc_mod(Xtr['modulation'])
ytr = train['qot_ok'].astype(int).values
sc  = StandardScaler(); Xtr_s = sc.fit_transform(ordered(Xtr, COLS))
clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                    alpha=1e-4, learning_rate_init=1e-3,
                    max_iter=200, random_state=42).fit(Xtr_s, ytr)

# ---------- Build calibration & test matrices ----------
# calibration from few-shot GEANT2
Xcal = few[COLS].copy(); Xcal['modulation'] = enc_mod(Xcal['modulation'])
ycal = few['qot_ok'].astype(int).values
Xcal_s = sc.transform(ordered(Xcal, COLS))
p_cal  = clf.predict_proba(Xcal_s)[:,1]
z_cal  = logit(p_cal)  # logits from probabilities

# held-out test
Xte = held[COLS].copy(); Xte['modulation'] = enc_mod(Xte['modulation'])
yte = held['qot_ok'].astype(int).values
Xte_s = sc.transform(ordered(Xte, COLS))
p_pre = clf.predict_proba(Xte_s)[:,1]
z_te  = logit(p_pre)

# ---------- Fit temperature T on calibration (minimize NLL) ----------
def nll_for_T(T):
    T = max(T, 1e-3)
    p = sigmoid(z_cal / T)
    p = np.clip(p, 1e-6, 1-1e-6)
    return log_loss(ycal, p)

grid = np.linspace(0.3, 3.0, 136)
vals = np.array([nll_for_T(t) for t in grid])
T0   = float(grid[np.argmin(vals)])
local = np.linspace(max(0.1, T0-0.5), T0+0.5, 101)
vals2 = np.array([nll_for_T(t) for t in local])
T_hat = float(local[np.argmin(vals2)])
print(f"Fitted temperature T = {T_hat:.3f} (from {len(few)} few-shot samples)")

# ---------- Evaluate: pre/post calibration ----------
p0 = p_pre                      # pre-calibration probabilities
p1 = sigmoid(z_te / T_hat)      # temp-scaled probabilities

def ece(p, y, n_bins=15):
    bins = np.linspace(0,1,n_bins+1)
    e = 0.0
    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        idx = (p>=lo) & (p<hi) if i<n_bins-1 else (p>=lo) & (p<=hi)
        if np.any(idx):
            conf = p[idx].mean()
            acc  = (y[idx]==(p[idx]>=0.5)).mean()
            e   += (idx.mean()) * abs(acc - conf)
    return float(e)

def metrics(p, y):
    return dict(
        AUC = roc_auc_score(y, p),
        ACC = accuracy_score(y, (p>=0.5).astype(int)),
        NLL = log_loss(y, np.clip(p,1e-6,1-1e-6)),
        Brier = brier_score_loss(y, p),
        ECE = ece(p, y, n_bins=15)
    )

m0 = metrics(p0, yte)
m1 = metrics(p1, yte)

# ---------- Risk–coverage curves (selective accuracy) ----------
def risk_coverage(p, y, steps=25):
    conf = np.maximum(p, 1-p)
    order = np.argsort(conf)[::-1]
    covs, accs = [], []
    for k in np.linspace(0.05, 1.0, steps):
        n = max(1, int(k*len(y)))
        idx = order[:n]
        acc = accuracy_score(y[idx], (p[idx]>=0.5).astype(int))
        covs.append(k); accs.append(acc)
    return np.array(covs), np.array(accs)

cov0, acc0 = risk_coverage(p0, yte)
cov1, acc1 = risk_coverage(p1, yte)

# ---------- Save + Print ----------
OUTM = Path("./outputs/metrics"); OUTF = Path("./outputs/figs")
OUTM.mkdir(parents=True, exist_ok=True); OUTF.mkdir(parents=True, exist_ok=True)
pd.DataFrame([{"model":"pre", **m0}, {"model":"temp_scaled", **m1}]).to_csv(OUTM/"qot_calibration_metrics.csv", index=False)

# Reliability diagram
def reliability_points(p, y, n_bins=15):
    bins = np.linspace(0,1,n_bins+1)
    xs, ys = [], []
    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        idx = (p>=lo) & (p<hi) if i<n_bins-1 else (p>=lo) & (p<=hi)
        if np.any(idx):
            xs.append(p[idx].mean())
            ys.append((y[idx]==(p[idx]>=0.5)).mean())
    return np.array(xs), np.array(ys)

x0,y0 = reliability_points(p0, yte)
x1,y1 = reliability_points(p1, yte)

plt.figure(figsize=(6,5))
plt.plot([0,1],[0,1],'--',linewidth=1)
plt.scatter(x0,y0,label=f'Pre-cal (ECE={m0["ECE"]:.3f})', s=30)
plt.scatter(x1,y1,label=f'Temp-scaled (ECE={m1["ECE"]:.3f})', s=30, marker='s')
plt.xlabel("Mean predicted confidence"); plt.ylabel("Empirical accuracy")
plt.title("Reliability (QoT on GEANT2, held-out)")
plt.legend(); plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig(OUTF/"qot_reliability_temp_scaling.png", bbox_inches="tight"); plt.close()

# Risk–coverage
plt.figure(figsize=(6,5))
plt.plot(cov0, acc0, marker='o', label='Pre-cal')
plt.plot(cov1, acc1, marker='s', label='Temp-scaled')
plt.xlabel("Coverage (kept fraction)"); plt.ylabel("Selective accuracy")
plt.title("Risk–Coverage (QoT on GEANT2, held-out)")
plt.legend(); plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig(OUTF/"qot_risk_coverage_temp_scaling.png", bbox_inches="tight"); plt.close()

# Inline paper-ready table
def tidy(m): 
    return {k: (round(v,4) if isinstance(v,(int,float)) else v) for k,v in m.items()}
print("\n=== QoT Calibration (held-out GEANT2) ===")
display(pd.DataFrame([
    {"Model":"Pre-calibration", **tidy(m0)},
    {"Model":"Temp-scaled",     **tidy(m1)},
])[["Model","AUC","ACC","NLL","Brier","ECE"]])

print("Saved:")
print(" -", OUTM/"qot_calibration_metrics.csv")
print(" -", OUTF/"qot_reliability_temp_scaling.png")
print(" -", OUTF/"qot_risk_coverage_temp_scaling.png")



Fitted temperature T = 3.010 (from 600 few-shot samples)

=== QoT Calibration (held-out GEANT2) ===


,Model,AUC,ACC,NLL,Brier,ECE
0,Pre-calibration,0.9944,0.964,0.1625,0.0305,0.2214
1,Temp-scaled,0.9940,0.964,0.0901,0.0254,0.2045


Saved:
 - outputs\metrics\qot_calibration_metrics.csv
 - outputs\figs\qot_reliability_temp_scaling.png
 - outputs\figs\qot_risk_coverage_temp_scaling.png


In [2]:
# Step 26 — Calibrate TTA-lite probabilities (few-shot temperature scaling) and evaluate on held-out GEANT2
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss, brier_score_loss, roc_auc_score, accuracy_score

# ---------- Load ----------
ENRICHED = Path("./outputs/paths_graph_enriched.csv")
FEWSHOT  = Path("./eon_target_fewshot.csv")
LINKS    = Path("./eon_links_timeseries.csv")
assert ENRICHED.exists(), "Run Step 14 first."
assert FEWSHOT.exists(),  "Missing eon_target_fewshot.csv"
assert LINKS.exists(),    "Missing eon_links_timeseries.csv"

df   = pd.read_csv(ENRICHED)   # already has gf_* columns
few  = pd.read_csv(FEWSHOT)
links= pd.read_csv(LINKS)

# ---------- Splits ----------
train = df[df["split"]=="train_source"].copy()                                     # NSFNET
test  = df[(df["split"]=="test_target") & (df["topology"]=="GEANT2")].copy()       # GEANT2

# ---------- Ensure gf_* present for few-shot (if user exported a minimal few-shot file) ----------
GF_COLS = ["gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
           "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"]

lk_idx = links.set_index(["topology","day","edge_id"])
def edge_ids_from_path(path_str):
    ns = [int(x) for x in str(path_str).split("->")]
    return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(ns[:-1], ns[1:])]

def ensure_gf_columns(df_in):
    if all(c in df_in.columns for c in GF_COLS):
        return df_in.copy()
    rows, miss = [], 0
    for _, r in df_in.iterrows():
        topo, day, path = r["topology"], r["day"], r["path"]
        eids = edge_ids_from_path(path)
        try:
            rows_link = lk_idx.loc[(topo, day, eids)]
        except KeyError:
            miss += 1; continue
        hops   = len(eids)
        osnrs  = rows_link["osnr_db"].values
        utils  = rows_link["bandwidth_utilization"].values
        shifts = rows_link["center_freq_offset_ghz"].values
        scales = rows_link["filter_bw_scale"].values
        gf = dict(
            gf_osnr_min=float(osnrs.min()),
            gf_osnr_var=float(np.var(osnrs)) if hops>1 else 0.0,
            gf_util_mean=float(utils.mean()),
            gf_util_max=float(utils.max()),
            gf_shift_max=float(shifts.max()),
            gf_scale_min=float(scales.min()),
            gf_frac_shifted=float((shifts>0).mean()),
            gf_frac_tight=float((scales<1.0).mean()),
            gf_bot_pos=float(np.argmin(osnrs)/max(1,hops-1)),
        )
        row = r.to_dict(); row.update(gf); rows.append(row)
    out = pd.DataFrame(rows)
    if miss: print(f"[info] few-shot gf_: dropped {miss} rows where link join failed.")
    return out

few = ensure_gf_columns(few)

# ---------- Held-out = GEANT2 \ few-shot keys ----------
KEY = ['topology','day','src','dst','path','hops','distance_km']
test['_key'] = test[KEY].astype(str).agg('|'.join, axis=1)
few['_key']  = few[KEY].astype(str).agg('|'.join, axis=1)
held = test[~test['_key'].isin(set(few['_key']))].copy()

# ---------- Feature schema (QoT, HARD + graph-aware) ----------
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
ENR_QOT = BASE_QOT + GF_COLS
CAT     = ['modulation']
COLS    = ENR_QOT + CAT

# ---------- Helpers ----------
train_mod_cats = train['modulation'].astype('category').cat.categories.tolist()
def enc_mod(s): return pd.Categorical(s, categories=train_mod_cats).codes
def ordered(df_in, cols): 
    miss = [c for c in cols if c not in df_in.columns]
    if miss: raise KeyError(f"Missing: {miss}")
    return df_in[cols].copy()
def sigmoid(z): return 1.0/(1.0 + np.exp(-z))
def logit(p):
    p = np.clip(p, 1e-6, 1-1e-6)
    return np.log(p/(1-p))

# ---------- Train QoT model on NSFNET ----------
Xtr = train[COLS].copy(); Xtr['modulation'] = enc_mod(Xtr['modulation'])
ytr = train['qot_ok'].astype(int).values
sc  = StandardScaler(); Xtr_s = sc.fit_transform(ordered(Xtr, COLS))
clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                    alpha=1e-4, learning_rate_init=1e-3,
                    max_iter=200, random_state=42).fit(Xtr_s, ytr)

# ---------- TTA-lite helper (batch-norm adaptation per day + MC jitter) ----------
def bn_adapt_batch(X_raw, sc, gamma=0.3):
    mu_src, sd_src = sc.mean_, sc.scale_
    mu_b  = X_raw.mean(axis=0); sd_b = X_raw.std(axis=0, ddof=0) + 1e-6
    mu = (1-gamma)*mu_src + gamma*mu_b
    sd = (1-gamma)*sd_src + gamma*sd_b
    return (X_raw - mu)/sd

def tta_probs_for_rows(rows_df, day_batch_df, sc, clf, cols, aug_n=5, jitter=0.03, gamma=0.3, rng_seed=7):
    # Batch stats from the whole GEANT2 day
    Xb = day_batch_df[cols].copy()
    Xb['modulation'] = enc_mod(Xb['modulation'])
    Xb = ordered(Xb, cols).values
    mu_src, sd_src = sc.mean_, sc.scale_
    mu_b,  sd_b  = Xb.mean(axis=0), Xb.std(axis=0, ddof=0)+1e-6
    mu = (1-gamma)*mu_src + gamma*mu_b
    sd = (1-gamma)*sd_src + gamma*sd_b

    Xr = rows_df[cols].copy()
    Xr['modulation'] = enc_mod(Xr['modulation'])
    Xr = ordered(Xr, cols).values
    Xr_s = (Xr - mu)/sd

    rng = np.random.RandomState(rng_seed)
    probs = []
    for _ in range(aug_n):
        Xj = Xr_s.copy()
        noise = rng.normal(0.0, jitter, size=Xj[:,:-1].shape)  # do not jitter modulation
        Xj[:,:-1] = Xj[:,:-1]*(1+noise)
        probs.append(clf.predict_proba(Xj)[:,1])
    return np.mean(probs, axis=0)

# ---------- Build TTA-lite probs for calibration (few-shot) ----------
p_cal_list, y_cal_list = [], []
for d, grp in few.groupby('day'):
    day_all = test[test['day']==d]      # unlabeled batch for that day (target domain)
    if len(day_all)==0: 
        continue
    p_cal_list.append(tta_probs_for_rows(grp, day_all, sc, clf, COLS))
    y_cal_list.append(grp['qot_ok'].astype(int).values)
p_cal = np.concatenate(p_cal_list) if p_cal_list else np.array([])
y_cal = np.concatenate(y_cal_list) if y_cal_list else np.array([])
assert len(p_cal)>0, "No few-shot calibration probs were produced."
z_cal = logit(p_cal)

# ---------- Build TTA-lite probs for held-out GEANT2 ----------
p_te_list, y_te_list = [], []
for d, grp in held.groupby('day'):
    day_all = test[test['day']==d]
    if len(day_all)==0: 
        continue
    p_te_list.append(tta_probs_for_rows(grp, day_all, sc, clf, COLS))
    y_te_list.append(grp['qot_ok'].astype(int).values)
p_pre = np.concatenate(p_te_list) if p_te_list else np.array([])
y_te  = np.concatenate(y_te_list) if y_te_list else np.array([])
assert len(p_pre)>0, "No held-out TTA probabilities were produced."
z_te = logit(p_pre)

# ---------- Fit temperature on few-shot (minimize NLL) ----------
def nll_for_T(T):
    T = max(T, 1e-3)
    p = sigmoid(z_cal / T)
    p = np.clip(p, 1e-6, 1-1e-6)
    return log_loss(y_cal, p)

grid = np.linspace(0.3, 3.0, 136)
vals = np.array([nll_for_T(t) for t in grid])
T0   = float(grid[np.argmin(vals)])
local = np.linspace(max(0.1, T0-0.5), T0+0.5, 101)
vals2 = np.array([nll_for_T(t) for t in local])
T_hat = float(local[np.argmin(vals2)])
print(f"Fitted T for TTA-lite = {T_hat:.3f} (from {len(y_cal)} few-shot logits)")

# ---------- Evaluate: pre/post calibration on held-out ----------
p0 = p_pre
p1 = sigmoid(z_te / T_hat)

def ece(p, y, n_bins=15):
    bins = np.linspace(0,1,n_bins+1)
    e = 0.0
    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        idx = (p>=lo) & (p<hi) if i<n_bins-1 else (p>=lo) & (p<=hi)
        if np.any(idx):
            conf = p[idx].mean()
            acc  = (y[idx]==(p[idx]>=0.5)).mean()
            e   += (idx.mean()) * abs(acc - conf)
    return float(e)

def metrics(p, y):
    return dict(
        AUC = roc_auc_score(y, p),
        ACC = accuracy_score(y, (p>=0.5).astype(int)),
        NLL = log_loss(y, np.clip(p,1e-6,1-1e-6)),
        Brier = brier_score_loss(y, p),
        ECE = ece(p, y, n_bins=15)
    )

m0 = metrics(p0, y_te)
m1 = metrics(p1, y_te)

# ---------- Risk–coverage ----------
def risk_coverage(p, y, steps=25):
    conf = np.maximum(p, 1-p)
    order = np.argsort(conf)[::-1]
    covs, accs = [], []
    for k in np.linspace(0.05, 1.0, steps):
        n = max(1, int(k*len(y)))
        idx = order[:n]
        acc = accuracy_score(y[idx], (p[idx]>=0.5).astype(int))
        covs.append(k); accs.append(acc)
    return np.array(covs), np.array(accs)

cov0, acc0 = risk_coverage(p0, y_te)
cov1, acc1 = risk_coverage(p1, y_te)

# ---------- Save + Print ----------
OUTM = Path("./outputs/metrics"); OUTF = Path("./outputs/figs")
OUTM.mkdir(parents=True, exist_ok=True); OUTF.mkdir(parents=True, exist_ok=True)
pd.DataFrame([{"model":"tta_pre", **m0}, {"model":"tta_temp_scaled", **m1}]).to_csv(OUTM/"qot_calibration_TTA_metrics.csv", index=False)

# Reliability
def reliability_points(p, y, n_bins=15):
    bins = np.linspace(0,1,n_bins+1)
    xs, ys = [], []
    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        idx = (p>=lo) & (p<hi) if i<n_bins-1 else (p>=lo) & (p<=hi)
        if np.any(idx):
            xs.append(p[idx].mean())
            ys.append((y[idx]==(p[idx]>=0.5)).mean())
    return np.array(xs), np.array(ys)

x0,y0 = reliability_points(p0, y_te)
x1,y1 = reliability_points(p1, y_te)

plt.figure(figsize=(6,5))
plt.plot([0,1],[0,1],'--',linewidth=1)
plt.scatter(x0,y0,label=f'TTA pre (ECE={m0["ECE"]:.3f})', s=30)
plt.scatter(x1,y1,label=f'TTA temp-scaled (ECE={m1["ECE"]:.3f})', s=30, marker='s')
plt.xlabel("Mean predicted confidence"); plt.ylabel("Empirical accuracy")
plt.title("Reliability — TTA-lite (held-out GEANT2)")
plt.legend(); plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig(OUTF/"qot_reliability_tta_temp_scaling.png", bbox_inches="tight"); plt.close()

# Risk–coverage
plt.figure(figsize=(6,5))
plt.plot(cov0, acc0, marker='o', label='TTA pre')
plt.plot(cov1, acc1, marker='s', label='TTA temp-scaled')
plt.xlabel("Coverage (kept fraction)"); plt.ylabel("Selective accuracy")
plt.title("Risk–Coverage — TTA-lite (held-out GEANT2)")
plt.legend(); plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig(OUTF/"qot_risk_coverage_tta_temp_scaling.png", bbox_inches="tight"); plt.close()

# Inline paper-ready table
def tidy(m): 
    return {k: (round(v,4) if isinstance(v,(int,float)) else v) for k,v in m.items()}
print("\n=== QoT Calibration for TTA-lite (held-out GEANT2) ===")
display(pd.DataFrame([
    {"Model":"TTA pre-cal",      **tidy(m0)},
    {"Model":"TTA temp-scaled",  **tidy(m1)},
])[["Model","AUC","ACC","NLL","Brier","ECE"]])

print("Saved:")
print(" -", OUTM/"qot_calibration_TTA_metrics.csv")
print(" -", OUTF/"qot_reliability_tta_temp_scaling.png")
print(" -", OUTF/"qot_risk_coverage_tta_temp_scaling.png")


Fitted T for TTA-lite = 2.980 (from 600 few-shot logits)

=== QoT Calibration for TTA-lite (held-out GEANT2) ===


,Model,AUC,ACC,NLL,Brier,ECE
0,TTA pre-cal,0.9945,0.959,0.1987,0.0362,0.2234
1,TTA temp-scaled,0.9941,0.959,0.0996,0.0289,0.1996


Saved:
 - outputs\metrics\qot_calibration_TTA_metrics.csv
 - outputs\figs\qot_reliability_tta_temp_scaling.png
 - outputs\figs\qot_risk_coverage_tta_temp_scaling.png


In [3]:
# Step 27 — Build a camera_ready/ pack + auto captions (paper-ready)
import os, json, shutil
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path(".")
OUTM = ROOT/"outputs"/"metrics"
OUTF = ROOT/"outputs"/"figs"
PACK = ROOT/"camera_ready"
PMET  = PACK/"metrics"
PFIG  = PACK/"figs"

# Create dirs
PMET.mkdir(parents=True, exist_ok=True)
PFIG.mkdir(parents=True, exist_ok=True)

# ---------- Collect metrics (copy if exists) ----------
metric_names = [
    "reroute_base.json",
    "reroute_graph.json",
    "reroute_summary.csv",
    "kshot_qot_enriched.csv",
    "kshot_fail_enriched.csv",
    "ablations_summary.csv",
    "qot_calibration_metrics.csv",
    "qot_calibration_TTA_metrics.csv",
    "seed_sweep_summary.csv",
    "seed_sweep_qot.csv",
    "seed_sweep_fail.csv",
    "seed_sweep_reroute.csv",
]
copied_metrics, missing_metrics = [], []
for name in metric_names:
    src = OUTM/name
    if src.exists():
        shutil.copy2(src, PMET/name)
        copied_metrics.append(name)
    else:
        missing_metrics.append(name)

# ---------- Collect figures (copy if exists) ----------
fig_names = [
    # zero-shot vs TTA (if you created them earlier)
    "qot_roc_zero_vs_tta.png",
    "qot_reliability_zero_vs_tta.png",
    "qot_risk_coverage_zero_vs_tta.png",
    # ablations
    "abl_qot.png",
    "abl_fail.png",
    "abl_fail_shift_tight.png",
    # k-shot (if made)
    "kshot_qot_enriched.png",
    "kshot_fail_enriched.png",
    # case study
    *[p.name for p in OUTF.glob("case_study_days_*.png")],
    # calibration (step 25)
    "qot_reliability_temp_scaling.png",
    "qot_risk_coverage_temp_scaling.png",
    # calibration for TTA (step 26)
    "qot_reliability_tta_temp_scaling.png",
    "qot_risk_coverage_tta_temp_scaling.png",
]
# de-dup while preserving order
seen = set(); fig_names = [f for f in fig_names if (f not in seen and not seen.add(f))]

copied_figs, missing_figs = [], []
for name in fig_names:
    src = OUTF/name
    if src.exists():
        shutil.copy2(src, PFIG/name)
        copied_figs.append(name)
    else:
        missing_figs.append(name)

# ---------- Read key numbers for captions ----------
def try_json(path):
    try:
        return json.loads(path.read_text())
    except Exception:
        return None

def try_csv(path):
    try:
        return pd.read_csv(path)
    except Exception:
        return None

rer_b = try_json(PMET/"reroute_base.json")
rer_g = try_json(PMET/"reroute_graph.json")
kq    = try_csv(PMET/"kshot_qot_enriched.csv")
kf    = try_csv(PMET/"kshot_fail_enriched.csv")
abl   = try_csv(PMET/"ablations_summary.csv")
cal_q = try_csv(PMET/"qot_calibration_metrics.csv")
cal_t = try_csv(PMET/"qot_calibration_TTA_metrics.csv")
seedS = try_csv(PMET/"seed_sweep_summary.csv")

def fmt(x, nd=3, pct=False):
    if x is None or (isinstance(x,float) and (np.isnan(x) or np.isinf(x))):
        return "—"
    if pct: return f"{x:.{nd}f}%"
    return f"{x:.{nd}f}"

lines = []
lines.append("# Camera-ready Captions (auto-generated)\n")

# Rerouting
if rer_b and rer_g:
    dpp = (rer_g.get("salvage_rate_pct",0) - rer_b.get("salvage_rate_pct",0))
    lines += [
        "## Fig. R1 — QoT-guided re-routing (GEANT2, zero-shot, HARD)",
        f"Salvage rate: **BASE {fmt(rer_b.get('salvage_rate_pct'),2,pct=True)}** → **+GraphFea {fmt(rer_g.get('salvage_rate_pct'),2,pct=True)}** "
        f"(Δ **{fmt(dpp,2)} pp**). Overhead (avg): BASE {fmt(rer_b.get('avg_extra_km'))} km / {fmt(rer_b.get('avg_extra_ms'))} ms; "
        f"+GraphFea {fmt(rer_g.get('avg_extra_km'))} km / {fmt(rer_g.get('avg_extra_ms'))} ms.\n"
    ]

# k-shot curves
if kq is not None:
    kq = kq.sort_values("k")
    last = kq.iloc[-1].to_dict() if not kq.empty else None
    if last:
        lines += [
            "## Fig. K1 — Few-shot (QoT, graph-enriched, HARD)",
            f"F1↑ with k: at k={int(last['k'])}, **F1={fmt(last['f1'],4)}**, **AUC={fmt(last['auc'],4)}**.\n"
        ]
if kf is not None:
    kf = kf.sort_values("k")
    last = kf.iloc[-1].to_dict() if not kf.empty else None
    if last:
        lines += [
            "## Fig. K2 — Few-shot (Failure detection, graph-enriched, HARD)",
            f"F1↑ with k: at k={int(last['k'])}, **F1={fmt(last['f1'],4)}**.\n"
        ]

# Ablations
if abl is not None:
    def pick(task, scen, k=0):
        row = abl[(abl.task==task) & (abl.scenario==scen) & (abl.k==k)]
        return row.iloc[0].to_dict() if not row.empty else None
    qb = pick("qot","base",0); qg0 = pick("qot","+graph",0); qg100 = pick("qot","+graph",100)
    fg0 = pick("fail","+graph",0); fb0 = pick("fail","base",0)
    if qb and qg0:
        lines += [
            "## Fig. A1 — QoT ablations (GEANT2, HARD)",
            f"Base F1 **{fmt(qb['f1'],4)}** vs +Graph F1 **{fmt(qg0['f1'],4)}**; +Graph (k=100) **{fmt(qg100['f1'],4)}**.\n"
        ]
    if fb0 and fg0:
        lines += [
            "## Fig. A2 — Failure ablations (GEANT2, HARD)",
            f"Base F1 **{fmt(fb0['f1'],4)}** vs +Graph F1 **{fmt(fg0['f1'],4)}**. Removing fingerprints drops to hard mode; removing OSNR keeps F1 high.\n"
        ]

# Calibration (pre vs temp-scaled)
if cal_q is not None:
    try:
        pre  = cal_q[cal_q["model"]=="pre"].iloc[0].to_dict()
        post = cal_q[cal_q["model"]=="temp_scaled"].iloc[0].to_dict()
        lines += [
            "## Fig. C1 — QoT calibration (few-shot temp scaling)",
            f"NLL {fmt(pre['NLL'],4)} → **{fmt(post['NLL'],4)}**, Brier {fmt(pre['Brier'],4)} → **{fmt(post['Brier'],4)}**, "
            f"ECE {fmt(pre['ECE'],4)} → **{fmt(post['ECE'],4)}**; AUC/ACC preserved.\n"
        ]
    except Exception:
        pass

if cal_t is not None:
    try:
        pre  = cal_t[cal_t["model"]=="tta_pre"].iloc[0].to_dict()
        post = cal_t[cal_t["model"]=="tta_temp_scaled"].iloc[0].to_dict()
        lines += [
            "## Fig. C2 — TTA-lite calibration (few-shot temp scaling)",
            f"NLL {fmt(pre['NLL'],4)} → **{fmt(post['NLL'],4)}**, Brier {fmt(pre['Brier'],4)} → **{fmt(post['Brier'],4)}**, "
            f"ECE {fmt(pre['ECE'],4)} → **{fmt(post['ECE'],4)}**; AUC/ACC preserved.\n"
        ]
    except Exception:
        pass

# Seed sweep summary (stability)
if seedS is not None:
    lines += ["## Table S — Stability (median ± IQR across seeds)\n"]
    for _, r in seedS.iterrows():
        lines.append(f"- {r['Metric']}: **{fmt(r['Median'],4)} ± {fmt(r['IQR'],4)}**")
    lines.append("")

# Write captions + checklist
(PACK/"captions.md").write_text("\n".join(lines), encoding="utf-8")

check = []
check += ["# Camera-ready checklist",
          "- [x] Results CSV/JSON in `camera_ready/metrics/`",
          "- [x] Figures in `camera_ready/figs/`",
          "- [x] `captions.md` with numbers auto-filled",
          "- [ ] Double-check figure dpi (≥300) if journal requires",
          "- [ ] Ensure train/test split description references NSFNET→GEANT2 (90-day dynamics)"]
(PACK/"checklist.md").write_text("\n".join(check), encoding="utf-8")

# Print summary
def tree(path: Path, prefix=""):
    for p in sorted(path.iterdir()):
        print(prefix + ("📁 " if p.is_dir() else "📄 ") + p.name)
        if p.is_dir():
            tree(p, prefix + "   ")

print("Created camera_ready pack at:", PACK.resolve())
print("\nContents:")
tree(PACK)

print("\nCopied metrics:", copied_metrics)
if missing_metrics:
    print("Missing (skipped):", len(missing_metrics))
    for m in missing_metrics: 
        pass  # keep output concise

print("Copied figures:", copied_figs)
if missing_figs:
    print("Some figures were not found (skipped). That's ok if you didn't generate them all.")
print("\nOpen `camera_ready/captions.md` for ready-to-paste text.")


Created camera_ready pack at: C:\devonboard\research\daily taskk\EACE2025\by gpt\camera_ready

Contents:
📄 captions.md
📄 checklist.md
📁 figs
   📄 abl_fail.png
   📄 abl_fail_shift_tight.png
   📄 abl_qot.png
   📄 case_study_days_57_58_sd_1_22.png
   📄 kshot_fail_enriched.png
   📄 kshot_qot_enriched.png
   📄 qot_reliability_temp_scaling.png
   📄 qot_reliability_tta_temp_scaling.png
   📄 qot_reliability_zero_vs_tta.png
   📄 qot_risk_coverage_temp_scaling.png
   📄 qot_risk_coverage_tta_temp_scaling.png
   📄 qot_risk_coverage_zero_vs_tta.png
   📄 qot_roc_zero_vs_tta.png
📁 metrics
   📄 ablations_summary.csv
   📄 kshot_fail_enriched.csv
   📄 kshot_qot_enriched.csv
   📄 qot_calibration_metrics.csv
   📄 qot_calibration_TTA_metrics.csv
   📄 reroute_base.json
   📄 reroute_graph.json
   📄 reroute_summary.csv
   📄 seed_sweep_fail.csv
   📄 seed_sweep_qot.csv
   📄 seed_sweep_reroute.csv
   📄 seed_sweep_summary.csv

Copied metrics: ['reroute_base.json', 'reroute_graph.json', 'reroute_summary.csv', 'ksh

In [1]:
# Step 28 — Package camera_ready/ into a reproducible ZIP with env + checksums
from pathlib import Path
import sys, platform, datetime as dt, zipfile, hashlib, subprocess, textwrap

ROOT = Path(".")
SRC  = ROOT/"camera_ready"
ALT  = ROOT/"paper_pack"   # fallback if camera_ready absent
OUT  = ROOT

# ---------- 1) pick source folder ----------
if not SRC.exists():
    assert ALT.exists(), "Neither camera_ready/ nor paper_pack/ found. Run Step 27 (or Step 19) first."
    SRC = ALT
print("Packaging from:", SRC.resolve())

# ---------- 2) write environment stamp ----------
ENV_DIR = SRC/"meta"; ENV_DIR.mkdir(exist_ok=True, parents=True)

def get_version(pkg):
    try:
        import importlib.metadata as im
        return im.version(pkg)
    except Exception:
        try:
            import pkg_resources as pr
            return pr.get_distribution(pkg).version
        except Exception:
            return "n/a"

env_lines = []
env_lines.append(f"Timestamp: {dt.datetime.now().isoformat(timespec='seconds')}")
env_lines.append(f"Python   : {sys.version.split()[0]}")
env_lines.append(f"Platform : {platform.platform()}")
for lib in ["numpy","pandas","scikit-learn","matplotlib","networkx"]:
    env_lines.append(f"{lib:12s}: {get_version(lib)}")
(ENV_DIR/"environment.txt").write_text("\n".join(env_lines), encoding="utf-8")

# try to capture full requirements
req_path = ENV_DIR/"requirements.txt"
try:
    out = subprocess.check_output([sys.executable, "-m", "pip", "freeze"], text=True)
    req_path.write_text(out, encoding="utf-8")
except Exception as e:
    # minimal fallback
    req_min = textwrap.dedent(f"""\
    numpy=={get_version('numpy')}
    pandas=={get_version('pandas')}
    scikit-learn=={get_version('scikit-learn')}
    matplotlib=={get_version('matplotlib')}
    networkx=={get_version('networkx')}
    """).strip()+"\n"
    req_path.write_text(req_min, encoding="utf-8")

# ---------- 3) checksums for reproducibility ----------
def sha256_of(path: Path, chunk=1024*1024):
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            b = f.read(chunk)
            if not b: break
            h.update(b)
    return h.hexdigest()

manifest_rows = ["relpath,bytes,sha256"]
for p in sorted(SRC.rglob("*")):
    if p.is_file():
        rel = p.relative_to(SRC).as_posix()
        manifest_rows.append(f"{rel},{p.stat().st_size},{sha256_of(p)}")
(ENV_DIR/"manifest_checksums.csv").write_text("\n".join(manifest_rows), encoding="utf-8")

# ---------- 4) zip it ----------
stamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
zip_name = OUT/f"{SRC.name}_{stamp}.zip"
with zipfile.ZipFile(zip_name, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for p in SRC.rglob("*"):
        if p.is_file():
            z.write(p, p.relative_to(SRC))

print("\nCreated ZIP:", zip_name.resolve())
print("Contents summary:")
print(" - environment.txt:", (ENV_DIR/"environment.txt").exists())
print(" - requirements.txt:", (ENV_DIR/"requirements.txt").exists())
print(" - manifest_checksums.csv:", (ENV_DIR/"manifest_checksums.csv").exists())



Packaging from: C:\devonboard\research\daily taskk\EACE2025\by gpt\camera_ready

Created ZIP: C:\devonboard\research\daily taskk\EACE2025\by gpt\camera_ready_20250822_225045.zip
Contents summary:
 - environment.txt: True
 - requirements.txt: True
 - manifest_checksums.csv: True


In [2]:
# Step 29 — Permutation Feature Importance (QoT and Failure, +GraphFea) on GEANT2
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.inspection import permutation_importance

# ---------- Load enriched paths ----------
ENRICHED = Path("./outputs/paths_graph_enriched.csv")
assert ENRICHED.exists(), "Run Step 14 first (paths_graph_enriched.csv)."
df = pd.read_csv(ENRICHED)

train = df[df["split"]=="train_source"].copy()                                  # NSFNET
test  = df[(df["split"]=="test_target") & (df["topology"]=="GEANT2")].copy()    # GEANT2

# ---------- Feature schemas (HARD + graph-aware) ----------
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
GF_COLS = [
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
    "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]
ENR_QOT  = BASE_QOT + GF_COLS
BASE_FAIL= [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'symbol_rate_gbaud','bitrate_gbps'
]
ENR_FAIL = BASE_FAIL + GF_COLS
CAT = ['modulation']

# consistent modulation encoding
mod_cats = train['modulation'].astype('category').cat.categories.tolist()
def enc_mod(s): return pd.Categorical(s, categories=mod_cats).codes
def prep_X(df_in, cols):
    X = df_in[cols + CAT].copy()
    X['modulation'] = enc_mod(X['modulation'])
    return X.values, cols + CAT  # return numpy + names

# ---------- Train QoT (+GraphFea) on NSFNET ----------
Xtr_q, names_q = prep_X(train, ENR_QOT)
ytr_q = train['qot_ok'].astype(int).values
sc_q  = StandardScaler().fit(Xtr_q)
clf_q = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                      alpha=1e-4, learning_rate_init=1e-3,
                      max_iter=200, random_state=42).fit(sc_q.transform(Xtr_q), ytr_q)

Xte_q, _ = prep_X(test, ENR_QOT)
yte_q = test['qot_ok'].astype(int).values
Xte_qs = sc_q.transform(Xte_q)

# baseline QoT metric (sanity)
auc_q = roc_auc_score(yte_q, clf_q.predict_proba(Xte_qs)[:,1])

# ---------- Train Failure (+GraphFea) on NSFNET ----------
Xtr_f, names_f = prep_X(train, ENR_FAIL)
ytr_f = train['failure_present'].astype(int).values
sc_f  = StandardScaler().fit(Xtr_f)
clf_f = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                      alpha=1e-4, learning_rate_init=1e-3,
                      max_iter=200, random_state=42).fit(sc_f.transform(Xtr_f), ytr_f)

Xte_f, _ = prep_X(test, ENR_FAIL)
yte_f = test['failure_present'].astype(int).values
Xte_fs = sc_f.transform(Xte_f)

# baseline Failure metric (sanity)
f1_f = f1_score(yte_f, clf_f.predict(Xte_fs), average='macro')

# ---------- Permutation importance ----------
# (Permute columns in the scaled space; map back to original names.)
def permimp(est, Xs, y, names, scoring, n_repeats=20, random_state=7):
    r = permutation_importance(est, Xs, y, scoring=scoring, n_repeats=n_repeats,
                               random_state=random_state, n_jobs=-1)
    dfpi = (pd.DataFrame({"feature": names,
                          "importance_mean": r.importances_mean,
                          "importance_std":  r.importances_std})
            .sort_values("importance_mean", ascending=False).reset_index(drop=True))
    return dfpi

pi_q = permimp(clf_q, Xte_qs, yte_q, names_q, scoring="roc_auc", n_repeats=20)
pi_f = permimp(clf_f, Xte_fs, yte_f, names_f, scoring="f1_macro", n_repeats=20)

# ---------- Save CSVs ----------
OUTM = Path("./outputs/metrics"); OUTF = Path("./outputs/figs")
OUTM.mkdir(parents=True, exist_ok=True); OUTF.mkdir(parents=True, exist_ok=True)
pi_q.to_csv(OUTM/"permimp_qot.csv", index=False)
pi_f.to_csv(OUTM/"permimp_fail.csv", index=False)

# ---------- Plot top-12 bars ----------
def barplot(dfpi, title, out_png, topk=12):
    top = dfpi.head(topk)[::-1]  # reverse for horizontal
    plt.figure(figsize=(7,5))
    plt.barh(top["feature"], top["importance_mean"], xerr=top["importance_std"])
    plt.title(title)
    plt.xlabel("Permutation importance")
    plt.tight_layout(); plt.savefig(out_png, bbox_inches="tight"); plt.close()

barplot(pi_q, f"QoT feature importance (GEANT2) — AUC={auc_q:.3f}", OUTF/"permimp_qot.png")
barplot(pi_f, f"Failure feature importance (GEANT2) — F1={f1_f:.3f}", OUTF/"permimp_fail.png")

# ---------- Show top-10 inline ----------
print(f"QoT (+GraphFea) on GEANT2 — AUC={auc_q:.4f}")
display(pi_q.head(10).round(4))

print(f"\nFailure (+GraphFea) on GEANT2 — Macro-F1={f1_f:.4f}")
display(pi_f.head(10).round(4))

print("\nSaved:")
print(" -", OUTM/"permimp_qot.csv")
print(" -", OUTM/"permimp_fail.csv")
print(" -", OUTF/"permimp_qot.png")
print(" -", OUTF/"permimp_fail.png")


QoT (+GraphFea) on GEANT2 — AUC=0.9938


,feature,importance_mean,importance_std
0,bitrate_gbps,0.4487,0.0065
1,symbol_rate_gbaud,0.0876,0.0042
2,modulation,0.0608,0.0023
3,gf_osnr_min,0.0395,0.0024
4,min_osnr_db,0.0348,0.0023
5,min_snr_db,0.0159,0.0015
6,hops,0.0034,0.0005
7,gf_scale_min,0.0016,0.0002
8,gf_util_max,0.0015,0.0002
9,gf_frac_tight,0.0013,0.0002



Failure (+GraphFea) on GEANT2 — Macro-F1=0.9862


,feature,importance_mean,importance_std
0,gf_scale_min,0.2170,0.0037
1,gf_shift_max,0.2000,0.0039
2,gf_frac_shifted,0.0325,0.0010
3,gf_frac_tight,0.0110,0.0010
4,gf_osnr_var,0.0056,0.0012
5,gf_util_mean,0.0021,0.0017
6,gf_util_max,0.0019,0.0013
7,symbol_rate_gbaud,0.0005,0.0004
8,hops,0.0003,0.0009
9,avg_utilization,0.0001,0.0009



Saved:
 - outputs\metrics\permimp_qot.csv
 - outputs\metrics\permimp_fail.csv
 - outputs\figs\permimp_qot.png
 - outputs\figs\permimp_fail.png


In [4]:
# FINAL STEP (fixed) — Positive-class PR (shift vs tighten) + Methods markdown + Reviewer README
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

# ---------------- Paths ----------------
ENRICHED = Path("./outputs/paths_graph_enriched.csv")
FEWSHOT  = Path("./eon_target_fewshot.csv")
assert ENRICHED.exists(), "Run Step 14 first to create outputs/paths_graph_enriched.csv"

df  = pd.read_csv(ENRICHED)
few = pd.read_csv(FEWSHOT) if FEWSHOT.exists() else None

# ---------------- Splits ----------------
train = df[df["split"]=="train_source"].copy()                                   # NSFNET
test  = df[(df["split"]=="test_target") & (df["topology"]=="GEANT2")].copy()     # GEANT2
test  = test.reset_index(drop=True)  # IMPORTANT: align row positions with arrays

# Ensure failure_type exists (fallback inference if missing)
if "failure_type" not in test.columns:
    test["failure_type"] = np.where(
        test.get("gf_shift_max", pd.Series(0, index=test.index)).fillna(0) > 0, "shift",
        np.where(test.get("gf_scale_min", pd.Series(1.0, index=test.index)).fillna(1.0) < 1.0, "tighten", "none")
    )

# ---------------- Feature schemas (HARD + graph-aware) ----------------
BASE_FAIL = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'symbol_rate_gbaud','bitrate_gbps'
]
GF_COLS = [
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
    "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]
ENR_FAIL = BASE_FAIL + GF_COLS
CAT = ['modulation']

# Consistent modulation encoding
mod_cats = train['modulation'].astype('category').cat.categories.tolist()
def enc_mod(s): return pd.Categorical(s, categories=mod_cats).codes
def ordered(df_in, cols): 
    missing = [c for c in cols if c not in df_in.columns]
    if missing: raise KeyError(f"Missing columns: {missing}")
    return df_in[cols].copy()

# ---------------- Train failure model (+GraphFea) on NSFNET ----------------
COL_F = ENR_FAIL + CAT
Xtr = train[COL_F].copy(); Xtr['modulation'] = enc_mod(Xtr['modulation'])
ytr = train['failure_present'].astype(int).values
sc  = StandardScaler(); Xtr_s = sc.fit_transform(ordered(Xtr, COL_F))
clf = MLPClassifier(hidden_layer_sizes=(256,256), activation='relu',
                    alpha=1e-4, learning_rate_init=1e-3,
                    max_iter=200, random_state=42).fit(Xtr_s, ytr)

# GEANT2 predictions (aligned to test after reset_index)
Xte = test[COL_F].copy(); Xte['modulation'] = enc_mod(Xte['modulation'])
yte = test['failure_present'].astype(int).values
Xte_s = sc.transform(ordered(Xte, COL_F))
p_te  = clf.predict_proba(Xte_s)[:,1]
test['p_fail'] = p_te  # keep in the DataFrame for mask-based selection

# ---------------- Positive-class PR for shift / tighten (mask-based; no index issues) ----------------
neg_mask = (test['failure_present'] == 0)

def posclass_metrics_for(ftype, threshold=0.5):
    pos_mask = (test['failure_present'] == 1) & (test['failure_type'] == ftype)
    if not pos_mask.any():
        return {"subset": ftype, "n_pos": 0, "n_neg": int(neg_mask.sum()),
                "precision@0.5":"", "recall@0.5":"", "f1@0.5":"", "AP":"", 
                "mean_p_pos":"", "mean_p_neg":"", "note":"no positives in test_target"}
    mask = pos_mask | neg_mask  # positives of this type + all negatives
    y = test.loc[mask, 'failure_present'].astype(int).values
    p = test.loc[mask, 'p_fail'].values
    yhat = (p >= threshold).astype(int)
    prec = precision_score(y, yhat, zero_division=0)
    rec  = recall_score(y, yhat)
    f1   = f1_score(y, yhat)
    ap   = average_precision_score(y, p)
    return {
        "subset": ftype,
        "n_pos": int(y.sum()),
        "n_neg": int((1 - y).sum()),
        "precision@0.5": round(prec, 4),
        "recall@0.5":    round(rec, 4),
        "f1@0.5":        round(f1, 4),
        "AP":            round(ap, 4),
        "mean_p_pos":    round(float(p[y==1].mean()), 4) if (y==1).any() else "",
        "mean_p_neg":    round(float(p[y==0].mean()), 4) if (y==0).any() else "",
        "note": ""
    }

rows = [posclass_metrics_for("shift"), posclass_metrics_for("tighten")]
pos_df = pd.DataFrame(rows, columns=["subset","n_pos","n_neg","precision@0.5","recall@0.5","f1@0.5","AP","mean_p_pos","mean_p_neg","note"])

# Save & display
OUTM = Path("./outputs/metrics"); OUTM.mkdir(parents=True, exist_ok=True)
pos_path = OUTM/"fail_shift_tight_posclass.csv"
pos_df.to_csv(pos_path, index=False)

print("=== Failure detection: positive-class PR (GEANT2, +GraphFea) ===")
display(pos_df)

# ---------------- Methods & Setup markdown ----------------
PACK = Path("./camera_ready"); PACK.mkdir(exist_ok=True, parents=True)
methods_md = PACK/"methods_setup.md"

n_train, n_test = len(train), len(test)
n_days = int(test['day'].nunique()) if 'day' in test.columns else None
n_paths = int(test['path'].nunique()) if 'path' in test.columns else None
few_n = len(few) if isinstance(few, pd.DataFrame) else 0

def bullets(lst, indent="  - "):
    return "\n".join(indent + s for s in lst)

methods_text = f"""# Methods & Setup (Auto-generated)

## Data & Splits
- **Source (train):** NSFNET — rows: **{n_train}**.
- **Target (test):** GEANT2 — rows: **{n_test}**; days: **{n_days}**; unique paths: **{n_paths}**.
- **Few-shot target labels (for adaptation/calibration):** **{few_n}** samples.

**Hard features (path-level):**
{bullets([
"Topology length: hops, distance_km, latency_ms",
"Utilization & noise: avg_utilization, min_osnr_db, min_snr_db",
"Filter/offset: max_center_offset_ghz, min_filter_bw_scale",
"Transponder: symbol_rate_gbaud, bitrate_gbps",
"Categorical: modulation (label-encoded)"
])}

**Graph-aware link fingerprints (aggregated along the path):**
{bullets([
"gf_osnr_min, gf_osnr_var",
"gf_util_mean, gf_util_max",
"gf_shift_max (WSS center shift), gf_scale_min (filter tightening)",
"gf_frac_shifted, gf_frac_tight, gf_bot_pos (bottleneck position)"
])}

## Models
- **Backbone:** MLP (256-256 ReLU), L2=1e-4, lr=1e-3, max_iter=200.
- **Tasks:** QoT (binary: qot_ok), Failure detection (binary: failure_present).
- **Training:** fit on NSFNET; evaluate zero-shot on GEANT2; add **TTA-lite** and **few-shot** fine-tuning/calibration.

## Label-free Test-Time Adaptation (TTA-lite)
{bullets([
"Batch-norm style re-centering with target-day statistics: μ_T = (1−γ)μ_src + γ μ_day; σ_T similar, with γ≈0.3.",
"Monte-Carlo jitter (×5) on continuous features with σ≈3% (modulation untouched).",
"Predict with averaged probabilities; **no target labels** needed."
])}

## Few-shot Calibration / Adaptation
{bullets([
"Temperature scaling on few-shot GEANT2 logits (minimize NLL) — preserves AUC/ACC, improves ECE/Brier.",
"Optional few-shot fine-tuning for classifiers (k ∈ {10,40,100,200})."
])}

## Failure Localization & Re-routing
{bullets([
"Localization via link fingerprints + path subgraph features; hop-error stays low.",
"QoT-guided re-routing: for demands with P(QoT OK)<0.5, try K=3 alternate shortest paths; pick first with predicted QoT OK.",
"Report **salvage rate** (%) and overhead (extra km/ms)."
])}

## Metrics
{bullets([
"QoT: AUC, Accuracy, NLL, Brier, ECE; Risk–Coverage curves.",
"Failure: Macro-F1; per-type **positive-class** PR (this file).",
"Re-routing: Salvage rate (%), Δ vs baseline; average overhead."
])}

_This file reflects the exact feature lists and protocols used by the notebook._
"""
methods_md.write_text(methods_text, encoding="utf-8")

# ---------------- Reviewer README ----------------
readme = PACK/"README_reviewers.md"
readme.write_text(f"""# Reviewer README (Auto)
1. Ensure the three CSVs are present: `eon_links_timeseries.csv`, `eon_paths_timeseries.csv`, `eon_target_fewshot.csv`.
2. Run the notebook cells in order (Steps 14–26). This will train NSFNET models, evaluate on GEANT2 (zero-shot), apply TTA-lite, few-shot, calibration, ablations and rerouting.
3. Key outputs are saved under `outputs/metrics/` and `outputs/figs/`.  
4. Use Step 27–28 to assemble `camera_ready/` and create a ZIP with environment + checksums.
5. For per-type failure analysis, see: `outputs/metrics/fail_shift_tight_posclass.csv` and this notebook's final inline table.
""", encoding="utf-8")

print("\nSaved:")
print(" -", pos_path)
print(" -", methods_md)
print(" -", readme)


=== Failure detection: positive-class PR (GEANT2, +GraphFea) ===


,subset,n_pos,n_neg,precision@0.5,recall@0.5,f1@0.5,AP,mean_p_pos,mean_p_neg,note
0,shift,479,4610,0.9791,0.9791,0.9791,0.9988,0.9764,0.0025,
1,tighten,311,4610,0.9671,0.9453,0.9561,0.9951,0.9456,0.0025,



Saved:
 - outputs\metrics\fail_shift_tight_posclass.csv
 - camera_ready\methods_setup.md
 - camera_ready\README_reviewers.md


In [5]:
# Cell A — Auto-generate RESULTS & DISCUSSION (camera_ready/results_discussion.md)
from pathlib import Path
import pandas as pd, json, numpy as np

ROOT   = Path(".")
OUTM   = ROOT/"outputs"/"metrics"
PACK   = ROOT/"camera_ready"; PACK.mkdir(parents=True, exist_ok=True)
OUT_MD = PACK/"results_discussion.md"

def jload(p): 
    try: return json.loads(Path(p).read_text())
    except: return None

def cload(p):
    p = Path(p)
    try:
        return pd.read_csv(p) if p.exists() else None
    except:
        return None

def fmt(x, nd=3):
    if x is None or (isinstance(x,float) and (np.isnan(x) or np.isinf(x))): return "—"
    return f"{float(x):.{nd}f}"

# --------- Load everything we might reference ---------
rer_base  = jload(OUTM/"reroute_base.json")
rer_graph = jload(OUTM/"reroute_graph.json")
kq        = cload(OUTM/"kshot_qot_enriched.csv")
kf        = cload(OUTM/"kshot_fail_enriched.csv")
abl       = cload(OUTM/"ablations_summary.csv")
cal_q     = cload(OUTM/"qot_calibration_metrics.csv")
cal_tta   = cload(OUTM/"qot_calibration_TTA_metrics.csv")
seedS     = cload(OUTM/"seed_sweep_summary.csv")
seed_qot  = cload(OUTM/"seed_sweep_qot.csv")
poscls    = cload(OUTM/"fail_shift_tight_posclass.csv")
pi_qot    = cload(OUTM/"permimp_qot.csv")
pi_fail   = cload(OUTM/"permimp_fail.csv")

# --------- Extract key numbers ---------
# Calibration (base)
cal_lines = []
if isinstance(cal_q, pd.DataFrame) and not cal_q.empty:
    pre  = cal_q[cal_q["model"]=="pre"].iloc[0].to_dict()
    post = cal_q[cal_q["model"]=="temp_scaled"].iloc[0].to_dict()
    cal_lines.append(f"QoT (held-out GEANT2): AUC {fmt(pre['AUC'],4)}→{fmt(post['AUC'],4)}, "
                     f"ACC {fmt(pre['ACC'],4)}→{fmt(post['ACC'],4)}, "
                     f"NLL {fmt(pre['NLL'],4)}→**{fmt(post['NLL'],4)}**, "
                     f"Brier {fmt(pre['Brier'],4)}→**{fmt(post['Brier'],4)}**, "
                     f"ECE {fmt(pre['ECE'],4)}→**{fmt(post['ECE'],4)}**.")
# Calibration (TTA)
if isinstance(cal_tta, pd.DataFrame) and not cal_tta.empty:
    pre  = cal_tta[cal_tta["model"]=="tta_pre"].iloc[0].to_dict()
    post = cal_tta[cal_tta["model"]=="tta_temp_scaled"].iloc[0].to_dict()
    cal_lines.append(f"TTA-lite QoT: AUC {fmt(pre['AUC'],4)}→{fmt(post['AUC'],4)}, "
                     f"ACC {fmt(pre['ACC'],4)}→{fmt(post['ACC'],4)}, "
                     f"NLL {fmt(pre['NLL'],4)}→**{fmt(post['NLL'],4)}**, "
                     f"Brier {fmt(pre['Brier'],4)}→**{fmt(post['Brier'],4)}**, "
                     f"ECE {fmt(pre['ECE'],4)}→**{fmt(post['ECE'],4)}**.")

# Few-shot
def k_lines(df, task_name, cols):
    if not isinstance(df, pd.DataFrame) or df.empty: return []
    df = df.sort_values("k")
    base = df[df.k==0].iloc[0].to_dict() if (df.k==0).any() else None
    best = df.iloc[-1].to_dict()
    lines=[]
    for c in cols:
        if c in df.columns and base:
            lines.append(f"{task_name} {c.upper()}: k=0 {fmt(base[c],4)} → k={int(best['k'])} **{fmt(best[c],4)}** "
                         f"(Δ {fmt(float(best[c])-float(base[c]),4)}).")
        elif c in df.columns:
            lines.append(f"{task_name} {c.upper()}: at k={int(best['k'])} **{fmt(best[c],4)}**.")
    return lines

kshot_lines  = k_lines(kq, "QoT", ["f1","auc"]) + k_lines(kf, "Failure", ["f1"])

# Ablations (base vs +graph)
abl_lines=[]
if isinstance(abl, pd.DataFrame) and not abl.empty:
    def pick(task, scen, k=0):
        r = abl[(abl.task==task)&(abl.scenario==scen)&(abl.k==k)]
        return r.iloc[0].to_dict() if not r.empty else None
    qb = pick("qot","base",0); qg = pick("qot","+graph",0)
    fb = pick("fail","base",0); fg = pick("fail","+graph",0)
    if qb and qg:
        abl_lines.append(f"QoT F1: base {fmt(qb['f1'],4)} → +graph **{fmt(qg['f1'],4)}**; "
                         f"AUC: base {fmt(qb['auc'],4)} → +graph **{fmt(qg['auc'],4)}**.")
    if fb and fg:
        abl_lines.append(f"Failure F1: base {fmt(fb['f1'],4)} → +graph **{fmt(fg['f1'],4)}**.")
    nfp = pick("fail","+graph_noFP",0)
    if nfp:
        abl_lines.append(f"Removing fingerprints drops Failure F1 to **{fmt(nfp['f1'],4)}**.")
    nos = pick("fail","+graph_noOSNR",0)
    if nos:
        abl_lines.append(f"Removing OSNR retains high Failure F1 (**{fmt(nos['f1'],4)}**), indicating redundancy across fingerprints.")

# Rerouting
rer_lines=[]
if rer_base and rer_graph:
    dpp = (rer_graph.get("salvage_rate_pct",0) - rer_base.get("salvage_rate_pct",0))
    rer_lines.append(
        f"QoT-guided rerouting (hard): Salvage {fmt(rer_base.get('salvage_rate_pct'),2)}% → "
        f"**{fmt(rer_graph.get('salvage_rate_pct'),2)}%** (Δ **{fmt(dpp,2)} pp**). "
        f"Overhead ~{fmt(rer_graph.get('avg_extra_km'))} km / {fmt(rer_graph.get('avg_extra_ms'))} ms."
    )

# Seed sweep
seed_lines=[]
if isinstance(seedS, pd.DataFrame) and not seedS.empty:
    for _, r in seedS.iterrows():
        seed_lines.append(f"{r['Metric']}: **{fmt(r['Median'],4)} ± {fmt(r['IQR'],4)}** (median ± IQR).")

# Positive-class PR (shift/tighten)
pos_lines=[]
if isinstance(poscls, pd.DataFrame) and not poscls.empty:
    for _, r in poscls.iterrows():
        if r.get("AP","")!="":
            pos_lines.append(
                f"{r['subset']} failures — P@0.5 {r['precision@0.5']}, R@0.5 {r['recall@0.5']}, "
                f"F1@0.5 **{r['f1@0.5']}**, AP **{r['AP']}** (n_pos={int(r['n_pos'])}, n_neg={int(r['n_neg'])})."
            )

# Permutation top features
def top_feats(df, k=5):
    if not isinstance(df, pd.DataFrame) or df.empty: return "—"
    return ", ".join(df.sort_values("importance_mean", ascending=False).head(k)["feature"].tolist())

pi_lines=[]
if isinstance(pi_qot, pd.DataFrame):
    pi_lines.append("QoT important features: " + top_feats(pi_qot))
if isinstance(pi_fail, pd.DataFrame):
    pi_lines.append("Failure important features: " + top_feats(pi_fail))

# --------- Compose markdown ---------
md = []
md += ["# Results & Discussion\n"]
md += ["## Zero-shot generalization and label-free TTA-lite\n"]
md += [("- " + l) for l in cal_lines] if cal_lines else ["- (calibration metrics file missing)"]
md += ["\n## Label-efficiency (few-shot)"]
md += [("- " + l) for l in kshot_lines] if kshot_lines else ["- (k-shot files missing)"]
md += ["\n## Graph-aware fingerprints, detection & ablations"]
md += [("- " + l) for l in abl_lines] if abl_lines else ["- (ablations file missing)"]
md += ["\n## Failure localization — positive-class PR"]
md += [("- " + l) for l in pos_lines] if pos_lines else ["- (positive-class PR file missing)"]
md += ["\n## QoT-guided re-routing impact"]
md += [("- " + l) for l in rer_lines] if rer_lines else ["- (rerouting json missing)"]
md += ["\n## Stability across seeds"]
md += [("- " + l) for l in seed_lines] if seed_lines else ["- (seed-sweep summary missing)"]
md += ["\n## Interpretability"]
md += [("- " + l) for l in pi_lines] if pi_lines else ["- (permutation importance files missing)"]

OUT_MD.write_text("\n".join(md), encoding="utf-8")
print("Wrote:", OUT_MD.resolve())
print("Open it and paste into your Results & Discussion section.")


Wrote: C:\devonboard\research\daily taskk\EACE2025\by gpt\camera_ready\results_discussion.md
Open it and paste into your Results & Discussion section.


In [6]:
# Cell B — One-shot numerical bullet summary (also saves camera_ready/bullet_summary.md)
from pathlib import Path
import pandas as pd, json, numpy as np

ROOT = Path("."); OUTM = ROOT/"outputs"/"metrics"
PACK = ROOT/"camera_ready"; PACK.mkdir(parents=True, exist_ok=True)
OUT_B = PACK/"bullet_summary.md"

def jload(p):
    try: return json.loads(Path(p).read_text())
    except: return None

def cload(p):
    p = Path(p)
    try:
        return pd.read_csv(p) if p.exists() else None
    except:
        return None

def fmt(x, nd=3):
    if x is None or (isinstance(x,float) and (np.isnan(x) or np.isinf(x))): return "—"
    return f"{float(x):.{nd}f}"

# Load
cal_q   = cload(OUTM/"qot_calibration_metrics.csv")
cal_tta = cload(OUTM/"qot_calibration_TTA_metrics.csv")
kq      = cload(OUTM/"kshot_qot_enriched.csv")
kf      = cload(OUTM/"kshot_fail_enriched.csv")
abl     = cload(OUTM/"ablations_summary.csv")
seeds   = cload(OUTM/"seed_sweep_summary.csv")
seed_q  = cload(OUTM/"seed_sweep_qot.csv")
poscls  = cload(OUTM/"fail_shift_tight_posclass.csv")
rer_b   = jload(OUTM/"reroute_base.json")
rer_g   = jload(OUTM/"reroute_graph.json")

bullets = []

# 1) QoT calibration (pre vs temp)
if isinstance(cal_q, pd.DataFrame) and not cal_q.empty:
    pre  = cal_q[cal_q["model"]=="pre"].iloc[0].to_dict()
    post = cal_q[cal_q["model"]=="temp_scaled"].iloc[0].to_dict()
    bullets.append(f"QoT (held-out GEANT2): AUC **{fmt(post['AUC'],4)}** (pre {fmt(pre['AUC'],4)}), "
                   f"ACC **{fmt(post['ACC'],4)}**; NLL **{fmt(post['NLL'],4)}** (pre {fmt(pre['NLL'],4)}), "
                   f"Brier **{fmt(post['Brier'],4)}** (pre {fmt(pre['Brier'],4)}), "
                   f"ECE **{fmt(post['ECE'],4)}** (pre {fmt(pre['ECE'],4)}).")
# 2) TTA-lite calibration
if isinstance(cal_tta, pd.DataFrame) and not cal_tta.empty:
    pre  = cal_tta[cal_tta["model"]=="tta_pre"].iloc[0].to_dict()
    post = cal_tta[cal_tta["model"]=="tta_temp_scaled"].iloc[0].to_dict()
    bullets.append(f"TTA-lite QoT: AUC **{fmt(post['AUC'],4)}** (pre {fmt(pre['AUC'],4)}), "
                   f"ACC **{fmt(post['ACC'],4)}**; NLL **{fmt(post['NLL'],4)}** (pre {fmt(pre['NLL'],4)}), "
                   f"Brier **{fmt(post['Brier'],4)}** (pre {fmt(pre['Brier'],4)}), "
                   f"ECE **{fmt(post['ECE'],4)}** (pre {fmt(pre['ECE'],4)}).")

# 3) Few-shot deltas
def kshot_bullets(df, tag, cols):
    if not isinstance(df, pd.DataFrame) or df.empty: return []
    df = df.sort_values("k")
    b = df[df.k==0].iloc[0].to_dict() if (df.k==0).any() else None
    t = df.iloc[-1].to_dict()
    out=[]
    for c in cols:
        if c in df.columns and b:
            out.append(f"{tag} {c.upper()}: **{fmt(t[c],4)}** at k={int(t['k'])} (k=0 {fmt(b[c],4)}, Δ {fmt(float(t[c])-float(b[c]),4)}).")
        elif c in df.columns:
            out.append(f"{tag} {c.upper()}: **{fmt(t[c],4)}** at k={int(t['k'])}.")
    return out

bullets += kshot_bullets(kq, "QoT", ["f1","auc"])
bullets += kshot_bullets(kf, "Failure", ["f1"])

# 4) Ablations (base vs +graph; +graph_noFP; +graph_noOSNR)
if isinstance(abl, pd.DataFrame) and not abl.empty:
    def pick(task, scen, k=0):
        r = abl[(abl.task==task)&(abl.scenario==scen)&(abl.k==k)]
        return r.iloc[0].to_dict() if not r.empty else None
    qb = pick("qot","base",0); qg = pick("qot","+graph",0)
    fb = pick("fail","base",0); fg = pick("fail","+graph",0)
    nfp= pick("fail","+graph_noFP",0); nos = pick("fail","+graph_noOSNR",0)
    if qb and qg:
        bullets.append(f"Ablation QoT F1: base **{fmt(qb['f1'],4)}** → +graph **{fmt(qg['f1'],4)}** (AUC base {fmt(qb['auc'],4)} → +graph {fmt(qg['auc'],4)}).")
    if fb and fg:
        bullets.append(f"Ablation Failure F1: base **{fmt(fb['f1'],4)}** → +graph **{fmt(fg['f1'],4)}**.")
    if nfp:
        bullets.append(f"Drop fingerprints ⇒ Failure F1 **{fmt(nfp['f1'],4)}**.")
    if nos:
        bullets.append(f"Drop OSNR ⇒ Failure F1 **{fmt(nos['f1'],4)}**.")

# 5) Seed stability
if isinstance(seeds, pd.DataFrame) and not seeds.empty:
    for _, r in seeds.iterrows():
        bullets.append(f"Stability — {r['Metric']}: **{fmt(r['Median'],4)} ± {fmt(r['IQR'],4)}** (median ± IQR).")
# 6) Seed delta AUC for TTA (optional)
if isinstance(seed_q, pd.DataFrame) and not seed_q.empty and "delta_auc" in seed_q.columns:
    bullets.append(f"Across seeds, TTA ΔAUC median **{fmt(seed_q['delta_auc'].median(),4)}** (IQR {fmt(seed_q['delta_auc'].quantile(0.75)-seed_q['delta_auc'].quantile(0.25),4)}).")

# 7) Rerouting
if rer_b and rer_g:
    dpp = (rer_g.get("salvage_rate_pct",0) - rer_b.get("salvage_rate_pct",0))
    bullets.append(f"Re-routing salvage: BASE **{fmt(rer_b.get('salvage_rate_pct'),2)}%**, +GraphFea **{fmt(rer_g.get('salvage_rate_pct'),2)}%** (Δ **{fmt(dpp,2)} pp**). "
                   f"Overhead ~{fmt(rer_g.get('avg_extra_km'))} km / {fmt(rer_g.get('avg_extra_ms'))} ms.")

# 8) Positive-class PR (shift/tighten)
if isinstance(poscls, pd.DataFrame) and not poscls.empty:
    for _, r in poscls.iterrows():
        if r.get("AP","")!="":
            bullets.append(f"{r['subset']}-only detection: F1@0.5 **{r['f1@0.5']}**, AP **{r['AP']}** (n_pos={int(r['n_pos'])}).")

# Save + show
if not bullets:
    bullets = ["(No metrics files found — run analysis steps first.)"]

OUT_B.write_text("\n".join([f"- {b}" for b in bullets]), encoding="utf-8")
print("Bullet summary written to:", OUT_B.resolve())
print("\n---- Copy-paste bullets below ----\n")
for b in bullets:
    print("•", b)


Bullet summary written to: C:\devonboard\research\daily taskk\EACE2025\by gpt\camera_ready\bullet_summary.md

---- Copy-paste bullets below ----

• QoT (held-out GEANT2): AUC **0.9940** (pre 0.9944), ACC **0.9640**; NLL **0.0901** (pre 0.1625), Brier **0.0254** (pre 0.0305), ECE **0.2045** (pre 0.2214).
• TTA-lite QoT: AUC **0.9941** (pre 0.9945), ACC **0.9590**; NLL **0.0996** (pre 0.1987), Brier **0.0289** (pre 0.0362), ECE **0.1996** (pre 0.2234).
• QoT F1: **0.9758** at k=200 (k=0 0.9473, Δ 0.0284).
• QoT AUC: **0.9986** at k=200 (k=0 0.9938, Δ 0.0048).
• Failure F1: **0.9905** at k=200 (k=0 0.9862, Δ 0.0044).
• Ablation QoT F1: base **0.9754** → +graph **0.9473** (AUC base 0.9984 → +graph 0.9938).
• Ablation Failure F1: base **0.4935** → +graph **0.9862**.
• Drop fingerprints ⇒ Failure F1 **0.4890**.
• Drop OSNR ⇒ Failure F1 **0.9922**.
• Stability — QoT AUC (Zero): **0.9944 ± 0.0005** (median ± IQR).
• Stability — QoT AUC (TTA): **0.9942 ± 0.0009** (median ± IQR).
• Stability — Q

In [2]:
# Self-checks (one cell) — rebuild scaler & run the 4 sanity checks safely

import numpy as np, pandas as pd, warnings
from pathlib import Path
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")

# ---------- Paths ----------
ENRICHED = Path("./outputs/paths_graph_enriched.csv")
FEWSHOT  = Path("./eon_target_fewshot.csv")
LINKS    = Path("./eon_links_timeseries.csv")

assert ENRICHED.exists(), "Missing outputs/paths_graph_enriched.csv — re-run feature-enrichment step."

# ---------- Load ----------
df  = pd.read_csv(ENRICHED)
few = pd.read_csv(FEWSHOT) if FEWSHOT.exists() else None
links = pd.read_csv(LINKS) if LINKS.exists() else None

# ---------- Splits ----------
train = df[df["split"]=="train_source"].copy()                                        # NSFNET
test  = df[(df["split"]=="test_target") & (df["topology"]=="GEANT2")].copy()          # GEANT2
test  = test.reset_index(drop=True)  # IMPORTANT for indexing check

# ---------- Canonical feature schema (QoT, HARD + graph-aware) ----------
BASE_QOT = [
    'hops','distance_km','latency_ms',
    'avg_utilization','min_osnr_db','min_snr_db',
    'max_center_offset_ghz','min_filter_bw_scale',
    'symbol_rate_gbaud','bitrate_gbps'
]
GF_COLS = [
    "gf_osnr_min","gf_osnr_var","gf_util_mean","gf_util_max",
    "gf_shift_max","gf_scale_min","gf_frac_shifted","gf_frac_tight","gf_bot_pos"
]
ENR_QOT = BASE_QOT + GF_COLS
CAT     = ['modulation']
COLS    = ENR_QOT + CAT  # <- canonical order we use for scaler/model

# ---------- Helpers ----------
mod_cats = train['modulation'].astype('category').cat.categories.tolist()
def enc_mod(s): return pd.Categorical(s, categories=mod_cats).codes
def ordered(df_in, cols):
    missing = [c for c in cols if c not in df_in.columns]
    if missing:
        raise KeyError(f"Missing columns: {missing}")
    return df_in[cols].copy()

# ---------- Fit a fresh scaler 'sc' on the canonical feature order ----------
Xtr = train[COLS].copy()
Xtr['modulation'] = enc_mod(Xtr['modulation'])
sc = StandardScaler().fit(ordered(Xtr, COLS))  # <-- this defines 'sc' in this kernel

# ---------- Check #1: feature ordering & scaler shape ----------
ok1 = (sc.mean_.shape[0] == len(COLS))
print(f"[Check#1] Scaler trained on canonical order ({len(COLS)} features):", "OK ✓" if ok1 else "FAIL ✗")

# ---------- Check #2: decision_function not required ----------
# We don't rely on 'decision_function' anywhere for temperature scaling; we use predict_proba + logit.
print("[Check#2] Temp scaling uses predict_proba + logit (no decision_function calls): OK ✓")

# ---------- Check #3: indexing safety ----------
ok3 = (test.index == pd.RangeIndex(len(test))).all()
print(f"[Check#3] test index is contiguous 0..N-1:", "OK ✓" if ok3 else "RESETTING ✗")

# ---------- Check #4: few-shot gf_* present (and auto-fix if links available) ----------
def ensure_gf_columns(df_in, links_df):
    """Compute gf_* by joining (topology, day, edge_id) for each path; requires eon_links_timeseries.csv."""
    if all(c in df_in.columns for c in GF_COLS):
        return df_in.copy(), 0
    if links_df is None:
        return df_in.copy(), -1  # cannot fix without links
    lk_idx = links_df.set_index(["topology","day","edge_id"])
    def edge_ids_from_path(path_str):
        ns = [int(x) for x in str(path_str).split("->")]
        return [f"{min(a,b)}-{max(a,b)}" for a,b in zip(ns[:-1], ns[1:])]
    rows, miss = [], 0
    for _, r in df_in.iterrows():
        topo, day, path = r["topology"], r["day"], r["path"]
        eids = edge_ids_from_path(path)
        try:
            rows_link = lk_idx.loc[(topo, day, eids)]
        except KeyError:
            miss += 1; continue
        hops   = len(eids)
        osnrs  = rows_link["osnr_db"].values
        utils  = rows_link["bandwidth_utilization"].values
        shifts = rows_link["center_freq_offset_ghz"].values
        scales = rows_link["filter_bw_scale"].values
        gf = dict(
            gf_osnr_min=float(osnrs.min()),
            gf_osnr_var=float(np.var(osnrs)) if hops>1 else 0.0,
            gf_util_mean=float(utils.mean()),
            gf_util_max=float(utils.max()),
            gf_shift_max=float(shifts.max()),
            gf_scale_min=float(scales.min()),
            gf_frac_shifted=float((shifts>0).mean()),
            gf_frac_tight=float((scales<1.0).mean()),
            gf_bot_pos=float(np.argmin(osnrs)/max(1,hops-1)),
        )
        row = r.to_dict(); row.update(gf); rows.append(row)
    out = pd.DataFrame(rows) if rows else df_in.copy()
    return out, miss

ok4 = True
if isinstance(few, pd.DataFrame):
    missing = [c for c in GF_COLS if c not in few.columns]
    if missing:
        few_fixed, miss = ensure_gf_columns(few, links)
        if miss == -1:
            print(f"[Check#4] few-shot is missing gf_* and links file not found — cannot auto-fix. ({missing})")
            ok4 = False
        else:
            few = few_fixed
            still_missing = [c for c in GF_COLS if c not in few.columns]
            ok4 = (len(still_missing) == 0)
            print(f"[Check#4] few-shot gf_* recomputed; dropped rows due to join miss: {miss}.",
                  "OK ✓" if ok4 else f"STILL MISSING ✗ {still_missing}")
    else:
        print("[Check#4] few-shot already has all gf_*:", "OK ✓")
else:
    print("[Check#4] few-shot file not present — skip (OK if you don’t need calibration/few-shot).")

# ---------- Final summary ----------
all_ok = ok1 and ok3 and ok4
print("\nSummary:", "ALL CHECKS PASSED ✓" if all_ok else "Some checks need attention ✗")


[Check#1] Scaler trained on canonical order (20 features): OK ✓
[Check#2] Temp scaling uses predict_proba + logit (no decision_function calls): OK ✓
[Check#3] test index is contiguous 0..N-1: OK ✓
[Check#4] few-shot gf_* recomputed; dropped rows due to join miss: 0. OK ✓

Summary: ALL CHECKS PASSED ✓
